Bibliotecas

# Nova seção

Instalando Bibliotecas

In [1]:
import requests
import zipfile
import os
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from docx import Document
from tqdm import tqdm
from datetime import datetime
import shutil
import numpy as np


Limpando os diretórios

In [ ]:
import os
import shutil

# Diretórios que deseja limpar
diretorios = [
    r"C:\Users\User\Desktop\valuation_cvm\valuation_cvm_clone\exports\arquivos_cvm_zip",
    r"C:\Users\User\Desktop\valuation_cvm\valuation_cvm_clone\exports\descompactados",
    r"C:\Users\User\Desktop\valuation_cvm\valuation_cvm_clone\exports\relatorios_trimestrais"
]

def limpar_diretorios(diretorios):
    for diretorio in diretorios:
        try:
            # Verificar se o diretório existe
            if os.path.exists(diretorio):
                # Listar todos os arquivos e subdiretórios no diretório
                for filename in os.listdir(diretorio):
                    file_path = os.path.join(diretorio, filename)
                    try:
                        # Verificar se é um arquivo ou diretório e remover
                        if os.path.isfile(file_path) or os.path.islink(file_path):
                            os.unlink(file_path)  # Remove arquivos
                        elif os.path.isdir(file_path):
                            shutil.rmtree(file_path)  # Remove diretórios
                    except Exception as e:
                        print(f'Erro ao excluir {file_path}. Razão: {e}')
            else:
                print(f'Diretório {diretorio} não encontrado.')
        except Exception as e:
            print(f'Erro ao acessar {diretorio}. Razão: {e}')

# Executar a função para limpar os diretórios
limpar_diretorios(diretorios)

print("Diretórios limpos com sucesso.")

Diretórios limpos com sucesso.


## **Extrair os arquivos da CVM**

Diretórios necessários para salvar/obter arquivos que serão gerados/utilizados no código

In [4]:
#Arquivos Históricos
# Diretório de destino para salvar os arquivos ZIP que serão exrtraídos da URL
diretorio_destino_arq_zip = r"C:\Users\User\Desktop\valuation_cvm\valuation_cvm_clone\exports\arquivos_cvm_zip"

# Diretório para salvar os arquivos descompactados (após o download é necessário descompactar por estarem no formato .zip)
diretorio_destino_arq_csv = r"C:\Users\User\Desktop\valuation_cvm\valuation_cvm_clone\exports\descompactados"

#Arquivos Trimestrais (local onde serão salvos os arquivos em formato .zip e .csv após descompactar)
diretorio_destino_arq_trim = r"C:\Users\User\Desktop\valuation_cvm\valuation_cvm_clone\exports\relatorios_trimestrais"
#diretório para salvar as listas para CNPJ (pasta anexa para salvar a lista obtida com os nomes das empresas e respectivos cnpjs obtidos)
diretório_destino_lista = r"C:\Users\User\Desktop\valuation_cvm\valuation_cvm_clone\exports\lista_cnpjs"

# Caminho para os arquivos DOCX (local onde foram salvos os arquivos .docx criados para guardar os elementos extraídos da B3 com os nomes das empresas dispensadas da CVM)
caminho_arq_docx_emp_dispensadas = r"C:\Users\User\Desktop\valuation_cvm\valuation_cvm_clone\imports\empresas_dispensadas_cvm"

# Caminho para os arquivos DOCX (local onde foram salvos os arquivos .docx criados para guardar os elementos extraídos da B3 com os nomes das empresas não dispensadas e local para salvar a lista das empresas após extração do HTML)
caminho_arq_docx_emp_list_b3 = Document(r"C:\Users\User\Desktop\valuation_cvm\valuation_cvm_clone\imports\elemento_html_b3 (3).docx")
arq_tentativa_extracao_url = r"C:\Users\User\Desktop\valuation_cvm\valuation_cvm_clone\imports\empresas_listadas_b3"

#Caminho para salvar os relatórios de indicadores financeiros
diretorio_rel_ind_financeiros = r"C:\Users\User\Desktop\valuation_cvm\valuation_cvm_clone\exports\rel_indicadores_financeiros"

In [5]:
# Anos em que os relatórios serão obtidos
ano_inicial = 2012
ano_final = 2023

In [6]:
url_base = 'http://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DADOS'

# Diretório de destino para salvar os arquivos ZIP
diretorio_destino = diretorio_destino_arq_zip

# Diretório para salvar os arquivos descompactados
diretorio_descompactados = diretorio_destino_arq_csv

# Verificando se o diretório de destino existe
os.makedirs(diretorio_destino, exist_ok=True)

for ano in range(ano_inicial, ano_final + 1):
    relatorio = "dfp_cia_aberta"
    arquivo_zip = f'{relatorio}_{ano}.zip'
    caminho_arquivo_local = os.path.join(diretorio_destino, arquivo_zip)

    # Verificando se o arquivo já existe no diretório de destino
    if os.path.exists(caminho_arquivo_local):
        print(f"Arquivo {arquivo_zip} já existe. Pulando o download e extração.")
    else:
        url_arquivo = f'{url_base}/{arquivo_zip}'

        # Download do arquivo ZIP
        response = requests.get(url_arquivo)
        if response.status_code == 200:
            with open(caminho_arquivo_local, 'wb') as arquivo_local:
                arquivo_local.write(response.content)

            # Extração do conteúdo do arquivo ZIP
            with zipfile.ZipFile(caminho_arquivo_local, 'r') as zip_ref:
                # Coletar os nomes dos arquivos extraídos
                lista_arquivos_zip = zip_ref.namelist()

                # Verifica se os arquivos CSV já existem na pasta do ano
                csv_faltantes = [f'{relatorio}_{ano}.csv' for arquivo in lista_arquivos_zip]
                csv_faltantes = [csv for csv in csv_faltantes if not os.path.exists(os.path.join(diretorio_descompactados, str(ano), csv))]

                if csv_faltantes:
                    # Cria um diretório para cada ano
                    diretorio_ano = os.path.join(diretorio_descompactados, str(ano))
                    os.makedirs(diretorio_ano, exist_ok=True)

                    # Exibindo a lista de nomes dos arquivos extraídos
                    print("Arquivos no arquivo ZIP:")
                    for arquivo in lista_arquivos_zip:
                        print(arquivo)

                        # Determinando qual arquivo do ZIP será lido.
                        with zip_ref.open(arquivo) as arquivo_zip:
                            df = pd.read_csv(arquivo_zip, encoding='cp1252', sep=';')

                            # Salva o DataFrame como um arquivo CSV na pasta do ano correspondente
                            caminho_arquivo_dataframe = os.path.join(diretorio_ano, f'{arquivo[:-4]}.csv')
                            df.to_csv(caminho_arquivo_dataframe, index=False)
                            print(f"DataFrame salvo em: {caminho_arquivo_dataframe}")
                else:
                    print(f"Arquivos CSV já existem na pasta {diretorio_ano}. Pulando a extração.")
        else:
            print(f"Falha ao baixar {arquivo_zip}. Código de status:", response.status_code)

Arquivos no arquivo ZIP:
dfp_cia_aberta_2012.csv
DataFrame salvo em: C:\Users\User\Desktop\valuation_cvm\valuation_cvm_clone\exports\descompactados\2012\dfp_cia_aberta_2012.csv
dfp_cia_aberta_BPA_con_2012.csv
DataFrame salvo em: C:\Users\User\Desktop\valuation_cvm\valuation_cvm_clone\exports\descompactados\2012\dfp_cia_aberta_BPA_con_2012.csv
dfp_cia_aberta_BPA_ind_2012.csv
DataFrame salvo em: C:\Users\User\Desktop\valuation_cvm\valuation_cvm_clone\exports\descompactados\2012\dfp_cia_aberta_BPA_ind_2012.csv
dfp_cia_aberta_BPP_con_2012.csv
DataFrame salvo em: C:\Users\User\Desktop\valuation_cvm\valuation_cvm_clone\exports\descompactados\2012\dfp_cia_aberta_BPP_con_2012.csv
dfp_cia_aberta_BPP_ind_2012.csv
DataFrame salvo em: C:\Users\User\Desktop\valuation_cvm\valuation_cvm_clone\exports\descompactados\2012\dfp_cia_aberta_BPP_ind_2012.csv
dfp_cia_aberta_DFC_MD_con_2012.csv
DataFrame salvo em: C:\Users\User\Desktop\valuation_cvm\valuation_cvm_clone\exports\descompactados\2012\dfp_cia_aber

Relatório trimestral ano corrente

In [7]:
# Configura o ano corrente
ano_corrente = ano_final  # Supondo que ano_final esteja definido anteriormente
ano_proximo = ano_corrente + 1

# Caminho para salvar os arquivos extraídos
save_folder = diretorio_destino_arq_trim

# Função para download e extração do arquivo ZIP
def baixar_e_extrair_arquivo(ano):
    file_url = f'https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/itr_cia_aberta_{ano}.zip'
    zip_file_path = os.path.join(save_folder, f'itr_cia_aberta_{ano}.zip')
    extracted_folder_path = os.path.join(save_folder, f'itr_cia_aberta_{ano}')

    # Verifica se o arquivo zip já existe no diretório
    if not os.path.exists(zip_file_path):
        # Faz o download do arquivo ZIP caso não exista
        response = requests.get(file_url)
        if response.status_code == 200:
            with open(zip_file_path, 'wb') as f:
                f.write(response.content)
            print(f"Download do arquivo ZIP para o ano {ano} concluído com sucesso!")
        else:
            print(f"Falha ao baixar o arquivo ZIP para o ano {ano}.")
    else:
        print(f"Arquivo ZIP para o ano {ano} já existe no diretório. Pulando o download.")

    # Verifica se o arquivo zip existe no diretório
    if os.path.exists(zip_file_path):
        # Extrai todos os arquivos do ZIP para um diretório temporário
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(save_folder)

        # Verifica e atualiza os arquivos se houver modificações
        if os.path.exists(extracted_folder_path):
            for root, dirs, files in os.walk(extracted_folder_path):
                for file in files:
                    extracted_file_path = os.path.join(root, file)
                    target_file_path = os.path.join(save_folder, file)
                    if not os.path.exists(target_file_path):
                        shutil.move(extracted_file_path, save_folder)
                        print(f"O arquivo '{file}' do ano {ano} foi adicionado.")
                    else:
                        zip_file_mod_time = datetime.fromtimestamp(os.path.getmtime(extracted_file_path))
                        target_file_mod_time = datetime.fromtimestamp(os.path.getmtime(target_file_path))
                        if zip_file_mod_time > target_file_mod_time:
                            os.remove(target_file_path)
                            shutil.move(extracted_file_path, save_folder)
                            print(f"O arquivo '{file}' do ano {ano} foi atualizado.")
                        else:
                            os.remove(extracted_file_path)

        # Remove o diretório temporário extraído do ZIP
        shutil.rmtree(extracted_folder_path, ignore_errors=True)
        print(f"Verificação e atualização para o ano {ano} concluídas.")
    else:
        print(f"Arquivo ZIP para o ano {ano} não encontrado no diretório.")

# Baixar e extrair os arquivos para o ano corrente e o próximo ano
baixar_e_extrair_arquivo(ano_corrente)
baixar_e_extrair_arquivo(ano_proximo)


Download do arquivo ZIP para o ano 2023 concluído com sucesso!
Verificação e atualização para o ano 2023 concluídas.
Download do arquivo ZIP para o ano 2024 concluído com sucesso!
Verificação e atualização para o ano 2024 concluídas.


Visualizando os arquivos obtidos

In [8]:
#Lista com todos os arquivos no diretório disponíveis para visualização
ano = ano_final
arquivos_disponiveis = os.listdir(os.path.join(diretorio_descompactados, str(ano)))

# Exibe a lista de arquivos disponíveis
for arquivo in arquivos_disponiveis:
    print(arquivo)

dfp_cia_aberta_2023.csv
dfp_cia_aberta_BPA_con_2023.csv
dfp_cia_aberta_BPA_ind_2023.csv
dfp_cia_aberta_BPP_con_2023.csv
dfp_cia_aberta_BPP_ind_2023.csv
dfp_cia_aberta_DFC_MD_con_2023.csv
dfp_cia_aberta_DFC_MD_ind_2023.csv
dfp_cia_aberta_DFC_MI_con_2023.csv
dfp_cia_aberta_DFC_MI_ind_2023.csv
dfp_cia_aberta_DMPL_con_2023.csv
dfp_cia_aberta_DMPL_ind_2023.csv
dfp_cia_aberta_DRA_con_2023.csv
dfp_cia_aberta_DRA_ind_2023.csv
dfp_cia_aberta_DRE_con_2023.csv
dfp_cia_aberta_DRE_ind_2023.csv
dfp_cia_aberta_DVA_con_2023.csv
dfp_cia_aberta_DVA_ind_2023.csv
dfp_cia_aberta_parecer_2023.csv


In [9]:
# Definindo relatório a ser aberto
relatorio = "dfp_cia_aberta_BPP_con" #inserir apenas o nome do arquivo, sem o ano

# Definindo o caminho do arquivo usando formatação de string
caminho_arquivo = f'{diretorio_destino_arq_csv}/{ano}/{relatorio}_{ano}.csv'

df = pd.read_csv(caminho_arquivo, encoding='utf-8', sep=',')




===============================================================================

================================================================================

## **Obtenção das variáveis dentros dos arquivos**

Visualização

In [10]:
# Definindo relatório a ser aberto
relatorio = "dfp_cia_aberta_BPP_con" #inserir apenas o nome do arquivo, sem o ano

# Definindo o caminho do arquivo usando formatação de string
caminho_arquivo = f'{diretorio_destino_arq_csv}/{ano}/{relatorio}_{ano}.csv'

df = pd.read_csv(caminho_arquivo, encoding='utf-8', sep=',')

lista_ds_conta = df[['CD_CONTA', 'DS_CONTA']].drop_duplicates()

lista_ds_conta = lista_ds_conta.sort_values(by=['DS_CONTA'])

df_lista = pd.DataFrame(lista_ds_conta)

df_lista


,CD_CONTA,DS_CONTA
53222,2.03.01.02,( - ) Ações em tesouraria
67454,2.03.01.02,( - ) Custo de emissão de ações
67470,2.03.02.07,( - ) Custo de emissão de ações
13754,2.03.01.02,(-) Capital Social a Integralizar
30742,2.03.01.02,(-) Capital social a integralizar
...,...,...
86466,2.03.02.08,Ágio na venda de ações em tesouraria
59814,2.03.08.01,Ágio/Deságio em Transação de Capital
81302,2.03.02.10,Ágio/deságio em transação de capital
10630,2.02.02.02.06,Ônus da concessão


Receita Líquida

Histórico

In [11]:
# Definindo relatório a ser aberto
relatorio = "dfp_cia_aberta_DRE_con"  # inserir apenas o nome do arquivo, sem o ano

# Lista de anos desejados
anos = list(range(ano_inicial, ano_final + 1))

# DataFrame vazio para armazenar os resultados
resultados_dre = pd.DataFrame()

# Loop pelos anos
for ano in anos:
    # Definindo o caminho do arquivo usando formatação de string
    caminho_arquivo = f'{diretorio_destino_arq_csv}/{ano}/{relatorio}_{ano}.csv'

    # Lendo o arquivo CSV
    df = pd.read_csv(caminho_arquivo, encoding='utf-8', sep=',')

    # Filtrando os dados para encontrar a Receita Operacional Líquida
    var_DRE = df[(df['ORDEM_EXERC'] == 'ÚLTIMO') & (df['CD_CONTA'] == '3.01')]

    # Selecionando colunas desejadas
    var_DRE = var_DRE[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA']]

    # Adicionando a coluna 'Ano' com o valor correspondente
    var_DRE['Ano'] = ano

    # Adicionando os resultados ao DataFrame final
    resultados_dre = pd.concat([resultados_dre, var_DRE], ignore_index=True)

# Agregando os valores duplicados
resultados_agregados_dre = resultados_dre.groupby(['DENOM_CIA', 'CNPJ_CIA', 'Ano'])['VL_CONTA'].sum().reset_index()

# Criando um DataFrame pivotado
receita_operacional = resultados_agregados_dre.pivot_table(index=['DENOM_CIA', 'CNPJ_CIA'], columns='Ano', values='VL_CONTA', aggfunc='sum').reset_index()

# Renomeando as colunas para remover o nome da coluna do índice
receita_operacional.name = None
receita_liquida = receita_operacional



Trimestral

In [12]:
# Definindo o caminho do arquivo CSV
caminho_arquivo_tri_prox = f'{diretorio_destino_arq_trim}/itr_cia_aberta_DRE_con_{ano_proximo}.csv'

# Lendo o arquivo CSV e ignorando as linhas com formatação incorreta
df_tri_prox = pd.read_csv(caminho_arquivo_tri_prox, encoding='cp1252', sep=';', on_bad_lines='skip')

# Filtrando os dados para encontrar a Receita Operacional Líquida do Período
receita_liquida_tri_prox = df_tri_prox[(df_tri_prox['ORDEM_EXERC'] == 'ÚLTIMO') & (df_tri_prox['CD_CONTA'] == '3.01')]

# Converter as colunas 'DT_INI_EXERC' e 'DT_FIM_EXERC' para o tipo datetime
receita_liquida_tri_prox['DT_INI_EXERC'] = pd.to_datetime(receita_liquida_tri_prox['DT_INI_EXERC'])
receita_liquida_tri_prox['DT_FIM_EXERC'] = pd.to_datetime(receita_liquida_tri_prox['DT_FIM_EXERC'])


# Calcular a diferença entre DT_FIM_EXERC e DT_INI_EXERC
receita_liquida_tri_prox['DIFERENCA'] = (receita_liquida_tri_prox['DT_FIM_EXERC'] - receita_liquida_tri_prox['DT_INI_EXERC']).abs()

# Encontrar os índices da menor diferença para cada grupo de DENOM_CIA, CNPJ_CIA, DT_FIM_EXERC
indices_min_DT_INI = receita_liquida_tri_prox.groupby(['DENOM_CIA', 'CNPJ_CIA', 'DT_FIM_EXERC'])['DIFERENCA'].idxmin()

# Filtrar o DataFrame principal usando os índices dos valores mínimos de 'DIFERENCA'
receita_liquida_trime_prox = receita_liquida_tri_prox.loc[indices_min_DT_INI]

# Selecionar colunas desejadas e adicionar a coluna 'Ano' com o valor correspondente
receita_liquida_trime_prox = receita_liquida_trime_prox[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA', 'DT_REFER']]  # Manter a coluna 'VL_CONTA'
receita_liquida_trime_prox['Ano_tri'] = ano_proximo




C:\Users\User\AppData\Local\Temp\ipykernel_25252\1031811841.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  receita_liquida_tri_prox['DT_INI_EXERC'] = pd.to_datetime(receita_liquida_tri_prox['DT_INI_EXERC'])
C:\Users\User\AppData\Local\Temp\ipykernel_25252\1031811841.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  receita_liquida_tri_prox['DT_FIM_EXERC'] = pd.to_datetime(receita_liquida_tri_prox['DT_FIM_EXERC'])
C:\Users\User\AppData\Local\Temp\ipykernel_25252\1031811841.py:16: SettingWithCopyWarn

In [13]:
# Definindo o caminho do arquivo CSV
caminho_arquivo_tri = f'{diretorio_destino_arq_trim}/itr_cia_aberta_DRE_con_{ano_corrente}.csv'

# Lendo o arquivo CSV e ignorando as linhas com formatação incorreta
df_tri = pd.read_csv(caminho_arquivo_tri, encoding='cp1252', sep=';', on_bad_lines='skip')

# Filtrando os dados para encontrar a Receita Operacional Líquida do Período
receita_liquida_tri = df_tri[(df_tri['ORDEM_EXERC'] == 'ÚLTIMO') & (df_tri['CD_CONTA'] == '3.01')]

# Converter as colunas 'DT_INI_EXERC' e 'DT_FIM_EXERC' para o tipo datetime
receita_liquida_tri['DT_INI_EXERC'] = pd.to_datetime(receita_liquida_tri['DT_INI_EXERC'])
receita_liquida_tri['DT_FIM_EXERC'] = pd.to_datetime(receita_liquida_tri['DT_FIM_EXERC'])

# Calcular a diferença entre DT_FIM_EXERC e DT_INI_EXERC
receita_liquida_tri['DIFERENCA'] = (receita_liquida_tri['DT_FIM_EXERC'] - receita_liquida_tri['DT_INI_EXERC']).abs()

# Encontrar os índices da menor diferença para cada grupo de DENOM_CIA, CNPJ_CIA, DT_FIM_EXERC
indices_min_DT_INI = receita_liquida_tri.groupby(['DENOM_CIA', 'CNPJ_CIA', 'DT_FIM_EXERC'])['DIFERENCA'].idxmin()

# Filtrar o DataFrame principal usando os índices dos valores mínimos de 'DIFERENCA'
receita_liquida_trime = receita_liquida_tri.loc[indices_min_DT_INI]

# Selecionar colunas desejadas e adicionar a coluna 'Ano' com o valor correspondente
receita_liquida_trime = receita_liquida_trime[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA', 'DT_REFER']]  # Manter a coluna 'VL_CONTA'
receita_liquida_trime['Ano_tri'] = ano_corrente


C:\Users\User\AppData\Local\Temp\ipykernel_25252\4153063541.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  receita_liquida_tri['DT_INI_EXERC'] = pd.to_datetime(receita_liquida_tri['DT_INI_EXERC'])
C:\Users\User\AppData\Local\Temp\ipykernel_25252\4153063541.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  receita_liquida_tri['DT_FIM_EXERC'] = pd.to_datetime(receita_liquida_tri['DT_FIM_EXERC'])
C:\Users\User\AppData\Local\Temp\ipykernel_25252\4153063541.py:15: SettingWithCopyWarning: 
A value is try

In [14]:
# Concatenando os dois DataFrames
receita_liquida_tri = pd.concat([receita_liquida_trime, receita_liquida_trime_prox], ignore_index=True)

In [15]:
# Convertendo a coluna 'DT_REFER' para o tipo datetime
receita_liquida_tri['DT_REFER'] = pd.to_datetime(receita_liquida_tri['DT_REFER'])

# Criando uma coluna de trimestre (Q1, Q2, etc.)
receita_liquida_tri['Quarter'] = receita_liquida_tri['DT_REFER'].dt.to_period('Q')

# Criando um novo DataFrame com as colunas corretas para cada trimestre
pivot = receita_liquida_tri.pivot_table(values='VL_CONTA', index=['DENOM_CIA', 'CNPJ_CIA'], columns='Quarter', aggfunc='first')

# Resetando o índice para transformar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
pivot.reset_index(inplace=True)

# Preenchendo valores NaN com 0, se necessário
pivot.fillna(0, inplace=True)

# Renomeando as colunas para incluir o trimestre e o ano (ex: Q1_2023, Q2_2023, etc.)
pivot.columns = [f'Q{col.quarter}_{col.start_time.year}' if isinstance(col, pd.Period) else col for col in pivot.columns]

# Atualizando o DataFrame original com o DataFrame pivotado e renomeado
receita_liquida_trim = pivot

Ativo total (BPA)

Histórico

In [16]:
# Definindo relatório a ser aberto
relatorio = "dfp_cia_aberta_BPA_con"  # inserir apenas o nome do arquivo, sem o ano

# Lista de anos desejados
anos = list(range(ano_inicial, ano_final+1))

# DataFrame vazio para armazenar os resultados
resultados_bpa = pd.DataFrame()

# Loop pelos anos
for ano in anos:
    # Definindo o caminho do arquivo usando formatação de string
    caminho_arquivo = f'{diretorio_destino_arq_csv}/{ano}/{relatorio}_{ano}.csv'

    # Lendo o arquivo CSV
    df = pd.read_csv(caminho_arquivo, encoding='utf-8', sep=',')

    # Filtrando os dados para encontrar o Lucro Líquido do Período
    var_BPA = df[(df['ORDEM_EXERC'] == 'ÚLTIMO') & (df['CD_CONTA'] == '1')]

    # Selecionando colunas desejadas
    var_BPA = var_BPA[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA']]

    # Adicionando a coluna 'Ano' com o valor correspondente
    var_BPA['Ano'] = ano

    # Adicionando os resultados ao DataFrame final
    resultados_bpa = pd.concat([resultados_bpa, var_BPA], ignore_index=True)

# Agregando os valores duplicados
resultados_agregados_bpa = resultados_bpa.groupby(['DENOM_CIA', 'CNPJ_CIA', 'Ano'])['VL_CONTA'].sum().reset_index()

# Criando um DataFrame pivotado
ativo_total = resultados_agregados_bpa.pivot_table(index=['DENOM_CIA', 'CNPJ_CIA'], columns='Ano', values='VL_CONTA', aggfunc='sum').reset_index()

# Renomeando as colunas para remover o nome da coluna do índice
ativo_total.name = None


Trimestral

In [17]:
# Definindo o caminho do arquivo CSV para o ano próximo
caminho_arquivo_tri_prox = f'{diretorio_destino_arq_trim}/itr_cia_aberta_BPA_con_{ano_proximo}.csv'

# Lendo o arquivo CSV e ignorando as linhas com formatação incorreta
df_tri_prox = pd.read_csv(caminho_arquivo_tri_prox, encoding='cp1252', sep=';', on_bad_lines='skip')

# Filtrando os dados para encontrar o Ativo Total do Período
ativo_total_tri_prox = df_tri_prox[(df_tri_prox['ORDEM_EXERC'] == 'ÚLTIMO') & (df_tri_prox['CD_CONTA'] == '1')]

# Converter a coluna 'DT_REFER' para o tipo datetime
ativo_total_tri_prox['DT_REFER'] = pd.to_datetime(ativo_total_tri_prox['DT_REFER'])

# Encontrar os índices da maior 'DT_REFER' quando houver datas 'DT_REFER' iguais
indices_max_DT_REFER_prox = ativo_total_tri_prox.groupby(['DENOM_CIA', 'CNPJ_CIA', 'DT_REFER'])['DT_REFER'].idxmax()

# Filtrar o DataFrame principal usando os índices dos valores máximos de 'DT_REFER'
ativo_total_trime_prox = ativo_total_tri_prox.loc[indices_max_DT_REFER_prox]

# Selecionar colunas desejadas e adicionar a coluna 'Ano' com o valor correspondente
ativo_total_trime_prox = ativo_total_trime_prox[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA', 'DT_REFER']]  # Manter a coluna 'VL_CONTA'
ativo_total_trime_prox['Ano_tri'] = ano_proximo

# Definindo o caminho do arquivo CSV para o ano corrente
caminho_arquivo_tri = f'{diretorio_destino_arq_trim}/itr_cia_aberta_BPA_con_{ano_corrente}.csv'

# Lendo o arquivo CSV e ignorando as linhas com formatação incorreta
df_tri = pd.read_csv(caminho_arquivo_tri, encoding='cp1252', sep=';', on_bad_lines='skip')

# Filtrando os dados para encontrar o Ativo Total do Período
ativo_total_tri = df_tri[(df_tri['ORDEM_EXERC'] == 'ÚLTIMO') & (df_tri['CD_CONTA'] == '1')]

# Converter a coluna 'DT_REFER' para o tipo datetime
ativo_total_tri['DT_REFER'] = pd.to_datetime(ativo_total_tri['DT_REFER'])

# Encontrar os índices da maior 'DT_REFER' quando houver datas 'DT_REFER' iguais
indices_max_DT_REFER = ativo_total_tri.groupby(['DENOM_CIA', 'CNPJ_CIA', 'DT_REFER'])['DT_REFER'].idxmax()

# Filtrar o DataFrame principal usando os índices dos valores máximos de 'DT_REFER'
ativo_total_trime = ativo_total_tri.loc[indices_max_DT_REFER]

# Selecionar colunas desejadas e adicionar a coluna 'Ano' com o valor correspondente
ativo_total_trime = ativo_total_trime[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA', 'DT_REFER']]  # Manter a coluna 'VL_CONTA'
ativo_total_trime['Ano_tri'] = ano_corrente

# Concatenando os dois DataFrames
ativo_total_tri = pd.concat([ativo_total_trime, ativo_total_trime_prox], ignore_index=True)

# Convertendo a coluna 'DT_REFER' para o tipo datetime
ativo_total_tri['DT_REFER'] = pd.to_datetime(ativo_total_tri['DT_REFER'])

# Criando uma coluna de trimestre (Q1, Q2, etc.)
ativo_total_tri['Quarter'] = ativo_total_tri['DT_REFER'].dt.to_period('Q')

# Criando um novo DataFrame com as colunas corretas para cada trimestre
pivot = ativo_total_tri.pivot_table(values='VL_CONTA', index=['DENOM_CIA', 'CNPJ_CIA'], columns='Quarter', aggfunc='first')

# Resetando o índice para transformar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
pivot.reset_index(inplace=True)

# Preenchendo valores NaN com 0, se necessário
pivot.fillna(0, inplace=True)

# Renomeando as colunas para incluir o trimestre e o ano (ex: Q1_2023, Q2_2023, etc.)
pivot.columns = [f'Q{col.quarter}_{col.start_time.year}' if isinstance(col, pd.Period) else col for col in pivot.columns]

# Atualizando o DataFrame original com o DataFrame pivotado e renomeado
ativo_total_trim = pivot



C:\Users\User\AppData\Local\Temp\ipykernel_25252\3432285945.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ativo_total_tri_prox['DT_REFER'] = pd.to_datetime(ativo_total_tri_prox['DT_REFER'])
C:\Users\User\AppData\Local\Temp\ipykernel_25252\3432285945.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ativo_total_tri['DT_REFER'] = pd.to_datetime(ativo_total_tri['DT_REFER'])


Patrimônio Líquido (BPP)

Histórico

In [18]:
# Definindo relatório a ser aberto
relatorio = 'dfp_cia_aberta_BPP_con'

# Lista de anos desejados
anos = list(range(ano_inicial, ano_final+1))

# DataFrame vazio para armazenar os resultados
resultados_bpp = pd.DataFrame()

# Loop pelos anos
for ano in anos:
    # Definindo o caminho do arquivo usando formatação de string
    caminho_arquivo = f'{diretorio_destino_arq_csv}/{ano}/{relatorio}_{ano}.csv'

    # Lendo o arquivo CSV
    df = pd.read_csv(caminho_arquivo, encoding='utf-8', sep=',')

    # Filtrando os dados para encontrar o Lucro Líquido do Período
    var_BPP = df[(df['ORDEM_EXERC'] == 'ÚLTIMO') & (df['CD_CONTA'] == '2.03')]

    # Selecionando colunas desejadas
    var_BPP = var_BPP[['DENOM_CIA','CNPJ_CIA', 'VL_CONTA']]

    # Adicionando a coluna 'Ano' com o valor correspondente
    var_BPP['Ano'] = ano

    # Adicionando os resultados ao DataFrame final
    resultados_bpp = pd.concat([resultados_bpp, var_BPP], ignore_index=True)

# Agregando os valores duplicados
resultados_agregados_bpp = resultados_bpp.groupby(['DENOM_CIA','CNPJ_CIA', 'Ano'])['VL_CONTA'].sum().reset_index()

# Criando um DataFrame pivotado
patrimonio_liquido_con = resultados_agregados_bpp.pivot_table(index=['DENOM_CIA','CNPJ_CIA'], columns='Ano', values='VL_CONTA', aggfunc='sum').reset_index()

# Renomeando as colunas para remover o nome da coluna do índice
patrimonio_liquido_con.name = None




In [19]:
#Participações dos acionistas não controladores
# Definindo relatório a ser aberto
relatorio = 'dfp_cia_aberta_BPP_con'

# Lista de anos desejados
anos = list(range(ano_inicial, ano_final+1))

# DataFrame vazio para armazenar os resultados
resultados_bpp = pd.DataFrame()

# Loop pelos anos
for ano in anos:
    # Definindo o caminho do arquivo usando formatação de string
    caminho_arquivo = f'{diretorio_destino_arq_csv}/{ano}/{relatorio}_{ano}.csv'

    # Lendo o arquivo CSV
    df = pd.read_csv(caminho_arquivo, encoding='utf-8', sep=',')

    # Filtrando os dados para encontrar o Lucro Líquido do Período
    var_BPP = df[(df['ORDEM_EXERC'] == 'ÚLTIMO') & (df['CD_CONTA'] == '2.03.09')]

    # Selecionando colunas desejadas
    var_BPP = var_BPP[['DENOM_CIA','CNPJ_CIA', 'VL_CONTA']]

    # Adicionando a coluna 'Ano' com o valor correspondente
    var_BPP['Ano'] = ano

    # Adicionando os resultados ao DataFrame final
    resultados_bpp = pd.concat([resultados_bpp, var_BPP], ignore_index=True)

# Agregando os valores duplicados
resultados_agregados_bpp = resultados_bpp.groupby(['DENOM_CIA','CNPJ_CIA', 'Ano'])['VL_CONTA'].sum().reset_index()

# Criando um DataFrame pivotado
part_ac_n_controladores = resultados_agregados_bpp.pivot_table(index=['DENOM_CIA','CNPJ_CIA'], columns='Ano', values='VL_CONTA', aggfunc='sum').reset_index()

# Renomeando as colunas para remover o nome da coluna do índice
part_ac_n_controladores.name = None


In [20]:
# Configurando as colunas 'DENOM_CIA' e 'CNPJ_CIA' como índices nos DataFrames
patrimonio_liquido_con.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)
part_ac_n_controladores.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)

# Substituindo valores NaN por 0
patrimonio_liquido_con.fillna(0, inplace=True)
part_ac_n_controladores.fillna(0, inplace=True)

# Subtraindo os valores para obter divida_liquida
patrimonio_liquido = patrimonio_liquido_con.sub(part_ac_n_controladores, fill_value=0)

# Resetando o índice para tornar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
patrimonio_liquido.reset_index(inplace=True)


Trimestral

In [21]:
# Definindo o caminho do arquivo CSV
caminho_arquivo_tri_prox = f'{diretorio_destino_arq_trim}/itr_cia_aberta_BPP_con_{ano_proximo}.csv'

# Lendo o arquivo CSV e ignorando as linhas com formatação incorreta
df_tri_prox = pd.read_csv(caminho_arquivo_tri_prox, encoding='cp1252', sep=';', on_bad_lines='skip')

# Filtrando os dados para encontrar o Patrimônio Líquido Consolidado
var_BPP_tri_prox = df_tri_prox[(df_tri_prox['ORDEM_EXERC'] == 'ÚLTIMO') & (df_tri_prox['CD_CONTA'] == '2.03')]

# Converter a coluna 'DT_REFER' para o tipo datetime
var_BPP_tri_prox['DT_REFER'] = pd.to_datetime(var_BPP_tri_prox['DT_REFER'])

# Encontrar os índices da maior 'DT_REFER' quando houver datas 'DT_REFER' iguais
indices_max_DT_REFER_prox = var_BPP_tri_prox.groupby(['DENOM_CIA', 'CNPJ_CIA', 'DT_REFER'])['DT_REFER'].idxmax()

# Filtrar o DataFrame principal usando os índices dos valores máximos de 'DT_REFER'
var_BPP_tri_prox = var_BPP_tri_prox.loc[indices_max_DT_REFER_prox]

# Selecionar colunas desejadas e adicionar a coluna 'Ano' com o valor correspondente
var_BPP_tri_prox = var_BPP_tri_prox[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA', 'DT_REFER']]  # Manter a coluna 'VL_CONTA'
var_BPP_tri_prox['Ano_tri'] = ano_proximo

# Definindo o caminho do arquivo CSV
caminho_arquivo_tri = f'{diretorio_destino_arq_trim}/itr_cia_aberta_BPP_con_{ano_corrente}.csv'

# Lendo o arquivo CSV e ignorando as linhas com formatação incorreta
df_tri = pd.read_csv(caminho_arquivo_tri, encoding='cp1252', sep=';', on_bad_lines='skip')

# Filtrando os dados para encontrar o Patrimônio Líquido Consolidado
var_BPP_tri = df_tri[(df_tri['ORDEM_EXERC'] == 'ÚLTIMO') & (df_tri['CD_CONTA'] == '2.03')]

# Converter a coluna 'DT_REFER' para o tipo datetime
var_BPP_tri['DT_REFER'] = pd.to_datetime(var_BPP_tri['DT_REFER'])

# Encontrar os índices da maior 'DT_REFER' quando houver datas 'DT_REFER' iguais
indices_max_DT_REFER = var_BPP_tri.groupby(['DENOM_CIA', 'CNPJ_CIA', 'DT_REFER'])['DT_REFER'].idxmax()

# Filtrar o DataFrame principal usando os índices dos valores máximos de 'DT_REFER'
var_BPP_tri = var_BPP_tri.loc[indices_max_DT_REFER]

# Selecionar colunas desejadas e adicionar a coluna 'Ano' com o valor correspondente
var_BPP_tri = var_BPP_tri[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA', 'DT_REFER']]  # Manter a coluna 'VL_CONTA'
var_BPP_tri['Ano_tri'] = ano_corrente

# Concatenando os dois DataFrames
patrimonio_liquido_con_tri = pd.concat([var_BPP_tri, var_BPP_tri_prox], ignore_index=True)

# Convertendo a coluna 'DT_REFER' para o tipo datetime
patrimonio_liquido_con_tri['DT_REFER'] = pd.to_datetime(patrimonio_liquido_con_tri['DT_REFER'])

# Criando uma coluna de trimestre (Q1, Q2, etc.)
patrimonio_liquido_con_tri['Quarter'] = patrimonio_liquido_con_tri['DT_REFER'].dt.to_period('Q')

# Criando um novo DataFrame com as colunas corretas para cada trimestre
pivot_con_tri = patrimonio_liquido_con_tri.pivot_table(values='VL_CONTA', index=['DENOM_CIA', 'CNPJ_CIA'], columns='Quarter', aggfunc='first')

# Resetando o índice para transformar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
pivot_con_tri.reset_index(inplace=True)

# Preenchendo valores NaN com 0, se necessário
pivot_con_tri.fillna(0, inplace=True)

# Renomeando as colunas para incluir o trimestre e o ano (ex: Q1_2023, Q2_2023, etc.)
pivot_con_tri.columns = [f'Q{col.quarter}_{col.start_time.year}' if isinstance(col, pd.Period) else col for col in pivot_con_tri.columns]

patrimonio_liquido_con_trim = pivot_con_tri

# Replicando o processo para as Participações de Acionistas Não Controladores
# Definindo o caminho do arquivo CSV
caminho_arquivo_tri_prox_controladores = f'{diretorio_destino_arq_trim}/itr_cia_aberta_BPP_con_{ano_proximo}.csv'

# Lendo o arquivo CSV e ignorando as linhas com formatação incorreta
df_tri_prox_controladores = pd.read_csv(caminho_arquivo_tri_prox_controladores, encoding='cp1252', sep=';', on_bad_lines='skip')

# Filtrando os dados para encontrar as Participações de Acionistas Não Controladores
var_BPP_tri_prox_controladores = df_tri_prox_controladores[(df_tri_prox_controladores['ORDEM_EXERC'] == 'ÚLTIMO') & (df_tri_prox_controladores['CD_CONTA'] == '2.03.09')]

# Converter a coluna 'DT_REFER' para o tipo datetime
var_BPP_tri_prox_controladores['DT_REFER'] = pd.to_datetime(var_BPP_tri_prox_controladores['DT_REFER'])

# Encontrar os índices da maior 'DT_REFER' quando houver datas 'DT_REFER' iguais
indices_max_DT_REFER_prox_controladores = var_BPP_tri_prox_controladores.groupby(['DENOM_CIA', 'CNPJ_CIA', 'DT_REFER'])['DT_REFER'].idxmax()

# Filtrar o DataFrame principal usando os índices dos valores máximos de 'DT_REFER'
var_BPP_tri_prox_controladores = var_BPP_tri_prox_controladores.loc[indices_max_DT_REFER_prox_controladores]

# Selecionar colunas desejadas e adicionar a coluna 'Ano' com o valor correspondente
var_BPP_tri_prox_controladores = var_BPP_tri_prox_controladores[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA', 'DT_REFER']]  # Manter a coluna 'VL_CONTA'
var_BPP_tri_prox_controladores['Ano_tri'] = ano_proximo

# Definindo o caminho do arquivo CSV para o ano corrente
caminho_arquivo_tri_controladores = f'{diretorio_destino_arq_trim}/itr_cia_aberta_BPP_con_{ano_corrente}.csv'

# Lendo o arquivo CSV e ignorando as linhas com formatação incorreta
df_tri_controladores = pd.read_csv(caminho_arquivo_tri_controladores, encoding='cp1252', sep=';', on_bad_lines='skip')

# Filtrando os dados para encontrar as Participações de Acionistas Não Controladores
var_BPP_tri_controladores = df_tri_controladores[(df_tri_controladores['ORDEM_EXERC'] == 'ÚLTIMO') & (df_tri_controladores['CD_CONTA'] == '2.03.09')]

# Converter a coluna 'DT_REFER' para o tipo datetime
var_BPP_tri_controladores['DT_REFER'] = pd.to_datetime(var_BPP_tri_controladores['DT_REFER'])

# Encontrar os índices da maior 'DT_REFER' quando houver datas 'DT_REFER' iguais
indices_max_DT_REFER_controladores = var_BPP_tri_controladores.groupby(['DENOM_CIA', 'CNPJ_CIA', 'DT_REFER'])['DT_REFER'].idxmax()

# Filtrar o DataFrame principal usando os índices dos valores máximos de 'DT_REFER'
var_BPP_tri_controladores = var_BPP_tri_controladores.loc[indices_max_DT_REFER_controladores]

# Selecionar colunas desejadas e adicionar a coluna 'Ano' com o valor correspondente
var_BPP_tri_controladores = var_BPP_tri_controladores[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA', 'DT_REFER']]  # Manter a coluna 'VL_CONTA'
var_BPP_tri_controladores['Ano_tri'] = ano_corrente

# Concatenando os dois DataFrames
part_ac_n_controladores_tri = pd.concat([var_BPP_tri_controladores, var_BPP_tri_prox_controladores], ignore_index=True)

# Convertendo a coluna 'DT_REFER' para o tipo datetime
part_ac_n_controladores_tri['DT_REFER'] = pd.to_datetime(part_ac_n_controladores_tri['DT_REFER'])

# Criando uma coluna de trimestre (Q1, Q2, etc.)
part_ac_n_controladores_tri['Quarter'] = part_ac_n_controladores_tri['DT_REFER'].dt.to_period('Q')

# Criando um novo DataFrame com as colunas corretas para cada trimestre
pivot_controladores_tri = part_ac_n_controladores_tri.pivot_table(values='VL_CONTA', index=['DENOM_CIA', 'CNPJ_CIA'], columns='Quarter', aggfunc='first')

# Resetando o índice para transformar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
pivot_controladores_tri.reset_index(inplace=True)

# Preenchendo valores NaN com 0, se necessário
pivot_controladores_tri.fillna(0, inplace=True)

# Renomeando as colunas para incluir o trimestre e o ano (ex: Q1_2023, Q2_2023, etc.)
pivot_controladores_tri.columns = [f'Q{col.quarter}_{col.start_time.year}' if isinstance(col, pd.Period) else col for col in pivot_controladores_tri.columns]

# Atribuindo o DataFrame resultante à variável final para Participações de Acionistas Não Controladores
part_ac_n_controladores_trim = pivot_controladores_tri

# Configurando as colunas 'DENOM_CIA' e 'CNPJ_CIA' como índices nos DataFrames
patrimonio_liquido_con_trim.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)
part_ac_n_controladores_trim.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)

# Substituindo valores NaN por 0 nos DataFrames
patrimonio_liquido_con_trim.fillna(0, inplace=True)
part_ac_n_controladores_trim.fillna(0, inplace=True)

# Subtraindo os valores das participações não controladoras para obter o Patrimônio Líquido Ajustado
patrimonio_liquido_trim = patrimonio_liquido_con_trim.sub(part_ac_n_controladores_trim, fill_value=0)

# Resetando o índice para tornar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
patrimonio_liquido_trim.reset_index(inplace=True)

# Exibindo o DataFrame resultante
patrimonio_liquido_trim

C:\Users\User\AppData\Local\Temp\ipykernel_25252\3934386652.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  var_BPP_tri_prox['DT_REFER'] = pd.to_datetime(var_BPP_tri_prox['DT_REFER'])
C:\Users\User\AppData\Local\Temp\ipykernel_25252\3934386652.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  var_BPP_tri['DT_REFER'] = pd.to_datetime(var_BPP_tri['DT_REFER'])
C:\Users\User\AppData\Local\Temp\ipykernel_25252\3934386652.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

,DENOM_CIA,CNPJ_CIA,Q1_2023,Q2_2023,Q3_2023,Q4_2023,Q1_2024,Q2_2024,Q3_2024
0,2W ECOBANK S.A.,08.773.135/0001-00,167499.0,1.900310e+05,1.268910e+05,0.0,0.000000e+00,0.000000e+00,0.0
1,3R PETROLEUM ÓLEO E GÁS S.A.,12.091.809/0001-55,4292691.0,5.252816e+06,5.185408e+06,0.0,5.278920e+06,4.944065e+06,0.0
2,AEGEA SANEAMENTO E PARTICIPAÇÕES S.A.,08.827.501/0001-58,5618247.0,5.485830e+06,5.393667e+06,0.0,5.506817e+06,5.327748e+06,0.0
3,AERIS IND. E COM. DE EQUIP. PARA GER. DE ENG. ...,12.528.708/0001-07,862600.0,8.432150e+05,8.150790e+05,0.0,1.097234e+06,1.094390e+06,0.0
4,AES BRASIL ENERGIA S.A.,37.663.076/0001-07,4401583.0,4.404140e+06,4.469481e+06,0.0,4.394454e+06,4.262700e+06,0.0
...,...,...,...,...,...,...,...,...,...
482,YBYRÁ CAPITAL S.A.,02.217.319/0001-07,2830303.0,4.665352e+09,4.672059e+09,0.0,4.685809e+09,4.682955e+09,0.0
483,YDUQS PARTICIPACOES S.A.,08.807.432/0001-10,3100240.0,3.138848e+06,3.239033e+06,0.0,3.197081e+06,3.142667e+06,0.0
484,YOU INC INCORPORADORA E PARTICIPAÇÃO S.A,11.284.204/0001-18,95765.0,9.861200e+04,1.154900e+05,0.0,8.653900e+04,1.567400e+04,0.0
485,YUNY INCORPORADORA HOLDING S.A.,09.267.996/0001-70,40895.0,5.885300e+04,6.261400e+04,0.0,6.144400e+04,5.797500e+04,0.0


Lucro Líquido consolidado do período (DRE)

Histórico

In [22]:
# Definindo relatório a ser aberto
relatorio = "dfp_cia_aberta_DRE_con"  # inserir apenas o nome do arquivo, sem o ano

# Lista de anos desejados
anos = list(range(ano_inicial, ano_final+1))

# DataFrame vazio para armazenar os resultados
resultados_dre = pd.DataFrame()

# Loop pelos anos
for ano in anos:
    # Definindo o caminho do arquivo usando formatação de string
    caminho_arquivo = f'{diretorio_destino_arq_csv}/{ano}/{relatorio}_{ano}.csv'

    # Lendo o arquivo CSV
    df = pd.read_csv(caminho_arquivo, encoding='utf-8', sep=',')

    # Filtrando os dados para encontrar o Lucro Líquido do Período
    var_DRE = df[(df['ORDEM_EXERC'] == 'ÚLTIMO') & (df['CD_CONTA'] == '3.11')]

    # Selecionando colunas desejadas
    var_DRE = var_DRE[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA']]

    # Adicionando a coluna 'Ano' com o valor correspondente
    var_DRE['Ano'] = ano

    # Adicionando os resultados ao DataFrame final
    resultados_dre = pd.concat([resultados_dre, var_DRE], ignore_index=True)

# Agregando os valores duplicados
resultados_agregados_dre = resultados_dre.groupby(['DENOM_CIA', 'CNPJ_CIA', 'Ano'])['VL_CONTA'].sum().reset_index()

# Criando um DataFrame pivotado
lucro_prej_liquido = resultados_agregados_dre.pivot_table(index=['DENOM_CIA', 'CNPJ_CIA'], columns='Ano', values='VL_CONTA', aggfunc='sum').reset_index()

# Renomeando as colunas para remover o nome da coluna do índice
lucro_prej_liquido.columns.name = None







In [23]:
# Definindo relatório a ser aberto
relatorio = "dfp_cia_aberta_DRE_con"  # inserir apenas o nome do arquivo, sem o ano

# Lista de anos desejados
anos = list(range(ano_inicial, ano_final+1))

# DataFrame vazio para armazenar os resultados
resultados_dre = pd.DataFrame()

# Loop pelos anos
for ano in anos:
    # Definindo o caminho do arquivo usando formatação de string
    caminho_arquivo = f'{diretorio_destino_arq_csv}/{ano}/{relatorio}_{ano}.csv'

    # Lendo o arquivo CSV
    df = pd.read_csv(caminho_arquivo, encoding='utf-8', sep=',')

    # Filtrando os dados para encontrar o Lucro Líquido do Período
    var_DRE = df[(df['ORDEM_EXERC'] == 'ÚLTIMO') & (df['CD_CONTA'] == '3.11.02')]

    # Selecionando colunas desejadas
    var_DRE = var_DRE[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA']]

    # Adicionando a coluna 'Ano' com o valor correspondente
    var_DRE['Ano'] = ano

    # Adicionando os resultados ao DataFrame final
    resultados_dre = pd.concat([resultados_dre, var_DRE], ignore_index=True)

# Agregando os valores duplicados
resultados_agregados_dre = resultados_dre.groupby(['DENOM_CIA', 'CNPJ_CIA', 'Ano'])['VL_CONTA'].sum().reset_index()

# Criando um DataFrame pivotado
atrib_n_socios_control = resultados_agregados_dre.pivot_table(index=['DENOM_CIA', 'CNPJ_CIA'], columns='Ano', values='VL_CONTA', aggfunc='sum').reset_index()

# Renomeando as colunas para remover o nome da coluna do índice
atrib_n_socios_control.columns.name = None



In [24]:
# Configurando as colunas 'DENOM_CIA' e 'CNPJ_CIA' como índices nos DataFrames
lucro_prej_liquido.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)
atrib_n_socios_control.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)

# Substituindo valores NaN por 0
lucro_prej_liquido.fillna(0, inplace=True)
atrib_n_socios_control.fillna(0, inplace=True)

# Subtraindo os valores para obter divida_liquida
lucro_liquido = lucro_prej_liquido.sub(atrib_n_socios_control, fill_value=0)

# Resetando o índice para tornar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
lucro_liquido.reset_index(inplace=True)


Trimestral

In [25]:
# Definindo o caminho do arquivo CSV para o ano próximo
caminho_arquivo_tri_prox = f'{diretorio_destino_arq_trim}/itr_cia_aberta_DRE_con_{ano_proximo}.csv'

# Lendo o arquivo CSV e ignorando as linhas com formatação incorreta
df_tri_prox = pd.read_csv(caminho_arquivo_tri_prox, encoding='cp1252', sep=';', on_bad_lines='skip')

# Filtrando os dados para encontrar o Lucro Líquido do Período
lucro_liquido_tri_prox = df_tri_prox[(df_tri_prox['ORDEM_EXERC'] == 'ÚLTIMO') & (df_tri_prox['CD_CONTA'] == '3.11')]

# Converter as colunas 'DT_INI_EXERC' e 'DT_FIM_EXERC' para o tipo datetime
lucro_liquido_tri_prox['DT_INI_EXERC'] = pd.to_datetime(lucro_liquido_tri_prox['DT_INI_EXERC'])
lucro_liquido_tri_prox['DT_FIM_EXERC'] = pd.to_datetime(lucro_liquido_tri_prox['DT_FIM_EXERC'])

# Calcular a diferença entre DT_FIM_EXERC e DT_INI_EXERC
lucro_liquido_tri_prox['DIFERENCA'] = (lucro_liquido_tri_prox['DT_FIM_EXERC'] - lucro_liquido_tri_prox['DT_INI_EXERC']).abs()

# Encontrar os índices da menor diferença para cada grupo de DENOM_CIA, CNPJ_CIA, DT_FIM_EXERC
indices_min_DT_INI_prox = lucro_liquido_tri_prox.groupby(['DENOM_CIA', 'CNPJ_CIA', 'DT_FIM_EXERC'])['DIFERENCA'].idxmin()

# Filtrar o DataFrame principal usando os índices dos valores mínimos de 'DIFERENCA'
lucro_liquido_trime_prox = lucro_liquido_tri_prox.loc[indices_min_DT_INI_prox]

# Selecionar colunas desejadas e adicionar a coluna 'Ano' com o valor correspondente
lucro_liquido_trime_prox = lucro_liquido_trime_prox[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA', 'DT_REFER']]
lucro_liquido_trime_prox['Ano_tri'] = ano_proximo

# Repetindo o processo para o ano corrente
caminho_arquivo_tri = f'{diretorio_destino_arq_trim}/itr_cia_aberta_DRE_con_{ano_corrente}.csv'
df_tri = pd.read_csv(caminho_arquivo_tri, encoding='cp1252', sep=';', on_bad_lines='skip')

lucro_liquido_tri = df_tri[(df_tri['ORDEM_EXERC'] == 'ÚLTIMO') & (df_tri['CD_CONTA'] == '3.11')]

lucro_liquido_tri['DT_INI_EXERC'] = pd.to_datetime(lucro_liquido_tri['DT_INI_EXERC'])
lucro_liquido_tri['DT_FIM_EXERC'] = pd.to_datetime(lucro_liquido_tri['DT_FIM_EXERC'])

lucro_liquido_tri['DIFERENCA'] = (lucro_liquido_tri['DT_FIM_EXERC'] - lucro_liquido_tri['DT_INI_EXERC']).abs()

indices_min_DT_INI = lucro_liquido_tri.groupby(['DENOM_CIA', 'CNPJ_CIA', 'DT_FIM_EXERC'])['DIFERENCA'].idxmin()

lucro_liquido_trime = lucro_liquido_tri.loc[indices_min_DT_INI]

lucro_liquido_trime = lucro_liquido_trime[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA', 'DT_REFER']]
lucro_liquido_trime['Ano_tri'] = ano_corrente

# Concatenando os dois DataFrames
lucro_liquido_tri = pd.concat([lucro_liquido_trime, lucro_liquido_trime_prox], ignore_index=True)

# Convertendo a coluna 'DT_REFER' para o tipo datetime e criando a coluna de trimestre
lucro_liquido_tri['DT_REFER'] = pd.to_datetime(lucro_liquido_tri['DT_REFER'])
lucro_liquido_tri['Quarter'] = lucro_liquido_tri['DT_REFER'].dt.to_period('Q')

# Pivotando os dados para criar colunas de trimestre
pivot_lucro_liquido = lucro_liquido_tri.pivot_table(values='VL_CONTA', index=['DENOM_CIA', 'CNPJ_CIA'], columns='Quarter', aggfunc='first')

pivot_lucro_liquido.reset_index(inplace=True)
pivot_lucro_liquido.fillna(0, inplace=True)

pivot_lucro_liquido.columns = [f'Q{col.quarter}_{col.start_time.year}' if isinstance(col, pd.Period) else col for col in pivot_lucro_liquido.columns]

lucro_liquido_final_trim = pivot_lucro_liquido

# Definindo o caminho do arquivo CSV para o ano próximo
atrib_n_socios_control_tri_prox = df_tri_prox[(df_tri_prox['ORDEM_EXERC'] == 'ÚLTIMO') & (df_tri_prox['CD_CONTA'] == '3.11.02')]

# Converter as colunas 'DT_INI_EXERC' e 'DT_FIM_EXERC' para o tipo datetime
atrib_n_socios_control_tri_prox['DT_INI_EXERC'] = pd.to_datetime(atrib_n_socios_control_tri_prox['DT_INI_EXERC'])
atrib_n_socios_control_tri_prox['DT_FIM_EXERC'] = pd.to_datetime(atrib_n_socios_control_tri_prox['DT_FIM_EXERC'])

# Calcular a diferença entre DT_FIM_EXERC e DT_INI_EXERC
atrib_n_socios_control_tri_prox['DIFERENCA'] = (atrib_n_socios_control_tri_prox['DT_FIM_EXERC'] - atrib_n_socios_control_tri_prox['DT_INI_EXERC']).abs()

# Encontrar os índices da menor diferença para cada grupo de DENOM_CIA, CNPJ_CIA, DT_FIM_EXERC
indices_min_DT_INI_prox = atrib_n_socios_control_tri_prox.groupby(['DENOM_CIA', 'CNPJ_CIA', 'DT_FIM_EXERC'])['DIFERENCA'].idxmin()

# Filtrar o DataFrame principal usando os índices dos valores mínimos de 'DIFERENCA'
atrib_n_socios_control_trime_prox = atrib_n_socios_control_tri_prox.loc[indices_min_DT_INI_prox]

# Selecionar colunas desejadas e adicionar a coluna 'Ano' com o valor correspondente
atrib_n_socios_control_trime_prox = atrib_n_socios_control_trime_prox[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA', 'DT_REFER']]
atrib_n_socios_control_trime_prox['Ano_tri'] = ano_proximo

# Repetindo o processo para o ano corrente
atrib_n_socios_control_tri = df_tri[(df_tri['ORDEM_EXERC'] == 'ÚLTIMO') & (df_tri['CD_CONTA'] == '3.11.02')]

atrib_n_socios_control_tri['DT_INI_EXERC'] = pd.to_datetime(atrib_n_socios_control_tri['DT_INI_EXERC'])
atrib_n_socios_control_tri['DT_FIM_EXERC'] = pd.to_datetime(atrib_n_socios_control_tri['DT_FIM_EXERC'])

atrib_n_socios_control_tri['DIFERENCA'] = (atrib_n_socios_control_tri['DT_FIM_EXERC'] - atrib_n_socios_control_tri['DT_INI_EXERC']).abs()

indices_min_DT_INI = atrib_n_socios_control_tri.groupby(['DENOM_CIA', 'CNPJ_CIA', 'DT_FIM_EXERC'])['DIFERENCA'].idxmin()

atrib_n_socios_control_trime = atrib_n_socios_control_tri.loc[indices_min_DT_INI]

atrib_n_socios_control_trime = atrib_n_socios_control_trime[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA', 'DT_REFER']]
atrib_n_socios_control_trime['Ano_tri'] = ano_corrente

# Concatenando os dois DataFrames
atrib_n_socios_control_tri = pd.concat([atrib_n_socios_control_trime, atrib_n_socios_control_trime_prox], ignore_index=True)

# Convertendo a coluna 'DT_REFER' para o tipo datetime e criando a coluna de trimestre
atrib_n_socios_control_tri['DT_REFER'] = pd.to_datetime(atrib_n_socios_control_tri['DT_REFER'])
atrib_n_socios_control_tri['Quarter'] = atrib_n_socios_control_tri['DT_REFER'].dt.to_period('Q')

# Pivotando os dados para criar colunas de trimestre
pivot_atrib_n_socios_control = atrib_n_socios_control_tri.pivot_table(values='VL_CONTA', index=['DENOM_CIA', 'CNPJ_CIA'], columns='Quarter', aggfunc='first')

# Resetando o índice para transformar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
pivot_atrib_n_socios_control.reset_index(inplace=True)

# Preenchendo valores NaN com 0, se necessário
pivot_atrib_n_socios_control.fillna(0, inplace=True)

# Renomeando as colunas para incluir o trimestre e o ano (ex: Q1_2023, Q2_2023, etc.)
pivot_atrib_n_socios_control.columns = [f'Q{col.quarter}_{col.start_time.year}' if isinstance(col, pd.Period) else col for col in pivot_atrib_n_socios_control.columns]

# Atualizando o DataFrame original com o DataFrame pivotado e renomeado
atrib_n_socios_control_trim = pivot_atrib_n_socios_control

# Configurando as colunas 'DENOM_CIA' e 'CNPJ_CIA' como índices nos DataFrames
lucro_liquido_final_trim.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)
atrib_n_socios_control_trim.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)

# Substituindo valores NaN por 0
lucro_liquido_final_trim.fillna(0, inplace=True)
atrib_n_socios_control_trim.fillna(0, inplace=True)

# Subtraindo os valores de atribuição aos sócios não controladores do lucro líquido consolidado para obter o lucro líquido ajustado
lucro_liquido_ajustado_trim = lucro_liquido_final_trim.sub(atrib_n_socios_control_trim, fill_value=0)

# Resetando o índice para tornar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
lucro_liquido_ajustado_trim.reset_index(inplace=True)

lucro_liquido_trim = lucro_liquido_ajustado_trim


C:\Users\User\AppData\Local\Temp\ipykernel_25252\2144873253.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lucro_liquido_tri_prox['DT_INI_EXERC'] = pd.to_datetime(lucro_liquido_tri_prox['DT_INI_EXERC'])
C:\Users\User\AppData\Local\Temp\ipykernel_25252\2144873253.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lucro_liquido_tri_prox['DT_FIM_EXERC'] = pd.to_datetime(lucro_liquido_tri_prox['DT_FIM_EXERC'])
C:\Users\User\AppData\Local\Temp\ipykernel_25252\2144873253.py:15: SettingWithCopyWarning: 
A 

Dívida líquida = empréstimos e financiamentos - caixa equivalente de caixa

Histórico

In [26]:
# Definindo relatório a ser aberto
relatorio = "dfp_cia_aberta_BPP_con"  # inserir apenas o nome do arquivo, sem o ano

# Lista de anos desejados
anos = list(range(ano_inicial, ano_final+1))

# DataFrame vazio para armazenar os resultados
resultados_bpp = pd.DataFrame()

# Loop pelos anos
for ano in anos:
    # Definindo o caminho do arquivo usando formatação de string
    caminho_arquivo = f'{diretorio_destino_arq_csv}/{ano}/{relatorio}_{ano}.csv'

    # Lendo o arquivo CSV
    df = pd.read_csv(caminho_arquivo, encoding='utf-8', sep=',')

    # Filtrando os dados para encontrar o Lucro Líquido do Período
    var_BPP = df[(df['ORDEM_EXERC'] == 'ÚLTIMO') & (df['CD_CONTA'] == '2.01.04')]

    # Selecionando colunas desejadas
    var_BPP = var_BPP[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA']]

    # Adicionando a coluna 'Ano' com o valor correspondente
    var_BPP['Ano'] = ano

    # Adicionando os resultados ao DataFrame final
    resultados_bpp = pd.concat([resultados_bpp, var_BPP], ignore_index=True)

# Agregando os valores duplicados
resultados_agregados_bpp = resultados_bpp.groupby(['DENOM_CIA', 'CNPJ_CIA', 'Ano'])['VL_CONTA'].sum().reset_index()

# Criando um DataFrame pivotado
divida_total_1 = resultados_agregados_bpp.pivot_table(index=['DENOM_CIA', 'CNPJ_CIA'], columns='Ano', values='VL_CONTA', aggfunc='first').reset_index()

# Renomeando as colunas para remover o nome da coluna do índice
divida_total_1.name = None


In [27]:
# Definindo relatório a ser aberto
relatorio = "dfp_cia_aberta_BPP_con"  # inserir apenas o nome do arquivo, sem o ano

# Lista de anos desejados
anos = list(range(ano_inicial, ano_final+1))

# DataFrame vazio para armazenar os resultados
resultados_bpp = pd.DataFrame()

# Loop pelos anos
for ano in anos:
    # Definindo o caminho do arquivo usando formatação de string
    caminho_arquivo = f'{diretorio_destino_arq_csv}/{ano}/{relatorio}_{ano}.csv'

    # Lendo o arquivo CSV
    df = pd.read_csv(caminho_arquivo, encoding='utf-8', sep=',')

    # Filtrando os dados para encontrar o Lucro Líquido do Período
    var_BPP = df[(df['ORDEM_EXERC'] == 'ÚLTIMO') & (df['CD_CONTA'] == '2.02.01')]

    # Selecionando colunas desejadas
    var_BPP = var_BPP[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA']]

    # Adicionando a coluna 'Ano' com o valor correspondente
    var_BPP['Ano'] = ano

    # Adicionando os resultados ao DataFrame final
    resultados_bpp = pd.concat([resultados_bpp, var_BPP], ignore_index=True)

# Agregando os valores duplicados
resultados_agregados_bpp = resultados_bpp.groupby(['DENOM_CIA', 'CNPJ_CIA', 'Ano'])['VL_CONTA'].sum().reset_index()

# Criando um DataFrame pivotado
divida_total_2 = resultados_agregados_bpp.pivot_table(index=['DENOM_CIA', 'CNPJ_CIA'], columns='Ano', values='VL_CONTA', aggfunc='first').reset_index()

# Renomeando as colunas para remover o nome da coluna do índice
divida_total_2.name = None

In [28]:
# Configurando as colunas 'DENOM_CIA' e 'CNPJ_CIA' como índices nos DataFrames
divida_total_1.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)
divida_total_2.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)

# Substituindo valores NaN por 0
divida_total_1.fillna(0, inplace=True)
divida_total_2.fillna(0, inplace=True)

# Somando os valores para obter divida_liquida
divida_total = divida_total_1.add(divida_total_2, fill_value=0)

# Resetando o índice para tornar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
divida_total.reset_index(inplace=True)

In [29]:
# Definindo relatório a ser aberto
relatorio = "dfp_cia_aberta_BPA_con"  # inserir apenas o nome do arquivo, sem o ano

# Lista de anos desejados
anos = list(range(ano_inicial, ano_final+1))

# DataFrame vazio para armazenar os resultados
resultados_bpa = pd.DataFrame()

# Loop pelos anos
for ano in anos:
    # Definindo o caminho do arquivo usando formatação de string
    caminho_arquivo = f'{diretorio_destino_arq_csv}/{ano}/{relatorio}_{ano}.csv'

    # Lendo o arquivo CSV
    df = pd.read_csv(caminho_arquivo, encoding='utf-8', sep=',')

    # Filtrando os dados para encontrar o Lucro Líquido do Período
    var_BPA = df[(df['ORDEM_EXERC'] == 'ÚLTIMO') & (df['CD_CONTA'] == '1.01.01')]

    # Selecionando colunas desejadas
    var_BPA = var_BPA[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA']]

    # Adicionando a coluna 'Ano' com o valor correspondente
    var_BPA['Ano'] = ano

    # Adicionando os resultados ao DataFrame final
    resultados_bpa = pd.concat([resultados_bpa, var_BPA], ignore_index=True)

# Agregando os valores duplicados
resultados_agregados_bpa= resultados_bpa.groupby(['DENOM_CIA', 'CNPJ_CIA', 'Ano'])['VL_CONTA'].sum().reset_index()

# Criando um DataFrame pivotado
cx_equiv_cx = resultados_agregados_bpa.pivot_table(index=['DENOM_CIA', 'CNPJ_CIA'], columns='Ano', values='VL_CONTA', aggfunc='first').reset_index()

# Renomeando as colunas para remover o nome da coluna do índice
cx_equiv_cx.name = None


In [30]:
# Configurando as colunas 'DENOM_CIA' e 'CNPJ_CIA' como índices nos DataFrames
divida_total.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)
cx_equiv_cx.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)

# Substituindo valores NaN por 0
divida_total.fillna(0, inplace=True)
cx_equiv_cx.fillna(0, inplace=True)

# Subtraindo os valores para obter divida_liquida
divida_liquida = divida_total.sub(cx_equiv_cx, fill_value=0)

# Resetando o índice para tornar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
divida_liquida.reset_index(inplace=True)


Trimestral

In [31]:
# Definindo o caminho do arquivo CSV para o ano corrente e ano próximo
caminho_arquivo_tri_corrente = f'{diretorio_destino_arq_trim}/itr_cia_aberta_BPA_con_{ano_corrente}.csv'
caminho_arquivo_tri_proximo = f'{diretorio_destino_arq_trim}/itr_cia_aberta_BPA_con_{ano_proximo}.csv'

# Lendo os arquivos CSV para o ano corrente e ano próximo, ignorando as linhas com formatação incorreta
df_tri_corrente = pd.read_csv(caminho_arquivo_tri_corrente, encoding='cp1252', sep=';', on_bad_lines='skip')
df_tri_proximo = pd.read_csv(caminho_arquivo_tri_proximo, encoding='cp1252', sep=';', on_bad_lines='skip')

# Filtrando os dados para encontrar o Saldo Final de Caixa e Equivalentes (CD_CONTA == '1.01.01') para o ano corrente
cx_equiv_cx_tri_corrente = df_tri_corrente[(df_tri_corrente['ORDEM_EXERC'] == 'ÚLTIMO') & (df_tri_corrente['CD_CONTA'] == '1.01.01')]

# Converter a coluna 'DT_REFER' para o tipo datetime para o ano corrente
cx_equiv_cx_tri_corrente['DT_REFER'] = pd.to_datetime(cx_equiv_cx_tri_corrente['DT_REFER'])

# Encontrar os índices da maior 'DT_REFER' quando houver datas 'DT_REFER' iguais
indices_max_DT_REFER_corrente = cx_equiv_cx_tri_corrente.groupby(['DENOM_CIA', 'CNPJ_CIA', 'DT_REFER'])['DT_REFER'].idxmax()

# Filtrar o DataFrame principal usando os índices dos valores máximos de 'DT_REFER' para o ano corrente
cx_equiv_cx_tri_corrente = cx_equiv_cx_tri_corrente.loc[indices_max_DT_REFER_corrente]

# Selecionar colunas desejadas e adicionar a coluna 'Ano' com o valor correspondente
cx_equiv_cx_tri_corrente = cx_equiv_cx_tri_corrente[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA', 'DT_REFER']]
cx_equiv_cx_tri_corrente['Ano_tri'] = ano_corrente

# Repetindo o processo para o ano próximo
cx_equiv_cx_tri_proximo = df_tri_proximo[(df_tri_proximo['ORDEM_EXERC'] == 'ÚLTIMO') & (df_tri_proximo['CD_CONTA'] == '1.01.01')]
cx_equiv_cx_tri_proximo['DT_REFER'] = pd.to_datetime(cx_equiv_cx_tri_proximo['DT_REFER'])

# Encontrar os índices da maior 'DT_REFER' para o ano próximo
indices_max_DT_REFER_proximo = cx_equiv_cx_tri_proximo.groupby(['DENOM_CIA', 'CNPJ_CIA', 'DT_REFER'])['DT_REFER'].idxmax()
cx_equiv_cx_tri_proximo = cx_equiv_cx_tri_proximo.loc[indices_max_DT_REFER_proximo]

# Selecionar colunas desejadas e adicionar a coluna 'Ano' com o valor correspondente para o ano próximo
cx_equiv_cx_tri_proximo = cx_equiv_cx_tri_proximo[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA', 'DT_REFER']]
cx_equiv_cx_tri_proximo['Ano_tri'] = ano_proximo

# Concatenando os dados de ambos os anos
cx_equiv_cx_tri = pd.concat([cx_equiv_cx_tri_corrente, cx_equiv_cx_tri_proximo], ignore_index=True)

# Criando uma coluna de trimestre (Q1, Q2, etc.)
cx_equiv_cx_tri['Quarter'] = 'Q' + cx_equiv_cx_tri['DT_REFER'].dt.quarter.astype(str) + '_' + cx_equiv_cx_tri['DT_REFER'].dt.year.astype(str)

# Criando um novo DataFrame com as colunas corretas para cada trimestre
pivot_cx = cx_equiv_cx_tri.pivot_table(values='VL_CONTA', index=['DENOM_CIA', 'CNPJ_CIA'], columns='Quarter', aggfunc='last')

# Resetando o índice para transformar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
pivot_cx.reset_index(inplace=True)

# Preenchendo valores NaN com 0, se necessário
pivot_cx.fillna(0, inplace=True)

# DataFrame final para Caixa e Equivalentes
cx_equiv_cx_trim = pivot_cx


C:\Users\User\AppData\Local\Temp\ipykernel_25252\3802403867.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cx_equiv_cx_tri_corrente['DT_REFER'] = pd.to_datetime(cx_equiv_cx_tri_corrente['DT_REFER'])
C:\Users\User\AppData\Local\Temp\ipykernel_25252\3802403867.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cx_equiv_cx_tri_proximo['DT_REFER'] = pd.to_datetime(cx_equiv_cx_tri_proximo['DT_REFER'])


In [32]:
# Definindo o caminho do relatório para o ano corrente e ano próximo
caminho_rel_corrente = f'{diretorio_destino_arq_trim}/itr_cia_aberta_BPP_con_{ano_corrente}.csv'
caminho_rel_proximo = f'{diretorio_destino_arq_trim}/itr_cia_aberta_BPP_con_{ano_proximo}.csv'

# Lendo os arquivos de relatório para o ano corrente e próximo, ignorando linhas com erros
df_tri_corrente = pd.read_csv(caminho_rel_corrente, encoding='cp1252', sep=';', on_bad_lines='skip')
df_tri_proximo = pd.read_csv(caminho_rel_proximo, encoding='cp1252', sep=';', on_bad_lines='skip')

### Cálculo da Dívida Total

# 1. Parte 1 (CD_CONTA == '2.01.04')
divida_total_1_corrente = df_tri_corrente[(df_tri_corrente['ORDEM_EXERC'] == 'ÚLTIMO') & (df_tri_corrente['CD_CONTA'] == '2.01.04')]
divida_total_1_prox = df_tri_proximo[(df_tri_proximo['ORDEM_EXERC'] == 'ÚLTIMO') & (df_tri_proximo['CD_CONTA'] == '2.01.04')]

# 2. Parte 2 (CD_CONTA == '2.02.01')
divida_total_2_corrente = df_tri_corrente[(df_tri_corrente['ORDEM_EXERC'] == 'ÚLTIMO') & (df_tri_corrente['CD_CONTA'] == '2.02.01')]
divida_total_2_prox = df_tri_proximo[(df_tri_proximo['ORDEM_EXERC'] == 'ÚLTIMO') & (df_tri_proximo['CD_CONTA'] == '2.02.01')]

### Concatenando e Preparando os Dados

# Função para preparar os dados de dívida total
def preparar_dados_divida(df_corrente, df_prox):
    df_corrente['DT_REFER'] = pd.to_datetime(df_corrente['DT_REFER'])
    df_prox['DT_REFER'] = pd.to_datetime(df_prox['DT_REFER'])

    df_total = pd.concat([df_corrente, df_prox], ignore_index=True)

    # Criar a coluna 'Quarter' no formato 'Q1_2023'
    df_total['Quarter'] = 'Q' + df_total['DT_REFER'].dt.quarter.astype(str) + '_' + df_total['DT_REFER'].dt.year.astype(str)

    # Pivotar os dados para organizar por trimestres
    pivot = df_total.pivot_table(values='VL_CONTA', index=['DENOM_CIA', 'CNPJ_CIA'], columns='Quarter', aggfunc='last')
    pivot.reset_index(inplace=True)
    pivot.fillna(0, inplace=True)

    return pivot

# Preparar dados para Parte 1 e Parte 2 da dívida
divida_total_1 = preparar_dados_divida(divida_total_1_corrente, divida_total_1_prox)
divida_total_2 = preparar_dados_divida(divida_total_2_corrente, divida_total_2_prox)

### Somando as duas Partes da Dívida Total

# Configurando os índices e somando as duas partes
divida_total_1.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)
divida_total_2.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)

divida_total_trim = divida_total_1.add(divida_total_2, fill_value=0)
divida_total_trim.reset_index(inplace=True)




C:\Users\User\AppData\Local\Temp\ipykernel_25252\845505469.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_corrente['DT_REFER'] = pd.to_datetime(df_corrente['DT_REFER'])
C:\Users\User\AppData\Local\Temp\ipykernel_25252\845505469.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_prox['DT_REFER'] = pd.to_datetime(df_prox['DT_REFER'])
C:\Users\User\AppData\Local\Temp\ipykernel_25252\845505469.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

In [33]:
# Configurando as colunas 'DENOM_CIA' e 'CNPJ_CIA' como índices nos DataFrames
divida_total_trim.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)
cx_equiv_cx_trim.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)

# Substituindo valores NaN por 0
divida_total_trim.fillna(0, inplace=True)
cx_equiv_cx_trim.fillna(0, inplace=True)

# Realizando a operação de subtração entre os DataFrames
divida_liquida_trim = divida_total_trim.sub(cx_equiv_cx_trim, fill_value=0)

# Resetando o índice para transformar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
divida_liquida_trim.reset_index(inplace=True)

# Identificar a coluna de 'Q1' do próximo ano de forma automática
coluna_proximo_ano_divida = [col for col in divida_liquida_trim.columns if col.startswith('Q1') and any(char.isdigit() for char in col)]

# Reordenar as colunas, movendo a coluna do próximo ano para o final
colunas_ordenadas_divida = [col for col in divida_liquida_trim.columns if col not in coluna_proximo_ano_divida] + coluna_proximo_ano_divida

# Aplicar a nova ordem de colunas ao DataFrame
divida_liquida_trim = divida_liquida_trim[colunas_ordenadas_divida]

# Definindo a ordem cronológica para as colunas, mantendo 'DENOM_CIA' e 'CNPJ_CIA' no início
colunas_ordenadas_divida = ['DENOM_CIA', 'CNPJ_CIA'] + sorted([col for col in divida_liquida_trim.columns if col not in ['DENOM_CIA', 'CNPJ_CIA']], key=lambda x: (x[-4:], x))

# Aplicar a nova ordem de colunas ao DataFrame
divida_liquida_trim = divida_liquida_trim[colunas_ordenadas_divida]


In [34]:
divida_liquida_trim

Quarter,DENOM_CIA,CNPJ_CIA,Q1_2023,Q2_2023,Q3_2023,Q4_2023,Q1_2024,Q2_2024,Q3_2024
0,2W ECOBANK S.A.,08.773.135/0001-00,1484119.0,1826254.0,1929850.0,0.0,0.0,0.0,0.0
1,3R PETROLEUM ÓLEO E GÁS S.A.,12.091.809/0001-55,382731.0,7318832.0,7774420.0,0.0,8550803.0,9579240.0,0.0
2,AEGEA SANEAMENTO E PARTICIPAÇÕES S.A.,08.827.501/0001-58,10166449.0,10758101.0,12793440.0,0.0,16889116.0,19253433.0,0.0
3,AERIS IND. E COM. DE EQUIP. PARA GER. DE ENG. ...,12.528.708/0001-07,984805.0,938618.0,1023531.0,0.0,774936.0,566252.0,0.0
4,AES BRASIL ENERGIA S.A.,37.663.076/0001-07,11808734.0,11563564.0,11673092.0,0.0,11407318.0,12020344.0,0.0
...,...,...,...,...,...,...,...,...,...
480,YBYRÁ CAPITAL S.A.,02.217.319/0001-07,-2517.0,-4298.0,-758497.0,0.0,-942789.0,-272.0,0.0
481,YDUQS PARTICIPACOES S.A.,08.807.432/0001-10,4681326.0,4606853.0,4457546.0,0.0,4571322.0,4798653.0,0.0
482,YOU INC INCORPORADORA E PARTICIPAÇÃO S.A,11.284.204/0001-18,784094.0,859466.0,831039.0,0.0,901986.0,863436.0,0.0
483,YUNY INCORPORADORA HOLDING S.A.,09.267.996/0001-70,128518.0,138317.0,130354.0,0.0,152147.0,188231.0,0.0


Ebitda

ebitda = LAIR + juros + depreciação + amortização

Histórico

Depreciação e amortização

In [35]:
# Definindo relatório a ser aberto
relatorio = "dfp_cia_aberta_DVA_con"  # inserir apenas o nome do arquivo, sem o ano

# DataFrame vazio para armazenar os resultados
resultados_dva = pd.DataFrame()

# Loop pelos anos
for ano in anos:
    # Definindo o caminho do arquivo usando formatação de string
    caminho_arquivo = f'{diretorio_destino_arq_csv}/{ano}/{relatorio}_{ano}.csv'

    # Lendo o arquivo CSV
    df = pd.read_csv(caminho_arquivo, encoding='utf-8', sep=',')

    # Filtrando os dados para encontrar o Lucro Líquido do Período
    var_DVA = df[(df['ORDEM_EXERC'] == 'ÚLTIMO') & (df['CD_CONTA'] == '7.04.01')]

    # Selecionando colunas desejadas
    var_DVA = var_DVA[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA']]

    # Adicionando a coluna 'Ano' com o valor correspondente
    var_DVA['Ano'] = ano

    # Adicionando os resultados ao DataFrame final
    resultados_dva = pd.concat([resultados_dva, var_DVA], ignore_index=True)

# Agregando os valores duplicados
resultados_agregados_dva = resultados_dva.groupby(['DENOM_CIA', 'CNPJ_CIA', 'Ano'])['VL_CONTA'].sum().reset_index()

# Criando um DataFrame pivotado
deprec_amortz = resultados_agregados_dva.pivot_table(index=['DENOM_CIA', 'CNPJ_CIA'], columns='Ano', values='VL_CONTA', aggfunc='sum').reset_index()

# Renomeando as colunas para remover o nome da coluna do índice
deprec_amortz.name = None


LAIR

In [36]:
# Definindo relatório a ser aberto
relatorio = "dfp_cia_aberta_DRE_con"  # inserir apenas o nome do arquivo, sem o ano

# DataFrame vazio para armazenar os resultados
resultados_dre = pd.DataFrame()

# Loop pelos anos
for ano in anos:
    # Definindo o caminho do arquivo usando formatação de string
    caminho_arquivo = f'{diretorio_destino_arq_csv}/{ano}/{relatorio}_{ano}.csv'

    # Lendo o arquivo CSV
    df = pd.read_csv(caminho_arquivo, encoding='utf-8', sep=',')

    # Filtrando os dados para encontrar o Lucro Líquido do Período
    var_DRE = df[(df['ORDEM_EXERC'] == 'ÚLTIMO') & (df['CD_CONTA'] == '3.05')]

    # Selecionando colunas desejadas
    var_DRE = var_DRE[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA']]

    # Adicionando a coluna 'Ano' com o valor correspondente
    var_DRE['Ano'] = ano

    # Adicionando os resultados ao DataFrame final
    resultados_dre = pd.concat([resultados_dre, var_DRE], ignore_index=True)

# Agregando os valores duplicados
resultados_agregados_dre = resultados_dre.groupby(['DENOM_CIA', 'CNPJ_CIA', 'Ano'])['VL_CONTA'].sum().reset_index()

# Criando um DataFrame pivotado
LAIR = resultados_agregados_dre.pivot_table(index=['DENOM_CIA', 'CNPJ_CIA'], columns='Ano', values='VL_CONTA', aggfunc='sum').reset_index()

# Renomeando as colunas para remover o nome da coluna do índice
LAIR.name = None




In [37]:
# Configurando as colunas 'DENOM_CIA' e 'CNPJ_CIA' como índices nos DataFrames
deprec_amortz.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)
LAIR.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)


# Substituindo valores NaN por 0
deprec_amortz.fillna(0, inplace=True)
LAIR.fillna(0, inplace=True)

# Calculando o EBTIDA (soma das colunas deprec_amortz, juros e LAIR)
EBITDA = LAIR.sub(deprec_amortz, fill_value=0)

# Resetando o índice para tornar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
EBITDA.reset_index(inplace=True)

Trimestral

In [38]:
# Definindo o caminho do arquivo CSV para o ano corrente e próximo
caminho_arquivo_tri_corrente = f'{diretorio_destino_arq_trim}/itr_cia_aberta_DVA_con_{ano_corrente}.csv'
caminho_arquivo_tri_prox = f'{diretorio_destino_arq_trim}/itr_cia_aberta_DVA_con_{ano_proximo}.csv'

# Lendo os arquivos CSV e ignorando as linhas com formatação incorreta
df_tri_corrente = pd.read_csv(caminho_arquivo_tri_corrente, encoding='cp1252', sep=';', on_bad_lines='skip')
df_tri_prox = pd.read_csv(caminho_arquivo_tri_prox, encoding='cp1252', sep=';', on_bad_lines='skip')

# 1. Cálculo da Depreciação e Amortização para o ano corrente

# Filtrando os dados para encontrar a Depreciação e Amortização (CD_CONTA == '7.04.01')
deprec_amortz_tri_corrente = df_tri_corrente[(df_tri_corrente['ORDEM_EXERC'] == 'ÚLTIMO') & (df_tri_corrente['CD_CONTA'] == '7.04.01')]

# Converter as colunas 'DT_INI_EXERC' e 'DT_FIM_EXERC' para o tipo datetime
deprec_amortz_tri_corrente['DT_INI_EXERC'] = pd.to_datetime(deprec_amortz_tri_corrente['DT_INI_EXERC'])
deprec_amortz_tri_corrente['DT_FIM_EXERC'] = pd.to_datetime(deprec_amortz_tri_corrente['DT_FIM_EXERC'])

# Calcular a diferença entre DT_FIM_EXERC e DT_INI_EXERC
deprec_amortz_tri_corrente['DIFERENCA'] = (deprec_amortz_tri_corrente['DT_FIM_EXERC'] - deprec_amortz_tri_corrente['DT_INI_EXERC']).abs()

# Encontrar os índices da menor diferença para cada grupo de DENOM_CIA, CNPJ_CIA, DT_FIM_EXERC
indices_min_DT_INI_corrente = deprec_amortz_tri_corrente.groupby(['DENOM_CIA', 'CNPJ_CIA', 'DT_FIM_EXERC'])['DIFERENCA'].idxmin()

# Filtrar o DataFrame principal usando os índices dos valores mínimos de 'DIFERENCA'
deprec_amortz_trime_corrente = deprec_amortz_tri_corrente.loc[indices_min_DT_INI_corrente]

# 2. Cálculo da Depreciação e Amortização para o ano próximo

# Filtrando os dados para encontrar a Depreciação e Amortização (CD_CONTA == '7.04.01')
deprec_amortz_tri_prox = df_tri_prox[(df_tri_prox['ORDEM_EXERC'] == 'ÚLTIMO') & (df_tri_prox['CD_CONTA'] == '7.04.01')]

# Converter as colunas 'DT_INI_EXERC' e 'DT_FIM_EXERC' para o tipo datetime
deprec_amortz_tri_prox['DT_INI_EXERC'] = pd.to_datetime(deprec_amortz_tri_prox['DT_INI_EXERC'])
deprec_amortz_tri_prox['DT_FIM_EXERC'] = pd.to_datetime(deprec_amortz_tri_prox['DT_FIM_EXERC'])

# Calcular a diferença entre DT_FIM_EXERC e DT_INI_EXERC
deprec_amortz_tri_prox['DIFERENCA'] = (deprec_amortz_tri_prox['DT_FIM_EXERC'] - deprec_amortz_tri_prox['DT_INI_EXERC']).abs()

# Encontrar os índices da menor diferença para cada grupo de DENOM_CIA, CNPJ_CIA, DT_FIM_EXERC
indices_min_DT_INI_prox = deprec_amortz_tri_prox.groupby(['DENOM_CIA', 'CNPJ_CIA', 'DT_FIM_EXERC'])['DIFERENCA'].idxmin()

# Filtrar o DataFrame principal usando os índices dos valores mínimos de 'DIFERENCA'
deprec_amortz_trime_prox = deprec_amortz_tri_prox.loc[indices_min_DT_INI_prox]

# Concatenando os dois DataFrames (ano corrente e ano próximo)
deprec_amortz_tri = pd.concat([deprec_amortz_trime_corrente, deprec_amortz_trime_prox], ignore_index=True)

# Criando uma coluna de trimestre (Q1, Q2, etc.)
deprec_amortz_tri['DT_REFER'] = pd.to_datetime(deprec_amortz_tri['DT_REFER'])
deprec_amortz_tri['Quarter'] = deprec_amortz_tri['DT_REFER'].dt.to_period('Q')

# Criando um novo DataFrame com as colunas corretas para cada trimestre
pivot_deprec_amortz = deprec_amortz_tri.pivot_table(values='VL_CONTA', index=['DENOM_CIA', 'CNPJ_CIA'], columns='Quarter', aggfunc='first')

# Resetando o índice para transformar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
pivot_deprec_amortz.reset_index(inplace=True)

# Preenchendo valores NaN com 0, se necessário
pivot_deprec_amortz.fillna(0, inplace=True)

# Renomeando as colunas para incluir o trimestre
pivot_deprec_amortz.columns = [f'Q{col.quarter}' if isinstance(col, pd.Period) else col for col in pivot_deprec_amortz.columns]

# DataFrame final para Depreciação e Amortização
deprec_amortz_trim = pivot_deprec_amortz

C:\Users\User\AppData\Local\Temp\ipykernel_25252\465901249.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  deprec_amortz_tri_corrente['DT_INI_EXERC'] = pd.to_datetime(deprec_amortz_tri_corrente['DT_INI_EXERC'])
C:\Users\User\AppData\Local\Temp\ipykernel_25252\465901249.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  deprec_amortz_tri_corrente['DT_FIM_EXERC'] = pd.to_datetime(deprec_amortz_tri_corrente['DT_FIM_EXERC'])
C:\Users\User\AppData\Local\Temp\ipykernel_25252\465901249.py:19: SettingWithCop

In [39]:
# Convertendo a coluna 'DT_REFER' para o tipo datetime
deprec_amortz_tri['DT_REFER'] = pd.to_datetime(deprec_amortz_tri['DT_REFER'])

# Criando uma coluna de trimestre (Q1, Q2, etc.)
deprec_amortz_tri['Quarter'] = deprec_amortz_tri['DT_REFER'].dt.to_period('Q')

# Criando um novo DataFrame com as colunas corretas para cada trimestre
pivot = deprec_amortz_tri.pivot_table(values='VL_CONTA', index=['DENOM_CIA', 'CNPJ_CIA'], columns='Quarter', aggfunc='first')

# Resetando o índice para transformar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
pivot.reset_index(inplace=True)

# Preenchendo valores NaN com 0, se necessário
pivot.fillna(0, inplace=True)

# Renomeando as colunas para incluir o trimestre e o ano (ex: Q1_2023, Q2_2023, etc.)
pivot.columns = [f'Q{col.quarter}_{col.start_time.year}' if isinstance(col, pd.Period) else col for col in pivot.columns]

# Remover possíveis duplicatas de colunas geradas incorretamente
pivot = pivot.loc[:, ~pivot.columns.duplicated()]

# Verificar se as colunas 'Q2' e 'Q1' existem e realizar a subtração
if any('Q2' in col for col in pivot.columns) and any('Q1' in col for col in pivot.columns):
    coluna_q2 = [col for col in pivot.columns if 'Q2' in col][0]
    coluna_q1 = [col for col in pivot.columns if 'Q1' in col][0]
    pivot[coluna_q2] = pivot[coluna_q2] - pivot[coluna_q1]

# Verificar se as colunas 'Q3', 'Q2', e 'Q1' existem e realizar a subtração
if any('Q3' in col for col in pivot.columns) and any('Q2' in col for col in pivot.columns) and any('Q1' in col for col in pivot.columns):
    coluna_q3 = [col for col in pivot.columns if 'Q3' in col][0]
    coluna_q2 = [col for col in pivot.columns if 'Q2' in col][0]
    coluna_q1 = [col for col in pivot.columns if 'Q1' in col][0]
    pivot[coluna_q3] = pivot[coluna_q3] - pivot[coluna_q2] - pivot[coluna_q1]

# Definir o DataFrame final para depreciação e amortização
deprec_amortz_trim = pivot





In [40]:
# Definindo o caminho do arquivo CSV para o ano corrente
caminho_arquivo_tri_corrente = f'{diretorio_destino_arq_trim}/itr_cia_aberta_DRE_con_{ano_corrente}.csv'
# Definindo o caminho do arquivo CSV para o ano próximo
caminho_arquivo_tri_prox = f'{diretorio_destino_arq_trim}/itr_cia_aberta_DRE_con_{ano_proximo}.csv'

# Lendo o arquivo CSV para o ano corrente e o próximo
df_tri_corrente = pd.read_csv(caminho_arquivo_tri_corrente, encoding='cp1252', sep=';', on_bad_lines='skip')
df_tri_prox = pd.read_csv(caminho_arquivo_tri_prox, encoding='cp1252', sep=';', on_bad_lines='skip')

# Filtrando os dados para encontrar o LAIR (CD_CONTA == '3.05')
LAIR_tri_corrente = df_tri_corrente[(df_tri_corrente['ORDEM_EXERC'] == 'ÚLTIMO') & (df_tri_corrente['CD_CONTA'] == '3.05')]
LAIR_tri_prox = df_tri_prox[(df_tri_prox['ORDEM_EXERC'] == 'ÚLTIMO') & (df_tri_prox['CD_CONTA'] == '3.05')]

# Converter as colunas 'DT_INI_EXERC' e 'DT_FIM_EXERC' para o tipo datetime
LAIR_tri_corrente['DT_INI_EXERC'] = pd.to_datetime(LAIR_tri_corrente['DT_INI_EXERC'])
LAIR_tri_corrente['DT_FIM_EXERC'] = pd.to_datetime(LAIR_tri_corrente['DT_FIM_EXERC'])

LAIR_tri_prox['DT_INI_EXERC'] = pd.to_datetime(LAIR_tri_prox['DT_INI_EXERC'])
LAIR_tri_prox['DT_FIM_EXERC'] = pd.to_datetime(LAIR_tri_prox['DT_FIM_EXERC'])

# Calcular a diferença entre DT_FIM_EXERC e DT_INI_EXERC para ambos os anos
LAIR_tri_corrente['DIFERENCA'] = (LAIR_tri_corrente['DT_FIM_EXERC'] - LAIR_tri_corrente['DT_INI_EXERC']).abs()
LAIR_tri_prox['DIFERENCA'] = (LAIR_tri_prox['DT_FIM_EXERC'] - LAIR_tri_prox['DT_INI_EXERC']).abs()

# Encontrar os índices da menor diferença para cada grupo de DENOM_CIA, CNPJ_CIA e DT_FIM_EXERC
indices_min_DT_INI_corrente = LAIR_tri_corrente.groupby(['DENOM_CIA', 'CNPJ_CIA', 'DT_FIM_EXERC'])['DIFERENCA'].idxmin()
indices_min_DT_INI_prox = LAIR_tri_prox.groupby(['DENOM_CIA', 'CNPJ_CIA', 'DT_FIM_EXERC'])['DIFERENCA'].idxmin()

# Filtrar o DataFrame principal usando os índices dos valores mínimos de 'DIFERENCA'
LAIR_tri_corrente_filtrado = LAIR_tri_corrente.loc[indices_min_DT_INI_corrente]
LAIR_tri_prox_filtrado = LAIR_tri_prox.loc[indices_min_DT_INI_prox]

# Selecionar colunas desejadas e adicionar a coluna 'Ano' com o valor correspondente para ambos os DataFrames
LAIR_tri_corrente_filtrado = LAIR_tri_corrente_filtrado[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA', 'DT_REFER']]
LAIR_tri_corrente_filtrado['Ano_tri'] = ano_corrente

LAIR_tri_prox_filtrado = LAIR_tri_prox_filtrado[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA', 'DT_REFER']]
LAIR_tri_prox_filtrado['Ano_tri'] = ano_proximo

# Concatenando os DataFrames do ano corrente e do ano próximo
LAIR_tri = pd.concat([LAIR_tri_corrente_filtrado, LAIR_tri_prox_filtrado], ignore_index=True)





C:\Users\User\AppData\Local\Temp\ipykernel_25252\818124257.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LAIR_tri_corrente['DT_INI_EXERC'] = pd.to_datetime(LAIR_tri_corrente['DT_INI_EXERC'])
C:\Users\User\AppData\Local\Temp\ipykernel_25252\818124257.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LAIR_tri_corrente['DT_FIM_EXERC'] = pd.to_datetime(LAIR_tri_corrente['DT_FIM_EXERC'])
C:\Users\User\AppData\Local\Temp\ipykernel_25252\818124257.py:18: SettingWithCopyWarning: 
A value is trying to be s

In [41]:
# Convertendo a coluna 'DT_REFER' para o tipo datetime (ano corrente)
LAIR_tri_corrente['DT_REFER'] = pd.to_datetime(LAIR_tri_corrente['DT_REFER'])

# Convertendo a coluna 'DT_REFER' para o tipo datetime (ano próximo)
LAIR_tri_prox['DT_REFER'] = pd.to_datetime(LAIR_tri_prox['DT_REFER'])

# Criando uma coluna de trimestre (Q1, Q2, etc.) para ambos os anos
LAIR_tri_corrente['Quarter'] = LAIR_tri_corrente['DT_REFER'].dt.to_period('Q')
LAIR_tri_prox['Quarter'] = LAIR_tri_prox['DT_REFER'].dt.to_period('Q')

# Criando DataFrames pivotados para os dois anos
pivot_corrente = LAIR_tri_corrente.pivot_table(values='VL_CONTA', index=['DENOM_CIA', 'CNPJ_CIA'], columns='Quarter', aggfunc='first')
pivot_prox = LAIR_tri_prox.pivot_table(values='VL_CONTA', index=['DENOM_CIA', 'CNPJ_CIA'], columns='Quarter', aggfunc='first')

# Resetando o índice para transformar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
pivot_corrente.reset_index(inplace=True)
pivot_prox.reset_index(inplace=True)

# Preenchendo valores NaN com 0, se necessário
pivot_corrente.fillna(0, inplace=True)
pivot_prox.fillna(0, inplace=True)

# Renomeando as colunas para incluir o trimestre e o ano (ex: Q1_2023, Q2_2023, etc.)
pivot_corrente.columns = [f'Q{col.quarter}_{col.start_time.year}' if isinstance(col, pd.Period) else col for col in pivot_corrente.columns]
pivot_prox.columns = [f'Q{col.quarter}_{col.start_time.year}' if isinstance(col, pd.Period) else col for col in pivot_prox.columns]

# Fazer o merge dos dois DataFrames para garantir alinhamento correto entre os anos
pivot = pd.merge(pivot_corrente, pivot_prox, on=['DENOM_CIA', 'CNPJ_CIA'], how='outer')

# Preenchendo valores NaN com 0, se necessário após o merge
pivot.fillna(0, inplace=True)

# Remover possíveis duplicatas de colunas geradas incorretamente
pivot = pivot.loc[:, ~pivot.columns.duplicated()]

# Verificar e calcular Q2 - Q1 se Q2 estiver presente
if any('Q2' in col for col in pivot.columns) and any('Q1' in col for col in pivot.columns):
    coluna_q2 = [col for col in pivot.columns if 'Q2' in col][0]
    coluna_q1 = [col for col in pivot.columns if 'Q1' in col][0]
    pivot[coluna_q2] = pivot[coluna_q2] - pivot[coluna_q1]

# Verificar e calcular Q3 - Q2 - Q1 se Q3 estiver presente
if any('Q3' in col for col in pivot.columns) and any('Q2' in col for col in pivot.columns) and any('Q1' in col for col in pivot.columns):
    coluna_q3 = [col for col in pivot.columns if 'Q3' in col][0]
    coluna_q2 = [col for col in pivot.columns if 'Q2' in col][0]
    coluna_q1 = [col for col in pivot.columns if 'Q1' in col][0]
    pivot[coluna_q3] = pivot[coluna_q3] - pivot[coluna_q2] - pivot[coluna_q1]

# DataFrame final com o LAIR trimestral ajustado para ambos os anos
LAIR_trim = pivot








C:\Users\User\AppData\Local\Temp\ipykernel_25252\2702014398.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LAIR_tri_corrente['DT_REFER'] = pd.to_datetime(LAIR_tri_corrente['DT_REFER'])
C:\Users\User\AppData\Local\Temp\ipykernel_25252\2702014398.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LAIR_tri_prox['DT_REFER'] = pd.to_datetime(LAIR_tri_prox['DT_REFER'])
C:\Users\User\AppData\Local\Temp\ipykernel_25252\2702014398.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

In [42]:
# Configurando as colunas 'DENOM_CIA' e 'CNPJ_CIA' como índices nos DataFrames
deprec_amortz_trim.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)
LAIR_trim.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)

# Substituindo valores NaN por 0
deprec_amortz_trim.fillna(0, inplace=True)
LAIR_trim.fillna(0, inplace=True)

# Calculando o EBITDA trimestral (subtração de deprec_amortz_trim do LAIR_trim)
EBITDA_trim = LAIR_trim.sub(deprec_amortz_trim, fill_value=0)

# Resetando o índice para tornar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
EBITDA_trim.reset_index(inplace=True)

# Identificar a coluna de 'Q1' do próximo ano de forma automática
coluna_proximo_ano = [col for col in EBITDA_trim.columns if col.startswith('Q1') and any(char.isdigit() for char in col)]

# Reordenar as colunas, movendo a coluna do próximo ano para o final
colunas_ordenadas = [col for col in EBITDA_trim.columns if col not in coluna_proximo_ano] + coluna_proximo_ano

# Aplicar a nova ordem de colunas ao DataFrame
EBITDA_trim = EBITDA_trim[colunas_ordenadas]



# Definindo a ordem cronológica para as colunas, mantendo 'DENOM_CIA' e 'CNPJ_CIA' no início
colunas_ordenadas = ['DENOM_CIA', 'CNPJ_CIA'] + sorted([col for col in EBITDA_trim.columns if col not in ['DENOM_CIA', 'CNPJ_CIA']], key=lambda x: (x[-4:], x))

# Aplicar a nova ordem de colunas ao DataFrame
EBITDA_trim = EBITDA_trim[colunas_ordenadas]



Margem EBITDA =  EBITDA / Rceita Líquida

Histórico

In [43]:
# Configurando as colunas 'DENOM_CIA' e 'CNPJ_CIA' como índices nos DataFrames históricos
EBITDA.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)
receita_liquida.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)

# Substituindo valores NaN por 0
EBITDA.fillna(0, inplace=True)
receita_liquida.fillna(0, inplace=True)

# Calculando Margem EBITDA (EBITDA/Receita Líquida) para os dados históricos
margem_ebitda = EBITDA.div(receita_liquida, fill_value=0)

# Resetando o índice para tornar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
margem_ebitda.reset_index(inplace=True)

trimestral

In [44]:
# Configurando as colunas 'DENOM_CIA' e 'CNPJ_CIA' como índices nos DataFrames trimestrais
EBITDA_trim.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)
receita_liquida_trim.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)

# Substituindo valores NaN por 0
EBITDA_trim.fillna(0, inplace=True)
receita_liquida_trim.fillna(0, inplace=True)

# Calculando Margem EBITDA (EBITDA/Receita Líquida) para os dados trimestrais
margem_ebitda_trim = EBITDA_trim.div(receita_liquida_trim, fill_value=0)

# Resetando o índice para tornar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
margem_ebitda_trim.reset_index(inplace=True)

In [45]:
margem_ebitda_trim

,DENOM_CIA,CNPJ_CIA,Q1_2023,Q2_2023,Q3_2023,Q4_2023,Q1_2024,Q2_2024,Q3_2024
0,2W ECOBANK S.A.,08.773.135/0001-00,0.371606,0.220850,-0.010835,NaN,NaN,NaN,NaN
1,3R PETROLEUM ÓLEO E GÁS S.A.,12.091.809/0001-55,0.271228,0.219600,0.351051,NaN,0.360915,0.608521,NaN
2,AEGEA SANEAMENTO E PARTICIPAÇÕES S.A.,08.827.501/0001-58,0.671529,0.539687,0.463738,NaN,0.496375,0.880517,NaN
3,AERIS IND. E COM. DE EQUIP. PARA GER. DE ENG. ...,12.528.708/0001-07,0.113781,0.106282,0.087991,NaN,0.068975,0.259571,NaN
4,AES BRASIL ENERGIA S.A.,37.663.076/0001-07,0.509299,0.541434,0.499334,NaN,0.407312,0.802574,NaN
...,...,...,...,...,...,...,...,...,...
482,YBYRÁ CAPITAL S.A.,02.217.319/0001-07,inf,0.517885,0.645316,NaN,-inf,-inf,NaN
483,YDUQS PARTICIPACOES S.A.,08.807.432/0001-10,0.376438,0.317427,0.349646,NaN,0.347654,0.701525,NaN
484,YOU INC INCORPORADORA E PARTICIPAÇÃO S.A,11.284.204/0001-18,0.063219,0.047227,0.170581,NaN,0.024657,0.145803,NaN
485,YUNY INCORPORADORA HOLDING S.A.,09.267.996/0001-70,-0.028844,0.397680,0.102327,NaN,0.056479,0.088467,NaN


Free cash flow = fluxo de caixa de atividades operacionais + investimentos em ativos de capital

Histórico

In [46]:
# Definindo relatório a ser aberto
relatorio = "dfp_cia_aberta_DFC_MI_con"  # inserir apenas o nome do arquivo, sem o ano

# Lista de anos desejados
anos = list(range(ano_inicial, ano_final+1))

# DataFrame vazio para armazenar os resultados
resultados_cfc_mi = pd.DataFrame()

# Loop pelos anos
for ano in anos:
    # Definindo o caminho do arquivo usando formatação de string
    caminho_arquivo = f'{diretorio_destino_arq_csv}/{ano}/{relatorio}_{ano}.csv'

    # Lendo o arquivo CSV
    df = pd.read_csv(caminho_arquivo, encoding='utf-8', sep=',')

    # Filtrando os dados para encontrar o Lucro Líquido do Período
    var_CFC_MI = df[(df['ORDEM_EXERC'] == 'ÚLTIMO') & (df['CD_CONTA'] == '6.01')]

    # Selecionando colunas desejadas
    var_CFC_MI = var_CFC_MI[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA']]

    # Adicionando a coluna 'Ano' com o valor correspondente
    var_CFC_MI['Ano'] = ano

    # Adicionando os resultados ao DataFrame final
    resultados_cfc_mi = pd.concat([resultados_cfc_mi, var_CFC_MI], ignore_index=True)

# Agregando os valores duplicados
resultados_agregados_cfc_mi= resultados_cfc_mi.groupby(['DENOM_CIA', 'CNPJ_CIA', 'Ano'])['VL_CONTA'].sum().reset_index()

# Criando um DataFrame pivotado
fluxo_caixa = resultados_agregados_cfc_mi.pivot_table(index=['DENOM_CIA', 'CNPJ_CIA'], columns='Ano', values='VL_CONTA', aggfunc='sum').reset_index()

# Renomeando as colunas para remover o nome da coluna do índice
fluxo_caixa.name = None


In [47]:
# Definindo relatório a ser aberto
relatorio = "dfp_cia_aberta_DFC_MI_con"  # inserir apenas o nome do arquivo, sem o ano

# Lista de anos desejados
anos = list(range(ano_inicial, ano_final+1))

# DataFrame vazio para armazenar os resultados
resultados_cfc_mi = pd.DataFrame()

# Loop pelos anos
for ano in anos:
    # Definindo o caminho do arquivo usando formatação de string
    caminho_arquivo = f'{diretorio_destino_arq_csv}/{ano}/{relatorio}_{ano}.csv'

    # Lendo o arquivo CSV
    df = pd.read_csv(caminho_arquivo, encoding='utf-8', sep=',')

    # Filtrando os dados para encontrar o Lucro Líquido do Período
    var_CFC_MI = df[(df['ORDEM_EXERC'] == 'ÚLTIMO') & (df['CD_CONTA'] == '6.02')]

    # Selecionando colunas desejadas
    var_CFC_MI = var_CFC_MI[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA']]

    # Adicionando a coluna 'Ano' com o valor correspondente
    var_CFC_MI['Ano'] = ano

    # Adicionando os resultados ao DataFrame final
    resultados_cfc_mi = pd.concat([resultados_cfc_mi, var_CFC_MI], ignore_index=True)

# Agregando os valores duplicados
resultados_agregados_cfc_mi= resultados_cfc_mi.groupby(['DENOM_CIA', 'CNPJ_CIA', 'Ano'])['VL_CONTA'].sum().reset_index()

# Criando um DataFrame pivotado
capex = resultados_agregados_cfc_mi.pivot_table(index=['DENOM_CIA', 'CNPJ_CIA'], columns='Ano', values='VL_CONTA', aggfunc='sum').reset_index()

# Renomeando as colunas para remover o nome da coluna do índice
capex.name = None


In [48]:
# Configurando as colunas 'DENOM_CIA' e 'CNPJ_CIA' como índices nos DataFrames
fluxo_caixa.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)
capex.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)

# Substituindo valores NaN por 0
fluxo_caixa.fillna(0, inplace=True)
capex.fillna(0, inplace=True)

# Subtraindo os valores para obter divida_liquida
free_cash_flow = fluxo_caixa.add(capex, fill_value=0)

# Resetando o índice para tornar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
free_cash_flow.reset_index(inplace=True)

Trimestral

In [49]:
# Definindo o caminho do arquivo CSV para o ano corrente e ano próximo
caminho_arquivo_tri_corrente = f'{diretorio_destino_arq_trim}/itr_cia_aberta_DFC_MI_con_{ano_corrente}.csv'
caminho_arquivo_tri_proximo = f'{diretorio_destino_arq_trim}/itr_cia_aberta_DFC_MI_con_{ano_proximo}.csv'

# Lendo o arquivo CSV para o ano corrente e próximo, ignorando as linhas com formatação incorreta
df_tri_corrente = pd.read_csv(caminho_arquivo_tri_corrente, encoding='cp1252', sep=';', on_bad_lines='skip')
df_tri_proximo = pd.read_csv(caminho_arquivo_tri_proximo, encoding='cp1252', sep=';', on_bad_lines='skip')

# Filtrando os dados para encontrar o Fluxo de Caixa Operacional (CD_CONTA == '6.01')
fluxo_caixa_tri_corrente = df_tri_corrente[(df_tri_corrente['ORDEM_EXERC'] == 'ÚLTIMO') & (df_tri_corrente['CD_CONTA'] == '6.01')]
fluxo_caixa_tri_proximo = df_tri_proximo[(df_tri_proximo['ORDEM_EXERC'] == 'ÚLTIMO') & (df_tri_proximo['CD_CONTA'] == '6.01')]

# Converter as colunas 'DT_INI_EXERC' e 'DT_FIM_EXERC' para o tipo datetime para ambos os anos
fluxo_caixa_tri_corrente['DT_INI_EXERC'] = pd.to_datetime(fluxo_caixa_tri_corrente['DT_INI_EXERC'])
fluxo_caixa_tri_corrente['DT_FIM_EXERC'] = pd.to_datetime(fluxo_caixa_tri_corrente['DT_FIM_EXERC'])

fluxo_caixa_tri_proximo['DT_INI_EXERC'] = pd.to_datetime(fluxo_caixa_tri_proximo['DT_INI_EXERC'])
fluxo_caixa_tri_proximo['DT_FIM_EXERC'] = pd.to_datetime(fluxo_caixa_tri_proximo['DT_FIM_EXERC'])

# Calcular a diferença entre DT_FIM_EXERC e DT_INI_EXERC para ambos os anos
fluxo_caixa_tri_corrente['DIFERENCA'] = (fluxo_caixa_tri_corrente['DT_FIM_EXERC'] - fluxo_caixa_tri_corrente['DT_INI_EXERC']).abs()
fluxo_caixa_tri_proximo['DIFERENCA'] = (fluxo_caixa_tri_proximo['DT_FIM_EXERC'] - fluxo_caixa_tri_proximo['DT_INI_EXERC']).abs()

# Encontrar os índices da menor diferença para cada grupo de DENOM_CIA, CNPJ_CIA, DT_FIM_EXERC
indices_min_DT_INI_corrente = fluxo_caixa_tri_corrente.groupby(['DENOM_CIA', 'CNPJ_CIA', 'DT_FIM_EXERC'])['DIFERENCA'].idxmin()
indices_min_DT_INI_proximo = fluxo_caixa_tri_proximo.groupby(['DENOM_CIA', 'CNPJ_CIA', 'DT_FIM_EXERC'])['DIFERENCA'].idxmin()

# Filtrar os DataFrames principais usando os índices dos valores mínimos de 'DIFERENCA'
fluxo_caixa_tri_corrente = fluxo_caixa_tri_corrente.loc[indices_min_DT_INI_corrente]
fluxo_caixa_tri_proximo = fluxo_caixa_tri_proximo.loc[indices_min_DT_INI_proximo]

# Selecionar colunas desejadas e adicionar a coluna 'Ano' com o valor correspondente
fluxo_caixa_tri_corrente = fluxo_caixa_tri_corrente[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA', 'DT_REFER']]
fluxo_caixa_tri_proximo = fluxo_caixa_tri_proximo[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA', 'DT_REFER']]
fluxo_caixa_tri_corrente['Ano_tri'] = ano_corrente
fluxo_caixa_tri_proximo['Ano_tri'] = ano_proximo

# Concatenando os dados de ambos os anos
fluxo_caixa_tri = pd.concat([fluxo_caixa_tri_corrente, fluxo_caixa_tri_proximo], ignore_index=True)

# Convertendo a coluna 'DT_REFER' para o tipo datetime
fluxo_caixa_tri['DT_REFER'] = pd.to_datetime(fluxo_caixa_tri['DT_REFER'])

# Criando uma coluna de trimestre (Q1, Q2, etc.)
fluxo_caixa_tri['Quarter'] = fluxo_caixa_tri['DT_REFER'].dt.to_period('Q')

# Criando um novo DataFrame com as colunas corretas para cada trimestre
pivot = fluxo_caixa_tri.pivot_table(values='VL_CONTA', index=['DENOM_CIA', 'CNPJ_CIA'], columns='Quarter', aggfunc='first')

# Resetando o índice para transformar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
pivot.reset_index(inplace=True)

# Preenchendo valores NaN com 0, se necessário
pivot.fillna(0, inplace=True)

# Renomeando as colunas para incluir o trimestre e o ano (ex: Q1_2023, Q2_2023, Q1_2024, etc.)
pivot.columns = [f'Q{col.quarter}_{col.start_time.year}' if isinstance(col, pd.Period) else col for col in pivot.columns]

# Remover possíveis duplicatas de colunas geradas incorretamente
pivot = pivot.loc[:, ~pivot.columns.duplicated()]

# Ordenar colunas cronologicamente (trimestres)
colunas_cronologicas = ['DENOM_CIA', 'CNPJ_CIA'] + sorted([col for col in pivot.columns if col not in ['DENOM_CIA', 'CNPJ_CIA']], key=lambda x: (x[-4:], x))

# Aplicar a nova ordem de colunas
fluxo_caixa_trim = pivot[colunas_cronologicas]





C:\Users\User\AppData\Local\Temp\ipykernel_25252\2941151349.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fluxo_caixa_tri_corrente['DT_INI_EXERC'] = pd.to_datetime(fluxo_caixa_tri_corrente['DT_INI_EXERC'])
C:\Users\User\AppData\Local\Temp\ipykernel_25252\2941151349.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fluxo_caixa_tri_corrente['DT_FIM_EXERC'] = pd.to_datetime(fluxo_caixa_tri_corrente['DT_FIM_EXERC'])
C:\Users\User\AppData\Local\Temp\ipykernel_25252\2941151349.py:17: SettingWithCopyWarn

In [50]:
# Convertendo a coluna 'DT_REFER' para o tipo datetime
fluxo_caixa_tri['DT_REFER'] = pd.to_datetime(fluxo_caixa_tri['DT_REFER'])

# Criando uma coluna de trimestre (Q1, Q2, etc.)
fluxo_caixa_tri['Quarter'] = fluxo_caixa_tri['DT_REFER'].dt.to_period('Q')

# Criando um novo DataFrame com as colunas corretas para cada trimestre
pivot = fluxo_caixa_tri.pivot_table(values='VL_CONTA', index=['DENOM_CIA', 'CNPJ_CIA'], columns='Quarter', aggfunc='first')

# Resetando o índice para transformar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
pivot.reset_index(inplace=True)

# Preenchendo valores NaN com 0, se necessário
pivot.fillna(0, inplace=True)

# Renomeando as colunas para incluir o trimestre e o ano (ex: Q1_2023, Q2_2023, etc.)
pivot.columns = [f'Q{col.quarter}_{col.start_time.year}' if isinstance(col, pd.Period) else col for col in pivot.columns]

# Remover possíveis duplicatas de colunas geradas incorretamente
pivot = pivot.loc[:, ~pivot.columns.duplicated()]

# Verificar e calcular Q2 - Q1 se ambos estiverem presentes
if any('Q2' in col for col in pivot.columns) and any('Q1' in col for col in pivot.columns):
    coluna_q2 = [col for col in pivot.columns if 'Q2' in col][0]
    coluna_q1 = [col for col in pivot.columns if 'Q1' in col][0]
    pivot[coluna_q2] = pivot[coluna_q2] - pivot[coluna_q1]

# Verificar e calcular Q3 - Q2 - Q1 se todos estiverem presentes
if any('Q3' in col for col in pivot.columns) and any('Q2' in col for col in pivot.columns) and any('Q1' in col for col in pivot.columns):
    coluna_q3 = [col for col in pivot.columns if 'Q3' in col][0]
    coluna_q2 = [col for col in pivot.columns if 'Q2' in col][0]
    coluna_q1 = [col for col in pivot.columns if 'Q1' in col][0]
    pivot[coluna_q3] = pivot[coluna_q3] - pivot[coluna_q2] - pivot[coluna_q1]

# Verificar e calcular Q1 do ano seguinte - Q4 do ano corrente, se ambos estiverem presentes
if any('Q1' in col for col in pivot.columns) and any('Q4' in col for col in pivot.columns):
    coluna_q1_prox = [col for col in pivot.columns if 'Q1' in col and '2024' in col][0]  # Exemplo com o próximo ano (ajustar dinamicamente)
    coluna_q4_corrente = [col for col in pivot.columns if 'Q4' in col][0]
    pivot[coluna_q1_prox] = pivot[coluna_q1_prox] - pivot[coluna_q4_corrente]

# DataFrame final com o Fluxo de Caixa trimestral ajustado
fluxo_caixa_trim = pivot



In [51]:
# Definindo o caminho do arquivo CSV para o ano corrente e próximo
caminho_arquivo_tri_corrente = f'{diretorio_destino_arq_trim}/itr_cia_aberta_DFC_MI_con_{ano_corrente}.csv'
caminho_arquivo_tri_proximo = f'{diretorio_destino_arq_trim}/itr_cia_aberta_DFC_MI_con_{ano_proximo}.csv'

# Lendo o arquivo CSV para o ano corrente e próximo, ignorando as linhas com formatação incorreta
df_tri_corrente = pd.read_csv(caminho_arquivo_tri_corrente, encoding='cp1252', sep=';', on_bad_lines='skip')
df_tri_proximo = pd.read_csv(caminho_arquivo_tri_proximo, encoding='cp1252', sep=';', on_bad_lines='skip')

# Filtrando os dados para encontrar o CAPEX (CD_CONTA == '6.02')
capex_tri_corrente = df_tri_corrente[(df_tri_corrente['ORDEM_EXERC'] == 'ÚLTIMO') & (df_tri_corrente['CD_CONTA'] == '6.02')]
capex_tri_proximo = df_tri_proximo[(df_tri_proximo['ORDEM_EXERC'] == 'ÚLTIMO') & (df_tri_proximo['CD_CONTA'] == '6.02')]

# Selecionar colunas desejadas e adicionar a coluna 'Ano' com o valor correspondente
capex_tri_corrente = capex_tri_corrente[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA', 'DT_REFER']]
capex_tri_proximo = capex_tri_proximo[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA', 'DT_REFER']]
capex_tri_corrente['Ano_tri'] = ano_corrente
capex_tri_proximo['Ano_tri'] = ano_proximo

# Concatenando os dados de ambos os anos
capex_tri = pd.concat([capex_tri_corrente, capex_tri_proximo], ignore_index=True)

# Convertendo a coluna 'DT_REFER' para o tipo datetime
capex_tri['DT_REFER'] = pd.to_datetime(capex_tri['DT_REFER'])

# Criando uma coluna de trimestre (Q1, Q2, etc.)
capex_tri['Quarter'] = capex_tri['DT_REFER'].dt.to_period('Q')

# Criando um novo DataFrame com as colunas corretas para cada trimestre
pivot = capex_tri.pivot_table(values='VL_CONTA', index=['DENOM_CIA', 'CNPJ_CIA'], columns='Quarter', aggfunc='first')

# Resetando o índice para transformar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
pivot.reset_index(inplace=True)

# Preenchendo valores NaN com 0, se necessário
pivot.fillna(0, inplace=True)

# Renomeando as colunas para incluir o trimestre e o ano (ex: Q1_2023, Q2_2023, Q1_2024, etc.)
pivot.columns = [f'Q{col.quarter}_{col.start_time.year}' if isinstance(col, pd.Period) else col for col in pivot.columns]

# Remover possíveis duplicatas de colunas geradas incorretamente
pivot = pivot.loc[:, ~pivot.columns.duplicated()]

# Ordenando as colunas cronologicamente
colunas_ordenadas = ['DENOM_CIA', 'CNPJ_CIA'] + sorted([col for col in pivot.columns if col not in ['DENOM_CIA', 'CNPJ_CIA']], key=lambda x: (x[-4:], x))

# Aplicar a nova ordem de colunas ao DataFrame
capex_trim = pivot[colunas_ordenadas]

# Verificar e calcular Q2 - Q1 se Q2 estiver presente
if 'Q2' in capex_trim.columns and 'Q1' in capex_trim.columns:
    capex_trim['Q2'] = capex_trim['Q2'] - capex_trim['Q1']

# Verificar e calcular Q3 - Q2 - Q1 se Q3 estiver presente
if 'Q3' in capex_trim.columns and 'Q2' in capex_trim.columns and 'Q1' in capex_trim.columns:
    capex_trim['Q3'] = capex_trim['Q3'] - capex_trim['Q2'] - capex_trim['Q1']



In [52]:
# Convertendo a coluna 'DT_REFER' para o tipo datetime
capex_tri['DT_REFER'] = pd.to_datetime(capex_tri['DT_REFER'])

# Criando uma coluna de trimestre (Q1, Q2, etc.)
capex_tri['Quarter'] = capex_tri['DT_REFER'].dt.to_period('Q')

# Criando um novo DataFrame com as colunas corretas para cada trimestre
pivot = capex_tri.pivot_table(values='VL_CONTA', index=['DENOM_CIA', 'CNPJ_CIA'], columns='Quarter', aggfunc='first')

# Resetando o índice para transformar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
pivot.reset_index(inplace=True)

# Preenchendo valores NaN com 0, se necessário
pivot.fillna(0, inplace=True)

# Renomeando as colunas para incluir o trimestre e o ano (ex: Q1_2023, Q2_2023, etc.)
pivot.columns = [f'Q{col.quarter}_{col.start_time.year}' if isinstance(col, pd.Period) else col for col in pivot.columns]

# Definir os trimestres Q2 e Q3 que queremos calcular em relação aos anteriores
trimestres_para_calcular = ['Q2', 'Q3']

# Encontrar todos os trimestres presentes
trimestres_presentes = [col for col in pivot.columns if col.startswith('Q')]

# Verificar e calcular Q2 - Q1 se Q2 estiver presente
if any('Q2' in col for col in pivot.columns) and any('Q1' in col for col in pivot.columns):
    coluna_q2 = [col for col in pivot.columns if 'Q2' in col][0]
    coluna_q1 = [col for col in pivot.columns if 'Q1' in col][0]
    pivot[coluna_q2] = pivot[coluna_q2] - pivot[coluna_q1]

# Verificar e calcular Q3 - Q2 - Q1 se Q3 estiver presente
if any('Q3' in col for col in pivot.columns) and any('Q2' in col for col in pivot.columns) and any('Q1' in col for col in pivot.columns):
    coluna_q3 = [col for col in pivot.columns if 'Q3' in col][0]
    coluna_q2 = [col for col in pivot.columns if 'Q2' in col][0]
    coluna_q1 = [col for col in pivot.columns if 'Q1' in col][0]
    pivot[coluna_q3] = pivot[coluna_q3] - pivot[coluna_q2] - pivot[coluna_q1]

# Verificar e calcular Q1 do ano próximo - Q4 do ano corrente
if any('Q1' in col for col in pivot.columns) and any('Q4' in col for col in pivot.columns):
    coluna_q1_prox = [col for col in pivot.columns if col.startswith('Q1') and '2024' in col][0]  # Exemplo para o ano próximo
    coluna_q4_corrente = [col for col in pivot.columns if 'Q4' in col][0]
    pivot[coluna_q1_prox] = pivot[coluna_q1_prox] - pivot[coluna_q4_corrente]

# DataFrame final com o CAPEX trimestral ajustado
capex_trim = pivot


In [53]:
# Configurando as colunas 'DENOM_CIA' e 'CNPJ_CIA' como índices nos DataFrames
fluxo_caixa_trim.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)
capex_trim.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)

# Substituindo valores NaN por 0
fluxo_caixa_trim.fillna(0, inplace=True)
capex_trim.fillna(0, inplace=True)

# Subtraindo os valores de capex de fluxo de caixa operacional para obter free_cash_flow_tri
free_cash_flow_trim = fluxo_caixa_trim.add(capex_trim, fill_value=0)

# Resetando o índice para tornar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
free_cash_flow_trim.reset_index(inplace=True)

# Identificar automaticamente a coluna que corresponde a 'Q1' do próximo ano
coluna_q1_proximo_ano = [col for col in free_cash_flow_trim.columns if col.startswith('Q1') and any(char.isdigit() for char in col)]

# Reordenar as colunas, movendo a coluna 'Q1' do próximo ano para o final
colunas_ordenadas = [col for col in free_cash_flow_trim.columns if col not in coluna_q1_proximo_ano] + coluna_q1_proximo_ano

# Aplicar a nova ordem de colunas ao DataFrame
free_cash_flow_trim = free_cash_flow_trim[colunas_ordenadas]

# Verificar e calcular Q2 - Q1 para o ano corrente
if any(['Q2' in col for col in free_cash_flow_trim.columns]) and any(['Q1' in col for col in free_cash_flow_trim.columns]):
    coluna_q2 = [col for col in free_cash_flow_trim.columns if 'Q2' in col][0]
    coluna_q1 = [col for col in free_cash_flow_trim.columns if 'Q1' in col][0]
    free_cash_flow_trim[coluna_q2] = free_cash_flow_trim[coluna_q2] - free_cash_flow_trim[coluna_q1]

# Verificar e calcular Q3 - Q2 - Q1 para o ano corrente
if any(['Q3' in col for col in free_cash_flow_trim.columns]) and any(['Q2' in col for col in free_cash_flow_trim.columns]) and any(['Q1' in col for col in free_cash_flow_trim.columns]):
    coluna_q3 = [col for col in free_cash_flow_trim.columns if 'Q3' in col][0]
    coluna_q2 = [col for col in free_cash_flow_trim.columns if 'Q2' in col][0]
    coluna_q1 = [col for col in free_cash_flow_trim.columns if 'Q1' in col][0]
    free_cash_flow_trim[coluna_q3] = free_cash_flow_trim[coluna_q3] - free_cash_flow_trim[coluna_q2] - free_cash_flow_trim[coluna_q1]

# Verificar e calcular Q1 do próximo ano - Q4 do ano corrente
if any(['Q1' in col for col in free_cash_flow_trim.columns]) and any(['Q4' in col for col in free_cash_flow_trim.columns]):
    coluna_q1_prox = [col for col in free_cash_flow_trim.columns if col.startswith('Q1') and '2024' in col][0]  # Exemplo com o próximo ano
    coluna_q4_corrente = [col for col in free_cash_flow_trim.columns if 'Q4' in col][0]
    free_cash_flow_trim[coluna_q1_prox] = free_cash_flow_trim[coluna_q1_prox] - free_cash_flow_trim[coluna_q4_corrente]

# DataFrame final com o Free Cash Flow trimestral ajustado e colunas reordenadas
free_cash_flow_trim = free_cash_flow_trim[colunas_ordenadas]


In [54]:
# Reordenar as colunas do DataFrame em ordem cronológica
colunas_cronologicas = ['DENOM_CIA', 'CNPJ_CIA'] + sorted([col for col in free_cash_flow_trim.columns if col not in ['DENOM_CIA', 'CNPJ_CIA']], key=lambda x: (x[-4:], x))

# Aplicar a nova ordem de colunas ao DataFrame
free_cash_flow_trim = free_cash_flow_trim[colunas_cronologicas]


Fluxo de Caixa operacional

Histórico

In [55]:
# Definindo relatório a ser aberto
relatorio = "dfp_cia_aberta_DFC_MI_con"  # inserir apenas o nome do arquivo, sem o ano

# Lista de anos desejados
anos = list(range(ano_inicial, ano_final+1))

# DataFrame vazio para armazenar os resultados
resultados_cfc_mi = pd.DataFrame()

# Loop pelos anos
for ano in anos:
    # Definindo o caminho do arquivo usando formatação de string
    caminho_arquivo = f'{diretorio_destino_arq_csv}/{ano}/{relatorio}_{ano}.csv'

    # Lendo o arquivo CSV
    df = pd.read_csv(caminho_arquivo, encoding='utf-8', sep=',')

    # Filtrando os dados para encontrar o Caixa Líquido de Atividades Operacionais (CD_CONTA == '6.01')
    var_CFC_MI = df[(df['ORDEM_EXERC'] == 'ÚLTIMO') & (df['CD_CONTA'] == '6.01')]

    # Selecionando colunas desejadas
    var_CFC_MI = var_CFC_MI[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA']]

    # Adicionando a coluna 'Ano' com o valor correspondente
    var_CFC_MI['Ano'] = ano

    # Adicionando os resultados ao DataFrame final
    resultados_cfc_mi = pd.concat([resultados_cfc_mi, var_CFC_MI], ignore_index=True)

# Agregando os valores duplicados
resultados_agregados_cfc_mi = resultados_cfc_mi.groupby(['DENOM_CIA', 'CNPJ_CIA', 'Ano'])['VL_CONTA'].sum().reset_index()

# Criando um DataFrame pivotado
fluxo_cx_operacional = resultados_agregados_cfc_mi.pivot_table(index=['DENOM_CIA', 'CNPJ_CIA'], columns='Ano', values='VL_CONTA', aggfunc='sum').reset_index()

# Renomeando as colunas para remover o nome da coluna do índice
fluxo_cx_operacional.name = None


Trimestrais

In [56]:
# Definindo os caminhos do arquivo CSV para o ano corrente e ano próximo
caminho_arquivo_tri_corrente = f'{diretorio_destino_arq_trim}/itr_cia_aberta_DFC_MI_con_{ano_corrente}.csv'
caminho_arquivo_tri_proximo = f'{diretorio_destino_arq_trim}/itr_cia_aberta_DFC_MI_con_{ano_proximo}.csv'

# Lendo os arquivos CSV para o ano corrente e próximo, ignorando as linhas com formatação incorreta
df_tri_corrente = pd.read_csv(caminho_arquivo_tri_corrente, encoding='cp1252', sep=';', on_bad_lines='skip')
df_tri_proximo = pd.read_csv(caminho_arquivo_tri_proximo, encoding='cp1252', sep=';', on_bad_lines='skip')

# Filtrando os dados para encontrar o Fluxo de Caixa Operacional (CD_CONTA == '6.01')
fluxo_cx_operacional_tri_corrente = df_tri_corrente[(df_tri_corrente['ORDEM_EXERC'] == 'ÚLTIMO') & (df_tri_corrente['CD_CONTA'] == '6.01')]
fluxo_cx_operacional_tri_proximo = df_tri_proximo[(df_tri_proximo['ORDEM_EXERC'] == 'ÚLTIMO') & (df_tri_proximo['CD_CONTA'] == '6.01')]

# Converter a coluna 'DT_REFER' para o tipo datetime para ambos os anos
fluxo_cx_operacional_tri_corrente['DT_REFER'] = pd.to_datetime(fluxo_cx_operacional_tri_corrente['DT_REFER'])
fluxo_cx_operacional_tri_proximo['DT_REFER'] = pd.to_datetime(fluxo_cx_operacional_tri_proximo['DT_REFER'])

# Selecionar colunas desejadas e adicionar a coluna 'Ano' com o valor correspondente
fluxo_cx_operacional_tri_corrente = fluxo_cx_operacional_tri_corrente[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA', 'DT_REFER']]
fluxo_cx_operacional_tri_proximo = fluxo_cx_operacional_tri_proximo[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA', 'DT_REFER']]
fluxo_cx_operacional_tri_corrente['Ano_tri'] = ano_corrente
fluxo_cx_operacional_tri_proximo['Ano_tri'] = ano_proximo

# Concatenando os dados de ambos os anos
fluxo_cx_operacional_tri = pd.concat([fluxo_cx_operacional_tri_corrente, fluxo_cx_operacional_tri_proximo], ignore_index=True)

# Criando uma coluna de trimestre (Q1, Q2, etc.)
fluxo_cx_operacional_tri['Quarter'] = fluxo_cx_operacional_tri['DT_REFER'].dt.to_period('Q')

# Criando um novo DataFrame com as colunas corretas para cada trimestre
pivot = fluxo_cx_operacional_tri.pivot_table(values='VL_CONTA', index=['DENOM_CIA', 'CNPJ_CIA'], columns='Quarter', aggfunc='first')

# Resetando o índice para transformar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
pivot.reset_index(inplace=True)

# Preenchendo valores NaN com 0, se necessário
pivot.fillna(0, inplace=True)

# Renomeando as colunas para incluir o trimestre e o ano (ex: Q1_2023, Q2_2023, etc.)
pivot.columns = [f'Q{col.quarter}_{col.start_time.year}' if isinstance(col, pd.Period) else col for col in pivot.columns]

# Remover possíveis duplicatas de colunas geradas incorretamente
pivot = pivot.loc[:, ~pivot.columns.duplicated()]

# DataFrame final com o Fluxo de Caixa Operacional trimestral ajustado
fluxo_cx_operacional_trim = pivot



C:\Users\User\AppData\Local\Temp\ipykernel_25252\3152282557.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fluxo_cx_operacional_tri_corrente['DT_REFER'] = pd.to_datetime(fluxo_cx_operacional_tri_corrente['DT_REFER'])
C:\Users\User\AppData\Local\Temp\ipykernel_25252\3152282557.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fluxo_cx_operacional_tri_proximo['DT_REFER'] = pd.to_datetime(fluxo_cx_operacional_tri_proximo['DT_REFER'])


In [57]:
# Convertendo a coluna 'DT_REFER' para o tipo datetime
fluxo_cx_operacional_tri['DT_REFER'] = pd.to_datetime(fluxo_cx_operacional_tri['DT_REFER'])

# Criando uma coluna de trimestre (Q1, Q2, etc.)
fluxo_cx_operacional_tri['Quarter'] = fluxo_cx_operacional_tri['DT_REFER'].dt.to_period('Q')

# Criando um novo DataFrame com as colunas corretas para cada trimestre
pivot = fluxo_cx_operacional_tri.pivot_table(values='VL_CONTA', index=['DENOM_CIA', 'CNPJ_CIA'], columns='Quarter', aggfunc='first')

# Resetando o índice para transformar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
pivot.reset_index(inplace=True)

# Preenchendo valores NaN com 0, se necessário
pivot.fillna(0, inplace=True)

# Renomeando as colunas para incluir o trimestre e o ano (ex: Q1_2023, Q2_2023, etc.)
pivot.columns = [f'Q{col.quarter}_{col.start_time.year}' if isinstance(col, pd.Period) else col for col in pivot.columns]

# Remover possíveis duplicatas de colunas geradas incorretamente
pivot = pivot.loc[:, ~pivot.columns.duplicated()]

# Definir os trimestres que queremos calcular em relação aos anteriores
trimestres_para_calcular = ['Q2', 'Q3']

# Encontrar todos os trimestres presentes
trimestres_presentes = [col for col in pivot.columns if col.startswith('Q')]

# Verificar e calcular Q2 - Q1 se Q2 e Q1 estiverem presentes
if any('Q2' in col for col in pivot.columns) and any('Q1' in col for col in pivot.columns):
    coluna_q2 = [col for col in pivot.columns if 'Q2' in col][0]
    coluna_q1 = [col for col in pivot.columns if 'Q1' in col][0]
    pivot[coluna_q2] = pivot[coluna_q2] - pivot[coluna_q1]

# Verificar e calcular Q3 - Q2 - Q1 se Q3, Q2 e Q1 estiverem presentes
if any('Q3' in col for col in pivot.columns) and any('Q2' in col for col in pivot.columns) and any('Q1' in col for col in pivot.columns):
    coluna_q3 = [col for col in pivot.columns if 'Q3' in col][0]
    coluna_q2 = [col for col in pivot.columns if 'Q2' in col][0]
    coluna_q1 = [col for col in pivot.columns if 'Q1' in col][0]
    pivot[coluna_q3] = pivot[coluna_q3] - pivot[coluna_q2] - pivot[coluna_q1]

# DataFrame final com o Fluxo de Caixa Operacional trimestral ajustado para ambos os anos
fluxo_cx_operacional_trim = pivot


Fluxo de caixa investimentos

Histórico

In [58]:
# Definindo relatório a ser aberto
relatorio = "dfp_cia_aberta_DFC_MI_con"  # inserir apenas o nome do arquivo, sem o ano

# Lista de anos desejados
anos = list(range(ano_inicial, ano_final+1))

# DataFrame vazio para armazenar os resultados
resultados_cfc_mi = pd.DataFrame()

# Loop pelos anos
for ano in anos:
    # Definindo o caminho do arquivo usando formatação de string
    caminho_arquivo = f'{diretorio_destino_arq_csv}/{ano}/{relatorio}_{ano}.csv'

    # Lendo o arquivo CSV
    df = pd.read_csv(caminho_arquivo, encoding='utf-8', sep=',')

    # Filtrando os dados para encontrar o Lucro Líquido do Período
    var_CFC_MI = df[(df['ORDEM_EXERC'] == 'ÚLTIMO') & (df['CD_CONTA'] == '6.02')]

    # Selecionando colunas desejadas
    var_CFC_MI = var_CFC_MI[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA']]

    # Adicionando a coluna 'Ano' com o valor correspondente
    var_CFC_MI['Ano'] = ano

    # Adicionando os resultados ao DataFrame final
    resultados_cfc_mi = pd.concat([resultados_cfc_mi, var_CFC_MI], ignore_index=True)

# Agregando os valores duplicados
resultados_agregados_cfc_mi= resultados_cfc_mi.groupby(['DENOM_CIA', 'CNPJ_CIA', 'Ano'])['VL_CONTA'].sum().reset_index()

# Criando um DataFrame pivotado
fluxo_cx_investimentos = resultados_agregados_cfc_mi.pivot_table(index=['DENOM_CIA', 'CNPJ_CIA'], columns='Ano', values='VL_CONTA', aggfunc='sum').reset_index()

# Renomeando as colunas para remover o nome da coluna do índice
fluxo_cx_investimentos.name = None


Trimestrais

In [59]:
# Definindo os caminhos do arquivo CSV para o ano corrente e ano próximo
caminho_arquivo_tri_corrente = f'{diretorio_destino_arq_trim}/itr_cia_aberta_DFC_MI_con_{ano_corrente}.csv'
caminho_arquivo_tri_proximo = f'{diretorio_destino_arq_trim}/itr_cia_aberta_DFC_MI_con_{ano_proximo}.csv'

# Lendo os arquivos CSV para o ano corrente e próximo, ignorando as linhas com formatação incorreta
df_tri_corrente = pd.read_csv(caminho_arquivo_tri_corrente, encoding='cp1252', sep=';', on_bad_lines='skip')
df_tri_proximo = pd.read_csv(caminho_arquivo_tri_proximo, encoding='cp1252', sep=';', on_bad_lines='skip')

# Filtrando os dados para encontrar o Fluxo de Caixa de Investimentos (CD_CONTA == '6.02')
fluxo_cx_investimentos_tri_corrente = df_tri_corrente[(df_tri_corrente['ORDEM_EXERC'] == 'ÚLTIMO') & (df_tri_corrente['CD_CONTA'] == '6.02')]
fluxo_cx_investimentos_tri_proximo = df_tri_proximo[(df_tri_proximo['ORDEM_EXERC'] == 'ÚLTIMO') & (df_tri_proximo['CD_CONTA'] == '6.02')]

# Converter a coluna 'DT_REFER' para o tipo datetime para ambos os anos
fluxo_cx_investimentos_tri_corrente['DT_REFER'] = pd.to_datetime(fluxo_cx_investimentos_tri_corrente['DT_REFER'])
fluxo_cx_investimentos_tri_proximo['DT_REFER'] = pd.to_datetime(fluxo_cx_investimentos_tri_proximo['DT_REFER'])

# Selecionar colunas desejadas e adicionar a coluna 'Ano' com o valor correspondente
fluxo_cx_investimentos_tri_corrente = fluxo_cx_investimentos_tri_corrente[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA', 'DT_REFER']]
fluxo_cx_investimentos_tri_proximo = fluxo_cx_investimentos_tri_proximo[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA', 'DT_REFER']]
fluxo_cx_investimentos_tri_corrente['Ano_tri'] = ano_corrente
fluxo_cx_investimentos_tri_proximo['Ano_tri'] = ano_proximo

# Concatenando os dados de ambos os anos
fluxo_cx_investimentos_tri = pd.concat([fluxo_cx_investimentos_tri_corrente, fluxo_cx_investimentos_tri_proximo], ignore_index=True)

# Criando uma coluna de trimestre (Q1, Q2, etc.)
fluxo_cx_investimentos_tri['Quarter'] = fluxo_cx_investimentos_tri['DT_REFER'].dt.to_period('Q')

# Criando um novo DataFrame com as colunas corretas para cada trimestre
pivot = fluxo_cx_investimentos_tri.pivot_table(values='VL_CONTA', index=['DENOM_CIA', 'CNPJ_CIA'], columns='Quarter', aggfunc='first')

# Resetando o índice para transformar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
pivot.reset_index(inplace=True)

# Preenchendo valores NaN com 0, se necessário
pivot.fillna(0, inplace=True)

# Renomeando as colunas para incluir o trimestre e o ano (ex: Q1_2023, Q2_2023, etc.)
pivot.columns = [f'Q{col.quarter}_{col.start_time.year}' if isinstance(col, pd.Period) else col for col in pivot.columns]

# Remover possíveis duplicatas de colunas geradas incorretamente
pivot = pivot.loc[:, ~pivot.columns.duplicated()]

# DataFrame final com o Fluxo de Caixa de Investimentos trimestral ajustado
fluxo_cx_investimentos_trim = pivot


C:\Users\User\AppData\Local\Temp\ipykernel_25252\788798927.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fluxo_cx_investimentos_tri_corrente['DT_REFER'] = pd.to_datetime(fluxo_cx_investimentos_tri_corrente['DT_REFER'])
C:\Users\User\AppData\Local\Temp\ipykernel_25252\788798927.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fluxo_cx_investimentos_tri_proximo['DT_REFER'] = pd.to_datetime(fluxo_cx_investimentos_tri_proximo['DT_REFER'])


In [60]:
# Convertendo a coluna 'DT_REFER' para o tipo datetime
fluxo_cx_investimentos_tri['DT_REFER'] = pd.to_datetime(fluxo_cx_investimentos_tri['DT_REFER'])

# Criando uma coluna de trimestre (Q1, Q2, etc.)
fluxo_cx_investimentos_tri['Quarter'] = fluxo_cx_investimentos_tri['DT_REFER'].dt.to_period('Q')

# Criando um novo DataFrame com as colunas corretas para cada trimestre
pivot = fluxo_cx_investimentos_tri.pivot_table(values='VL_CONTA', index=['DENOM_CIA', 'CNPJ_CIA'], columns='Quarter', aggfunc='first')

# Resetando o índice para transformar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
pivot.reset_index(inplace=True)

# Preenchendo valores NaN com 0, se necessário
pivot.fillna(0, inplace=True)

# Renomeando as colunas para incluir o trimestre e o ano (ex: Q1_2023, Q2_2023, etc.)
pivot.columns = [f'Q{col.quarter}_{col.start_time.year}' if isinstance(col, pd.Period) else col for col in pivot.columns]

# Remover possíveis duplicatas de colunas geradas incorretamente
pivot = pivot.loc[:, ~pivot.columns.duplicated()]

# Definir os trimestres que queremos calcular em relação aos anteriores
trimestres_para_calcular = ['Q2', 'Q3']

# Encontrar todos os trimestres presentes
trimestres_presentes = [col for col in pivot.columns if col.startswith('Q')]

# Verificar e calcular Q2 - Q1 se Q2 e Q1 estiverem presentes
if any('Q2' in col for col in pivot.columns) and any('Q1' in col for col in pivot.columns):
    coluna_q2 = [col for col in pivot.columns if 'Q2' in col][0]
    coluna_q1 = [col for col in pivot.columns if 'Q1' in col][0]
    pivot[coluna_q2] = pivot[coluna_q2] - pivot[coluna_q1]

# Verificar e calcular Q3 - Q2 - Q1 se Q3, Q2 e Q1 estiverem presentes
if any('Q3' in col for col in pivot.columns) and any('Q2' in col for col in pivot.columns) and any('Q1' in col for col in pivot.columns):
    coluna_q3 = [col for col in pivot.columns if 'Q3' in col][0]
    coluna_q2 = [col for col in pivot.columns if 'Q2' in col][0]
    coluna_q1 = [col for col in pivot.columns if 'Q1' in col][0]
    pivot[coluna_q3] = pivot[coluna_q3] - pivot[coluna_q2] - pivot[coluna_q1]

# DataFrame final com o Fluxo de Caixa de Investimentos trimestral ajustado para ambos os anos
fluxo_cx_investimentos_trim = pivot



Caixa Líquido atividades de finaciamento

Histórico

In [61]:
# Definindo relatório a ser aberto
relatorio = "dfp_cia_aberta_DFC_MI_con"  # inserir apenas o nome do arquivo, sem o ano

# Lista de anos desejados
anos = list(range(ano_inicial, ano_final+1))

# DataFrame vazio para armazenar os resultados
resultados_cfc_mi = pd.DataFrame()

# Loop pelos anos
for ano in anos:
    # Definindo o caminho do arquivo usando formatação de string
    caminho_arquivo = f'{diretorio_destino_arq_csv}/{ano}/{relatorio}_{ano}.csv'

    # Lendo o arquivo CSV
    df = pd.read_csv(caminho_arquivo, encoding='utf-8', sep=',')

    # Filtrando os dados para encontrar o Lucro Líquido do Período
    var_CFC_MI = df[(df['ORDEM_EXERC'] == 'ÚLTIMO') & (df['CD_CONTA'] == '6.03')]

    # Selecionando colunas desejadas
    var_CFC_MI = var_CFC_MI[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA']]

    # Adicionando a coluna 'Ano' com o valor correspondente
    var_CFC_MI['Ano'] = ano

    # Adicionando os resultados ao DataFrame final
    resultados_cfc_mi = pd.concat([resultados_cfc_mi, var_CFC_MI], ignore_index=True)

# Agregando os valores duplicados
resultados_agregados_cfc_mi= resultados_cfc_mi.groupby(['DENOM_CIA', 'CNPJ_CIA', 'Ano'])['VL_CONTA'].sum().reset_index()

# Criando um DataFrame pivotado
cx_liq_ativ_financ = resultados_agregados_cfc_mi.pivot_table(index=['DENOM_CIA', 'CNPJ_CIA'], columns='Ano', values='VL_CONTA', aggfunc='sum').reset_index()

# Renomeando as colunas para remover o nome da coluna do índice
cx_liq_ativ_financ.name = None



Trimestral

In [62]:
# Definindo os caminhos do arquivo CSV para o ano corrente e ano próximo
caminho_arquivo_tri_corrente = f'{diretorio_destino_arq_trim}/itr_cia_aberta_DFC_MI_con_{ano_corrente}.csv'
caminho_arquivo_tri_proximo = f'{diretorio_destino_arq_trim}/itr_cia_aberta_DFC_MI_con_{ano_proximo}.csv'

# Lendo os arquivos CSV para o ano corrente e próximo, ignorando as linhas com formatação incorreta
df_tri_corrente = pd.read_csv(caminho_arquivo_tri_corrente, encoding='cp1252', sep=';', on_bad_lines='skip')
df_tri_proximo = pd.read_csv(caminho_arquivo_tri_proximo, encoding='cp1252', sep=';', on_bad_lines='skip')

# Filtrando os dados para encontrar o Caixa Líquido de Atividades de Financiamento (CD_CONTA == '6.03')
cx_liq_ativ_financ_tri_corrente = df_tri_corrente[(df_tri_corrente['ORDEM_EXERC'] == 'ÚLTIMO') & (df_tri_corrente['CD_CONTA'] == '6.03')]
cx_liq_ativ_financ_tri_proximo = df_tri_proximo[(df_tri_proximo['ORDEM_EXERC'] == 'ÚLTIMO') & (df_tri_proximo['CD_CONTA'] == '6.03')]

# Converter a coluna 'DT_REFER' para o tipo datetime para ambos os anos
cx_liq_ativ_financ_tri_corrente['DT_REFER'] = pd.to_datetime(cx_liq_ativ_financ_tri_corrente['DT_REFER'])
cx_liq_ativ_financ_tri_proximo['DT_REFER'] = pd.to_datetime(cx_liq_ativ_financ_tri_proximo['DT_REFER'])

# Selecionar colunas desejadas e adicionar a coluna 'Ano' com o valor correspondente
cx_liq_ativ_financ_tri_corrente = cx_liq_ativ_financ_tri_corrente[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA', 'DT_REFER']]
cx_liq_ativ_financ_tri_proximo = cx_liq_ativ_financ_tri_proximo[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA', 'DT_REFER']]
cx_liq_ativ_financ_tri_corrente['Ano_tri'] = ano_corrente
cx_liq_ativ_financ_tri_proximo['Ano_tri'] = ano_proximo

# Concatenando os dados de ambos os anos
cx_liq_ativ_financ_tri = pd.concat([cx_liq_ativ_financ_tri_corrente, cx_liq_ativ_financ_tri_proximo], ignore_index=True)

# Convertendo a coluna 'DT_REFER' para o tipo datetime
cx_liq_ativ_financ_tri['DT_REFER'] = pd.to_datetime(cx_liq_ativ_financ_tri['DT_REFER'])

# Criando uma coluna de trimestre (Q1, Q2, etc.)
cx_liq_ativ_financ_tri['Quarter'] = cx_liq_ativ_financ_tri['DT_REFER'].dt.to_period('Q')

# Criando um novo DataFrame com as colunas corretas para cada trimestre
pivot = cx_liq_ativ_financ_tri.pivot_table(values='VL_CONTA', index=['DENOM_CIA', 'CNPJ_CIA'], columns='Quarter', aggfunc='first')

# Resetando o índice para transformar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
pivot.reset_index(inplace=True)

# Preenchendo valores NaN com 0, se necessário
pivot.fillna(0, inplace=True)

# Renomeando as colunas para incluir o trimestre e o ano (ex: Q1_2023, Q2_2023, etc.)
pivot.columns = [f'Q{col.quarter}_{col.start_time.year}' if isinstance(col, pd.Period) else col for col in pivot.columns]

# Remover possíveis duplicatas de colunas geradas incorretamente
pivot = pivot.loc[:, ~pivot.columns.duplicated()]

# DataFrame final com o Caixa Líquido de Atividades de Financiamento trimestral ajustado
cx_liq_ativ_financ_trim = pivot


C:\Users\User\AppData\Local\Temp\ipykernel_25252\2498051559.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cx_liq_ativ_financ_tri_corrente['DT_REFER'] = pd.to_datetime(cx_liq_ativ_financ_tri_corrente['DT_REFER'])
C:\Users\User\AppData\Local\Temp\ipykernel_25252\2498051559.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cx_liq_ativ_financ_tri_proximo['DT_REFER'] = pd.to_datetime(cx_liq_ativ_financ_tri_proximo['DT_REFER'])


In [63]:
# Convertendo a coluna 'DT_REFER' para o tipo datetime
cx_liq_ativ_financ_tri['DT_REFER'] = pd.to_datetime(cx_liq_ativ_financ_tri['DT_REFER'])

# Criando uma coluna de trimestre (Q1, Q2, etc.)
cx_liq_ativ_financ_tri['Quarter'] = cx_liq_ativ_financ_tri['DT_REFER'].dt.to_period('Q')

# Criando um novo DataFrame com as colunas corretas para cada trimestre
pivot = cx_liq_ativ_financ_tri.pivot_table(values='VL_CONTA', index=['DENOM_CIA', 'CNPJ_CIA'], columns='Quarter', aggfunc='first')

# Resetando o índice para transformar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
pivot.reset_index(inplace=True)

# Preenchendo valores NaN com 0, se necessário
pivot.fillna(0, inplace=True)

# Renomeando as colunas para incluir o trimestre e o ano (ex: Q1_2023, Q2_2023, etc.)
pivot.columns = [f'Q{col.quarter}_{col.start_time.year}' if isinstance(col, pd.Period) else col for col in pivot.columns]

# Remover possíveis duplicatas de colunas geradas incorretamente
pivot = pivot.loc[:, ~pivot.columns.duplicated()]

# Definir os trimestres que queremos calcular em relação aos anteriores
trimestres_para_calcular = ['Q2', 'Q3']

# Encontrar todos os trimestres presentes
trimestres_presentes = [col for col in pivot.columns if col.startswith('Q')]

# Verificar e calcular Q2 - Q1 se Q2 e Q1 estiverem presentes
if any('Q2' in col for col in pivot.columns) and any('Q1' in col for col in pivot.columns):
    coluna_q2 = [col for col in pivot.columns if 'Q2' in col][0]
    coluna_q1 = [col for col in pivot.columns if 'Q1' in col][0]
    pivot[coluna_q2] = pivot[coluna_q2] - pivot[coluna_q1]

# Verificar e calcular Q3 - Q2 - Q1 se Q3, Q2 e Q1 estiverem presentes
if any('Q3' in col for col in pivot.columns) and any('Q2' in col for col in pivot.columns) and any('Q1' in col for col in pivot.columns):
    coluna_q3 = [col for col in pivot.columns if 'Q3' in col][0]
    coluna_q2 = [col for col in pivot.columns if 'Q2' in col][0]
    coluna_q1 = [col for col in pivot.columns if 'Q1' in col][0]
    pivot[coluna_q3] = pivot[coluna_q3] - pivot[coluna_q2] - pivot[coluna_q1]

# DataFrame final com o Caixa Líquido de Atividades de Financiamento trimestral ajustado para ambos os anos
cx_liq_ativ_financ_trim = pivot


In [64]:
# Diretório base onde os arquivos CSV serão salvos
diretorio_base = diretorio_rel_ind_financeiros

# Resetar o índice para garantir que 'DENOM_CIA' e 'CNPJ_CIA' estejam como colunas
receita_liquida.reset_index(inplace=True)
receita_liquida_trim.reset_index(inplace=True)
ativo_total.reset_index(inplace=True)
ativo_total_trim.reset_index(inplace=True)
patrimonio_liquido.reset_index(inplace=True)
patrimonio_liquido_trim.reset_index(inplace=True)
lucro_liquido.reset_index(inplace=True)
lucro_liquido_trim.reset_index(inplace=True)
divida_liquida.reset_index(inplace=True)
divida_liquida_trim.reset_index(inplace=True)
EBITDA.reset_index(inplace=True)
EBITDA_trim.reset_index(inplace=True)
free_cash_flow.reset_index(inplace=True)
free_cash_flow_trim.reset_index(inplace=True)
cx_liq_ativ_financ.reset_index(inplace=True)
cx_liq_ativ_financ_trim.reset_index(inplace=True)
LAIR.reset_index(inplace=True)
LAIR_trim.reset_index(inplace=True)
fluxo_cx_investimentos.reset_index(inplace=True)
fluxo_cx_investimentos_trim.reset_index(inplace=True)
fluxo_cx_operacional.reset_index(inplace=True)
fluxo_cx_operacional_trim.reset_index(inplace=True)
margem_ebitda.reset_index(inplace=True)
margem_ebitda_trim.reset_index(inplace=True)

# Salvando os arquivos CSV
receita_liquida.to_csv(os.path.join(diretorio_base, 'receita_liquida.csv'), sep=';', index=False)
receita_liquida_trim.to_csv(os.path.join(diretorio_base, 'receita_liquida_trim.csv'), sep=';', index=False)
ativo_total.to_csv(os.path.join(diretorio_base, 'ativo_total.csv'), sep=';', index=False)
ativo_total_trim.to_csv(os.path.join(diretorio_base, 'ativo_total_trim.csv'), sep=';', index=False)
patrimonio_liquido.to_csv(os.path.join(diretorio_base, 'patrimonio_liquido.csv'), sep=';', index=False)
patrimonio_liquido_trim.to_csv(os.path.join(diretorio_base, 'patrimonio_liquido_trim.csv'), sep=';', index=False)
lucro_liquido.to_csv(os.path.join(diretorio_base, 'lucro_liquido.csv'), sep=';', index=False)
lucro_liquido_trim.to_csv(os.path.join(diretorio_base, 'lucro_liquido_trim.csv'), sep=';', index=False)
divida_liquida.to_csv(os.path.join(diretorio_base, 'divida_liquida.csv'), sep=';', index=False)
divida_liquida_trim.to_csv(os.path.join(diretorio_base, 'divida_liquida_trim.csv'), sep=';', index=False)
EBITDA.to_csv(os.path.join(diretorio_base, 'EBITDA.csv'), sep=';', index=False)
EBITDA_trim.to_csv(os.path.join(diretorio_base, 'EBITDA_trim.csv'), sep=';', index=False)
free_cash_flow.to_csv(os.path.join(diretorio_base, 'free_cash_flow.csv'), sep=';', index=False)
free_cash_flow_trim.to_csv(os.path.join(diretorio_base, 'free_cash_flow_trim.csv'), sep=';', index=False)
cx_liq_ativ_financ.to_csv(os.path.join(diretorio_base, 'cx_liq_ativ_financ.csv'), sep=';', index=False)
cx_liq_ativ_financ_trim.to_csv(os.path.join(diretorio_base, 'cx_liq_ativ_financ_trim.csv'), sep=';', index=False)
LAIR.to_csv(os.path.join(diretorio_base, 'LAIR.csv'), sep=';', index=False)
LAIR_trim.to_csv(os.path.join(diretorio_base, 'LAIR_trim.csv'), sep=';', index=False)
fluxo_cx_investimentos.to_csv(os.path.join(diretorio_base, 'fluxo_cx_investimentos.csv'), sep=';', index=False)
fluxo_cx_investimentos_trim.to_csv(os.path.join(diretorio_base, 'fluxo_cx_investimentos_trim.csv'), sep=';', index=False)
fluxo_cx_operacional.to_csv(os.path.join(diretorio_base, 'fluxo_cx_operacional.csv'), sep=';', index=False)
fluxo_cx_operacional_trim.to_csv(os.path.join(diretorio_base, 'fluxo_cx_operacional_trim.csv'), sep=';', index=False)
margem_ebitda.to_csv(os.path.join(diretorio_base, 'margem_ebitda.csv'), sep=';', index=False)
margem_ebitda_trim.to_csv(os.path.join(diretorio_base, 'margem_ebitda_trim.csv'), sep=';', index=False)

Importando os dfs para cálculo dos indicadores que seguem

In [65]:
# Diretório base onde os arquivos CSV estão armazenados
diretorio_base = diretorio_rel_ind_financeiros

# Abrindo os arquivos CSV
free_cash_flow = pd.read_csv(os.path.join(diretorio_base, 'free_cash_flow.csv'), sep=';')
free_cash_flow_trim = pd.read_csv(os.path.join(diretorio_base, 'free_cash_flow_trim.csv'), sep=';')
receita_liquida = pd.read_csv(os.path.join(diretorio_base, 'receita_liquida.csv'), sep=';')
receita_liquida_trim = pd.read_csv(os.path.join(diretorio_base, 'receita_liquida_trim.csv'), sep=';')
lucro_liquido = pd.read_csv(os.path.join(diretorio_base, 'lucro_liquido.csv'), sep=';')
lucro_liquido_trim = pd.read_csv(os.path.join(diretorio_base, 'lucro_liquido_trim.csv'), sep=';')
divida_liquida = pd.read_csv(os.path.join(diretorio_base, 'divida_liquida.csv'), sep=';')
divida_liquida_trim = pd.read_csv(os.path.join(diretorio_base, 'divida_liquida_trim.csv'), sep=';')
EBITDA = pd.read_csv(os.path.join(diretorio_base, 'EBITDA.csv'), sep=';')
EBITDA_trim = pd.read_csv(os.path.join(diretorio_base, 'EBITDA_trim.csv'), sep=';')
patrimonio_liquido = pd.read_csv(os.path.join(diretorio_base, 'patrimonio_liquido.csv'), sep=';')
patrimonio_liquido_trim = pd.read_csv(os.path.join(diretorio_base, 'patrimonio_liquido_trim.csv'), sep=';')
ativo_total = pd.read_csv(os.path.join(diretorio_base, 'ativo_total.csv'), sep=';')
ativo_total_trim = pd.read_csv(os.path.join(diretorio_base, 'ativo_total_trim.csv'), sep=';')
cx_liq_ativ_financ = pd.read_csv(os.path.join(diretorio_base, 'cx_liq_ativ_financ.csv'), sep=';')
cx_liq_ativ_financ_trim = pd.read_csv(os.path.join(diretorio_base, 'cx_liq_ativ_financ_trim.csv'), sep=';')
LAIR = pd.read_csv(os.path.join(diretorio_base, 'LAIR.csv'), sep=';')
LAIR_trim = pd.read_csv(os.path.join(diretorio_base, 'LAIR_trim.csv'), sep=';')

FCF_RL = free cash flow / receita líquida

Histórico

In [66]:
# Evitando definir as colunas 'DENOM_CIA' e 'CNPJ_CIA' como índices permanentes
# Em vez disso, fazemos a operação sem alterar permanentemente o índice

# Substituindo valores NaN por 0
free_cash_flow.fillna(0, inplace=True)
receita_liquida.fillna(0, inplace=True)

# Calculando FCF/RL (Free Cash Flow/Receita Líquida) temporariamente usando o índice apenas na operação
fcf_rl = free_cash_flow.set_index(['DENOM_CIA', 'CNPJ_CIA']).div(receita_liquida.set_index(['DENOM_CIA', 'CNPJ_CIA']), fill_value=0)

# Resetando o índice para trazer 'DENOM_CIA' e 'CNPJ_CIA' de volta como colunas
fcf_rl.reset_index(inplace=True)



Trimestral

In [67]:
# Configurando o index como 'DENOM_CIA' e 'CNPJ_CIA' para ambos os DataFrames
free_cash_flow_trim.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)
receita_liquida_trim.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)

# Convertendo as colunas numéricas para garantir o tipo float
free_cash_flow_trim = free_cash_flow_trim.apply(pd.to_numeric, errors='coerce')
receita_liquida_trim = receita_liquida_trim.apply(pd.to_numeric, errors='coerce')

# Substituindo NaN por 0 antes de realizar a divisão
free_cash_flow_trim.fillna(0, inplace=True)
receita_liquida_trim.fillna(0, inplace=True)

# Evitando divisão por 0 ao substituir 0 por um valor muito pequeno
receita_liquida_trim.replace(0, 1e-10, inplace=True)

# Realizando a divisão para calcular FCF/RL
fcf_rl_trim = free_cash_flow_trim.div(receita_liquida_trim)

# Substituindo valores inf e NaN resultantes da divisão por 0
fcf_rl_trim.replace([np.inf, -np.inf], 0, inplace=True)
fcf_rl_trim.fillna(0, inplace=True)

# Resetando o índice para tornar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
fcf_rl_trim.reset_index(inplace=True)

In [68]:
fcf_rl_trim

,DENOM_CIA,CNPJ_CIA,Q1_2023,Q1_2024,Q2_2023,Q2_2024,Q3_2023,Q3_2024,Q4_2023,index
0,2W ECOBANK S.A.,08.773.135/0001-00,-3.214821e-01,0.000000e+00,-0.600860,0.000000e+00,0.923946,0.0,0.0,0.0
1,3R PETROLEUM ÓLEO E GÁS S.A.,12.091.809/0001-55,-1.573058e-01,-1.931290e-01,-9.554903,-6.798309e-02,3.464609,0.0,0.0,0.0
2,AEGEA SANEAMENTO E PARTICIPAÇÕES S.A.,08.827.501/0001-58,-2.938518e-01,3.189393e-01,-0.161766,-1.560219e-01,0.076774,0.0,0.0,0.0
3,AERIS IND. E COM. DE EQUIP. PARA GER. DE ENG. ...,12.528.708/0001-07,-2.585634e-01,-2.719320e-01,0.353885,2.709245e-01,-0.302652,0.0,0.0,0.0
4,AES BRASIL ENERGIA S.A.,37.663.076/0001-07,-1.175499e+00,-2.101761e-01,1.958608,-4.447051e-01,-0.462331,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
482,YBYRÁ CAPITAL S.A.,02.217.319/0001-07,-2.129812e+16,4.140000e+12,-20921.361366,-2.343550e+15,895.561357,0.0,0.0,0.0
483,YDUQS PARTICIPACOES S.A.,08.807.432/0001-10,5.890222e-02,1.217992e-01,0.146709,1.335042e-01,-0.023138,0.0,0.0,0.0
484,YOU INC INCORPORADORA E PARTICIPAÇÃO S.A,11.284.204/0001-18,-3.884728e-01,-1.469331e-01,0.058161,-4.378640e-02,0.584767,0.0,0.0,0.0
485,YUNY INCORPORADORA HOLDING S.A.,09.267.996/0001-70,-1.778130e-01,-6.458109e-02,-0.172026,-6.951392e-01,0.166308,0.0,0.0,0.0


Lucro Líquido / Receita Líquida

Histórico

In [69]:
# Configurando o index como 'DENOM_CIA' e 'CNPJ_CIA' para ambos os DataFrames
lucro_liquido.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)
receita_liquida.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)

# Convertendo as colunas numéricas para garantir o tipo float
lucro_liquido = lucro_liquido.apply(pd.to_numeric, errors='coerce')
receita_liquida = receita_liquida.apply(pd.to_numeric, errors='coerce')

# Substituindo NaN por 0 antes de realizar a divisão
lucro_liquido.fillna(0, inplace=True)
receita_liquida.fillna(0, inplace=True)

# Evitando divisão por 0 ao substituir 0 por um valor muito pequeno
receita_liquida.replace(0, 1e-10, inplace=True)

# Realizando a divisão para calcular Lucro Líquido sobre Receita Líquida (LL/RL)
ll_sobre_rl = lucro_liquido.div(receita_liquida)

# Substituindo valores inf e NaN resultantes da divisão por 0
ll_sobre_rl.replace([np.inf, -np.inf], 0, inplace=True)
ll_sobre_rl.fillna(0, inplace=True)

# Resetando o índice para tornar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
ll_sobre_rl.reset_index(inplace=True)

trimestral

In [70]:
# Verificar se 'DENOM_CIA' e 'CNPJ_CIA' estão presentes nos DataFrames
if 'DENOM_CIA' not in lucro_liquido_trim.columns or 'CNPJ_CIA' not in lucro_liquido_trim.columns:
    print("Colunas 'DENOM_CIA' e 'CNPJ_CIA' ausentes em lucro_liquido_trim")

if 'DENOM_CIA' not in receita_liquida_trim.columns or 'CNPJ_CIA' not in receita_liquida_trim.columns:
    print("Colunas 'DENOM_CIA' e 'CNPJ_CIA' ausentes em receita_liquida_trim")

# Configurar os índices corretos antes de realizar operações
if 'DENOM_CIA' in lucro_liquido_trim.columns and 'CNPJ_CIA' in lucro_liquido_trim.columns:
    lucro_liquido_trim.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)

if 'DENOM_CIA' in receita_liquida_trim.columns and 'CNPJ_CIA' in receita_liquida_trim.columns:
    receita_liquida_trim.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)

# Garantir que os valores nas colunas sejam numéricos
lucro_liquido_trim = lucro_liquido_trim.apply(pd.to_numeric, errors='coerce')
receita_liquida_trim = receita_liquida_trim.apply(pd.to_numeric, errors='coerce')

# Substituir NaN por 0
lucro_liquido_trim.fillna(0, inplace=True)
receita_liquida_trim.fillna(0, inplace=True)

# Evitar divisão por 0 substituindo valores 0 por um valor muito pequeno
receita_liquida_trim.replace(0, 1e-10, inplace=True)

# Realizar a divisão para calcular Lucro Líquido sobre Receita Líquida (LL/RL) trimestral
ll_sobre_rl_trim = lucro_liquido_trim.div(receita_liquida_trim)

# Substituir valores inf e NaN resultantes da divisão por 0
ll_sobre_rl_trim.replace([np.inf, -np.inf], 0, inplace=True)
ll_sobre_rl_trim.fillna(0, inplace=True)

# Resetar o índice para exibir as colunas corretamente
ll_sobre_rl_trim.reset_index(inplace=True)

# Exibir o DataFrame resultante
print(ll_sobre_rl_trim)

Colunas 'DENOM_CIA' e 'CNPJ_CIA' ausentes em receita_liquida_trim
                                             DENOM_CIA            CNPJ_CIA  \
0                                      2W ECOBANK S.A.  08.773.135/0001-00   
1                         3R PETROLEUM ÓLEO E GÁS S.A.  12.091.809/0001-55   
2                AEGEA SANEAMENTO E PARTICIPAÇÕES S.A.  08.827.501/0001-58   
3    AERIS IND. E COM. DE EQUIP. PARA GER. DE ENG. ...  12.528.708/0001-07   
4                              AES BRASIL ENERGIA S.A.  37.663.076/0001-07   
..                                                 ...                 ...   
482                                 YBYRÁ CAPITAL S.A.  02.217.319/0001-07   
483                           YDUQS PARTICIPACOES S.A.  08.807.432/0001-10   
484           YOU INC INCORPORADORA E PARTICIPAÇÃO S.A  11.284.204/0001-18   
485                    YUNY INCORPORADORA HOLDING S.A.  09.267.996/0001-70   
486                                            ZAMP SA  13.574.594/0001-96  

Dívida Líquida / EBITDA

Histórico

In [71]:
# Verificar se 'DENOM_CIA' e 'CNPJ_CIA' estão presentes
if 'DENOM_CIA' not in divida_liquida.columns or 'CNPJ_CIA' not in divida_liquida.columns:
    print("Colunas 'DENOM_CIA' e 'CNPJ_CIA' ausentes em divida_liquida")

if 'DENOM_CIA' not in EBITDA.columns or 'CNPJ_CIA' not in EBITDA.columns:
    print("Colunas 'DENOM_CIA' e 'CNPJ_CIA' ausentes em EBITDA")

# Certificar-se de que os DataFrames têm os índices corretos antes de realizar operações
if 'DENOM_CIA' in divida_liquida.columns and 'CNPJ_CIA' in divida_liquida.columns:
    divida_liquida.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)

if 'DENOM_CIA' in EBITDA.columns and 'CNPJ_CIA' in EBITDA.columns:
    EBITDA.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)

# Garantir que os valores nas colunas sejam numéricos
divida_liquida = divida_liquida.apply(pd.to_numeric, errors='coerce')
EBITDA = EBITDA.apply(pd.to_numeric, errors='coerce')

# Substituir NaN por 0
divida_liquida.fillna(0, inplace=True)
EBITDA.fillna(0, inplace=True)

# Evitar divisão por 0 substituindo valores 0 por um valor muito pequeno em EBITDA
EBITDA.replace(0, 1e-10, inplace=True)

# Calculando Dívida Líquida/EBITDA para os dados históricos
divida_sobre_ebitda = divida_liquida.div(EBITDA)

# Substituir valores inf e NaN resultantes da divisão por 0
divida_sobre_ebitda.replace([np.inf, -np.inf], 0, inplace=True)
divida_sobre_ebitda.fillna(0, inplace=True)

# Resetando o índice para tornar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
divida_sobre_ebitda.reset_index(inplace=True)

# Exibindo o DataFrame resultante
print(divida_sobre_ebitda)


                                             DENOM_CIA            CNPJ_CIA  \
0                                      2W ECOBANK S.A.  08.773.135/0001-00   
1                         3R PETROLEUM ÓLEO E GÁS S.A.  12.091.809/0001-55   
2     ADVANCED DIGITAL HEALTH MEDICINA PREVENTIVA S.A.  10.345.009/0001-98   
3                AEGEA SANEAMENTO E PARTICIPAÇÕES S.A.  08.827.501/0001-58   
4    AERIS IND. E COM. DE EQUIP. PARA GER. DE ENG. ...  12.528.708/0001-07   
..                                                 ...                 ...   
651           YOU INC INCORPORADORA E PARTICIPAÇÃO S.A  11.284.204/0001-18   
652                    YUNY INCORPORADORA HOLDING S.A.  09.267.996/0001-70   
653                                            ZAMP SA  13.574.594/0001-96   
654                                           ÉVORA SA  91.820.068/0001-72   
655                      ÓLEO E GÁS PARTICIPAÇÕES S.A.  07.957.093/0001-96   

         2012      2013      2014      2015       2016       20

Trimestral

In [72]:
# Verificar se 'DENOM_CIA' e 'CNPJ_CIA' estão presentes nos DataFrames
if 'DENOM_CIA' not in divida_liquida_trim.columns or 'CNPJ_CIA' not in divida_liquida_trim.columns:
    print("Colunas 'DENOM_CIA' e 'CNPJ_CIA' ausentes em divida_liquida_trim")

if 'DENOM_CIA' not in EBITDA_trim.columns or 'CNPJ_CIA' not in EBITDA_trim.columns:
    print("Colunas 'DENOM_CIA' e 'CNPJ_CIA' ausentes em EBITDA_trim")

# Configurar os índices corretos antes de realizar operações
if 'DENOM_CIA' in divida_liquida_trim.columns and 'CNPJ_CIA' in divida_liquida_trim.columns:
    divida_liquida_trim.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)

if 'DENOM_CIA' in EBITDA_trim.columns and 'CNPJ_CIA' in EBITDA_trim.columns:
    EBITDA_trim.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)

# Garantir que os valores nas colunas sejam numéricos
divida_liquida_trim = divida_liquida_trim.apply(pd.to_numeric, errors='coerce')
EBITDA_trim = EBITDA_trim.apply(pd.to_numeric, errors='coerce')

# Substituir NaN por 0
divida_liquida_trim.fillna(0, inplace=True)
EBITDA_trim.fillna(0, inplace=True)


# Realizar a divisão para calcular Dívida Líquida sobre EBITDA trimestral
divida_sobre_ebitda_trim = divida_liquida_trim.div(EBITDA_trim)

# Substituir valores inf e NaN resultantes da divisão por 0
divida_sobre_ebitda_trim.replace([np.inf, -np.inf], 0, inplace=True)
divida_sobre_ebitda_trim.fillna(0, inplace=True)

# Resetar o índice para exibir as colunas corretamente
divida_sobre_ebitda_trim.reset_index(inplace=True)

# Exibir o DataFrame resultante
print(divida_sobre_ebitda_trim)


                                             DENOM_CIA            CNPJ_CIA  \
0                                      2W ECOBANK S.A.  08.773.135/0001-00   
1                         3R PETROLEUM ÓLEO E GÁS S.A.  12.091.809/0001-55   
2                AEGEA SANEAMENTO E PARTICIPAÇÕES S.A.  08.827.501/0001-58   
3    AERIS IND. E COM. DE EQUIP. PARA GER. DE ENG. ...  12.528.708/0001-07   
4                              AES BRASIL ENERGIA S.A.  37.663.076/0001-07   
..                                                 ...                 ...   
482                                 YBYRÁ CAPITAL S.A.  02.217.319/0001-07   
483                           YDUQS PARTICIPACOES S.A.  08.807.432/0001-10   
484           YOU INC INCORPORADORA E PARTICIPAÇÃO S.A  11.284.204/0001-18   
485                    YUNY INCORPORADORA HOLDING S.A.  09.267.996/0001-70   
486                                            ZAMP SA  13.574.594/0001-96   

        Q1_2023     Q1_2024    Q2_2023    Q2_2024     Q3_2023  

ROE = Lucro Líquido / Patrimônio Líquido

Histórico

In [73]:
# Verificar se 'DENOM_CIA' e 'CNPJ_CIA' estão presentes nos DataFrames
if 'DENOM_CIA' not in lucro_liquido.columns or 'CNPJ_CIA' not in lucro_liquido.columns:
    print("Colunas 'DENOM_CIA' e 'CNPJ_CIA' ausentes em lucro_liquido")

if 'DENOM_CIA' not in patrimonio_liquido.columns or 'CNPJ_CIA' not in patrimonio_liquido.columns:
    print("Colunas 'DENOM_CIA' e 'CNPJ_CIA' ausentes em patrimonio_liquido")

# Configurar os índices corretos antes de realizar operações
if 'DENOM_CIA' in lucro_liquido.columns and 'CNPJ_CIA' in lucro_liquido.columns:
    lucro_liquido.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)

if 'DENOM_CIA' in patrimonio_liquido.columns and 'CNPJ_CIA' in patrimonio_liquido.columns:
    patrimonio_liquido.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)

# Garantir que os valores nas colunas sejam numéricos
lucro_liquido = lucro_liquido.apply(pd.to_numeric, errors='coerce')
patrimonio_liquido = patrimonio_liquido.apply(pd.to_numeric, errors='coerce')

# Substituir NaN por 0
lucro_liquido.fillna(0, inplace=True)
patrimonio_liquido.fillna(0, inplace=True)

# Realizar a divisão para calcular ROE (Lucro Líquido sobre Patrimônio Líquido)
roe = lucro_liquido.div(patrimonio_liquido)

# Substituir valores inf e NaN resultantes da divisão por 0
roe.replace([np.inf, -np.inf], 0, inplace=True)
roe.fillna(0, inplace=True)

# Resetar o índice para exibir as colunas corretamente
roe.reset_index(inplace=True)

# Exibir o DataFrame resultante
print(roe.head())


Colunas 'DENOM_CIA' e 'CNPJ_CIA' ausentes em lucro_liquido
                                           DENOM_CIA            CNPJ_CIA  \
0                                    2W ECOBANK S.A.  08.773.135/0001-00   
1                       3R PETROLEUM ÓLEO E GÁS S.A.  12.091.809/0001-55   
2   ADVANCED DIGITAL HEALTH MEDICINA PREVENTIVA S.A.  10.345.009/0001-98   
3              AEGEA SANEAMENTO E PARTICIPAÇÕES S.A.  08.827.501/0001-58   
4  AERIS IND. E COM. DE EQUIP. PARA GER. DE ENG. ...  12.528.708/0001-07   

   index      2012      2013      2014      2015      2016      2017  \
0    0.0  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
1    1.0  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2    1.0  1.343577  0.545958  0.544689  0.415854  0.429348  0.580081   
3    1.0  0.000000  0.088299  0.120271  0.127698  0.101201  0.121976   
4    1.0  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   

       2018      2019      2020      2021      2022

Trimestral

In [74]:
# Verificar se 'DENOM_CIA' e 'CNPJ_CIA' estão presentes nos DataFrames trimestrais
if 'DENOM_CIA' not in lucro_liquido_trim.columns or 'CNPJ_CIA' not in lucro_liquido_trim.columns:
    print("Colunas 'DENOM_CIA' e 'CNPJ_CIA' ausentes em lucro_liquido_trim")

if 'DENOM_CIA' not in patrimonio_liquido_trim.columns or 'CNPJ_CIA' not in patrimonio_liquido_trim.columns:
    print("Colunas 'DENOM_CIA' e 'CNPJ_CIA' ausentes em patrimonio_liquido_trim")

# Configurar os índices corretos antes de realizar operações
if 'DENOM_CIA' in lucro_liquido_trim.columns and 'CNPJ_CIA' in lucro_liquido_trim.columns:
    lucro_liquido_trim.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)

if 'DENOM_CIA' in patrimonio_liquido_trim.columns and 'CNPJ_CIA' in patrimonio_liquido_trim.columns:
    patrimonio_liquido_trim.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)

# Garantir que os valores nas colunas sejam numéricos
lucro_liquido_trim = lucro_liquido_trim.apply(pd.to_numeric, errors='coerce')
patrimonio_liquido_trim = patrimonio_liquido_trim.apply(pd.to_numeric, errors='coerce')

# Substituir NaN por 0
lucro_liquido_trim.fillna(0, inplace=True)
patrimonio_liquido_trim.fillna(0, inplace=True)

# Realizar a divisão para calcular ROE (Lucro Líquido sobre Patrimônio Líquido) trimestral
roe_trim = lucro_liquido_trim.div(patrimonio_liquido_trim)

# Substituir valores inf e NaN resultantes da divisão por 0
roe_trim.replace([np.inf, -np.inf], 0, inplace=True)
roe_trim.fillna(0, inplace=True)

# Resetar o índice para exibir as colunas corretamente
roe_trim.reset_index(inplace=True)

# Exibir o DataFrame resultante
print(roe_trim.head())


Colunas 'DENOM_CIA' e 'CNPJ_CIA' ausentes em lucro_liquido_trim
                                           DENOM_CIA            CNPJ_CIA  \
0                                    2W ECOBANK S.A.  08.773.135/0001-00   
1                       3R PETROLEUM ÓLEO E GÁS S.A.  12.091.809/0001-55   
2              AEGEA SANEAMENTO E PARTICIPAÇÕES S.A.  08.827.501/0001-58   
3  AERIS IND. E COM. DE EQUIP. PARA GER. DE ENG. ...  12.528.708/0001-07   
4                            AES BRASIL ENERGIA S.A.  37.663.076/0001-07   

   index   Q1_2023   Q2_2023   Q3_2023  Q4_2023   Q1_2024   Q2_2024  Q3_2024  
0    0.0  0.277375  0.132831 -0.339646      0.0  0.000000  0.000000      0.0  
1    1.0  0.002956  0.014662 -0.016101      0.0 -0.044635 -0.073754      0.0  
2    1.0  0.024885  0.010762  0.028540      0.0  0.011237  0.009579      0.0  
3    1.0 -0.025743 -0.022648 -0.035204      0.0 -0.037593 -0.002824      0.0  
4    1.0  0.003166  0.001851  0.011932      0.0 -0.024191 -0.032602      0.0  


ROA = Lucro Líquido / ativo total

Histórico

In [75]:
# Verificar se 'DENOM_CIA' e 'CNPJ_CIA' estão presentes nos DataFrames históricos
if 'DENOM_CIA' not in lucro_liquido.columns or 'CNPJ_CIA' not in lucro_liquido.columns:
    print("Colunas 'DENOM_CIA' e 'CNPJ_CIA' ausentes em lucro_liquido")

if 'DENOM_CIA' not in ativo_total.columns or 'CNPJ_CIA' not in ativo_total.columns:
    print("Colunas 'DENOM_CIA' e 'CNPJ_CIA' ausentes em ativo_total")

# Configurar os índices corretos antes de realizar operações
if 'DENOM_CIA' in lucro_liquido.columns and 'CNPJ_CIA' in lucro_liquido.columns:
    lucro_liquido.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)

if 'DENOM_CIA' in ativo_total.columns and 'CNPJ_CIA' in ativo_total.columns:
    ativo_total.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)

# Garantir que os valores nas colunas sejam numéricos
lucro_liquido = lucro_liquido.apply(pd.to_numeric, errors='coerce')
ativo_total = ativo_total.apply(pd.to_numeric, errors='coerce')

# Substituir NaN por 0
lucro_liquido.fillna(0, inplace=True)
ativo_total.fillna(0, inplace=True)

# Realizar a divisão para calcular ROA (Lucro Líquido sobre Ativo Total)
roa = lucro_liquido.div(ativo_total)

# Substituir valores inf e NaN resultantes da divisão por 0
roa.replace([np.inf, -np.inf], 0, inplace=True)
roa.fillna(0, inplace=True)

# Resetar o índice para exibir as colunas corretamente
roa.reset_index(inplace=True)

# Exibir o DataFrame resultante
print(roa.head())


Colunas 'DENOM_CIA' e 'CNPJ_CIA' ausentes em lucro_liquido
                                           DENOM_CIA            CNPJ_CIA  \
0                                    2W ECOBANK S.A.  08.773.135/0001-00   
1                       3R PETROLEUM ÓLEO E GÁS S.A.  12.091.809/0001-55   
2   ADVANCED DIGITAL HEALTH MEDICINA PREVENTIVA S.A.  10.345.009/0001-98   
3              AEGEA SANEAMENTO E PARTICIPAÇÕES S.A.  08.827.501/0001-58   
4  AERIS IND. E COM. DE EQUIP. PARA GER. DE ENG. ...  12.528.708/0001-07   

   index      2012      2013      2014      2015      2016      2017  \
0    0.0  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
1    1.0  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2    1.0 -0.596057 -1.599824 -6.137127 -2.215088 -0.660727 -2.246160   
3    1.0  0.000000  0.035791  0.041610  0.040348  0.031360  0.026448   
4    1.0  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   

        2018      2019      2020      2021      202

Trimestral

In [76]:
# Verificar se 'DENOM_CIA' e 'CNPJ_CIA' estão presentes nos DataFrames trimestrais
if 'DENOM_CIA' not in lucro_liquido_trim.columns or 'CNPJ_CIA' not in lucro_liquido_trim.columns:
    print("Colunas 'DENOM_CIA' e 'CNPJ_CIA' ausentes em lucro_liquido_trim")

if 'DENOM_CIA' not in ativo_total_trim.columns or 'CNPJ_CIA' not in ativo_total_trim.columns:
    print("Colunas 'DENOM_CIA' e 'CNPJ_CIA' ausentes em ativo_total_trim")

# Configurar os índices corretos antes de realizar operações
if 'DENOM_CIA' in lucro_liquido_trim.columns and 'CNPJ_CIA' in lucro_liquido_trim.columns:
    lucro_liquido_trim.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)

if 'DENOM_CIA' in ativo_total_trim.columns and 'CNPJ_CIA' in ativo_total_trim.columns:
    ativo_total_trim.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)

# Garantir que os valores nas colunas sejam numéricos
lucro_liquido_trim = lucro_liquido_trim.apply(pd.to_numeric, errors='coerce')
ativo_total_trim = ativo_total_trim.apply(pd.to_numeric, errors='coerce')

# Substituir NaN por 0
lucro_liquido_trim.fillna(0, inplace=True)
ativo_total_trim.fillna(0, inplace=True)

# Realizar a divisão para calcular ROA (Lucro Líquido sobre Ativo Total) trimestral
roa_trim = lucro_liquido_trim.div(ativo_total_trim)

# Substituir valores inf e NaN resultantes da divisão por 0
roa_trim.replace([np.inf, -np.inf], 0, inplace=True)
roa_trim.fillna(0, inplace=True)

# Resetar o índice para exibir as colunas corretamente
roa_trim.reset_index(inplace=True)

# Exibir o DataFrame resultante
print(roa_trim.head())


Colunas 'DENOM_CIA' e 'CNPJ_CIA' ausentes em lucro_liquido_trim
                                           DENOM_CIA            CNPJ_CIA  \
0                                    2W ECOBANK S.A.  08.773.135/0001-00   
1                       3R PETROLEUM ÓLEO E GÁS S.A.  12.091.809/0001-55   
2              AEGEA SANEAMENTO E PARTICIPAÇÕES S.A.  08.827.501/0001-58   
3  AERIS IND. E COM. DE EQUIP. PARA GER. DE ENG. ...  12.528.708/0001-07   
4                            AES BRASIL ENERGIA S.A.  37.663.076/0001-07   

   index   Q1_2023   Q2_2023   Q3_2023  Q4_2023   Q1_2024   Q2_2024  Q3_2024  
0    0.0  0.017087  0.008224 -0.014231      0.0  0.000000  0.000000      0.0  
1    1.0  0.001590  0.004123 -0.004331      0.0 -0.011646 -0.016772      0.0  
2    1.0  0.007822  0.003130  0.005385      0.0  0.001822  0.001419      0.0  
3    1.0 -0.006752 -0.006002 -0.007508      0.0 -0.011875 -0.000846      0.0  
4    1.0  0.000696  0.000409  0.002690      0.0 -0.005439 -0.006846      0.0  


debt_to_equity = dívida líquida / patrimonio líquido

Histórico

In [77]:
# Verificar se 'DENOM_CIA' e 'CNPJ_CIA' estão presentes nos DataFrames históricos
if 'DENOM_CIA' not in divida_liquida.columns or 'CNPJ_CIA' not in divida_liquida.columns:
    print("Colunas 'DENOM_CIA' e 'CNPJ_CIA' ausentes em divida_liquida")

if 'DENOM_CIA' not in patrimonio_liquido.columns or 'CNPJ_CIA' not in patrimonio_liquido.columns:
    print("Colunas 'DENOM_CIA' e 'CNPJ_CIA' ausentes em patrimonio_liquido")

# Configurar os índices corretos antes de realizar operações
if 'DENOM_CIA' in divida_liquida.columns and 'CNPJ_CIA' in divida_liquida.columns:
    divida_liquida.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)

if 'DENOM_CIA' in patrimonio_liquido.columns and 'CNPJ_CIA' in patrimonio_liquido.columns:
    patrimonio_liquido.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)

# Garantir que os valores nas colunas sejam numéricos
divida_liquida = divida_liquida.apply(pd.to_numeric, errors='coerce')
patrimonio_liquido = patrimonio_liquido.apply(pd.to_numeric, errors='coerce')

# Substituir NaN por 0
divida_liquida.fillna(0, inplace=True)
patrimonio_liquido.fillna(0, inplace=True)

# Realizar a divisão para calcular Debt-to-Equity (Dívida Líquida/Patrimônio Líquido)
debt_to_equity = divida_liquida.div(patrimonio_liquido)

# Substituir valores inf e NaN resultantes da divisão por 0
debt_to_equity.replace([np.inf, -np.inf], 0, inplace=True)
debt_to_equity.fillna(0, inplace=True)

# Resetar o índice para tornar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
debt_to_equity.reset_index(inplace=True)

# Exibir o DataFrame resultante
print(debt_to_equity.head())


Colunas 'DENOM_CIA' e 'CNPJ_CIA' ausentes em divida_liquida
Colunas 'DENOM_CIA' e 'CNPJ_CIA' ausentes em patrimonio_liquido
                                           DENOM_CIA            CNPJ_CIA  \
0                                    2W ECOBANK S.A.  08.773.135/0001-00   
1                       3R PETROLEUM ÓLEO E GÁS S.A.  12.091.809/0001-55   
2   ADVANCED DIGITAL HEALTH MEDICINA PREVENTIVA S.A.  10.345.009/0001-98   
3              AEGEA SANEAMENTO E PARTICIPAÇÕES S.A.  08.827.501/0001-58   
4  AERIS IND. E COM. DE EQUIP. PARA GER. DE ENG. ...  12.528.708/0001-07   

   index      2012      2013      2014      2015      2016      2017  \
0    0.0  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
1    1.0  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2    1.0  1.650767  0.188282  0.003319  0.001489  0.004962  0.016831   
3    1.0  0.000000 -0.040987  1.437060  1.712659  1.691780  2.979174   
4    1.0  0.000000  0.000000  0.000000  0.000000  0.000000 

Trimestral

In [78]:
# Verificar se 'DENOM_CIA' e 'CNPJ_CIA' estão presentes nos DataFrames trimestrais
if 'DENOM_CIA' not in divida_liquida_trim.columns or 'CNPJ_CIA' not in divida_liquida_trim.columns:
    print("Colunas 'DENOM_CIA' e 'CNPJ_CIA' ausentes em divida_liquida_trim")

if 'DENOM_CIA' not in patrimonio_liquido_trim.columns or 'CNPJ_CIA' not in patrimonio_liquido_trim.columns:
    print("Colunas 'DENOM_CIA' e 'CNPJ_CIA' ausentes em patrimonio_liquido_trim")

# Configurar os índices corretos antes de realizar operações
if 'DENOM_CIA' in divida_liquida_trim.columns and 'CNPJ_CIA' in divida_liquida_trim.columns:
    divida_liquida_trim.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)

if 'DENOM_CIA' in patrimonio_liquido_trim.columns and 'CNPJ_CIA' in patrimonio_liquido_trim.columns:
    patrimonio_liquido_trim.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)

# Garantir que os valores nas colunas sejam numéricos
divida_liquida_trim = divida_liquida_trim.apply(pd.to_numeric, errors='coerce')
patrimonio_liquido_trim = patrimonio_liquido_trim.apply(pd.to_numeric, errors='coerce')

# Substituir NaN por 0
divida_liquida_trim.fillna(0, inplace=True)
patrimonio_liquido_trim.fillna(0, inplace=True)

# Realizar a divisão para calcular Debt-to-Equity (Dívida Líquida/Patrimônio Líquido)
debt_to_equity_trim = divida_liquida_trim.div(patrimonio_liquido_trim)

# Substituir valores inf e NaN resultantes da divisão por 0
debt_to_equity_trim.replace([np.inf, -np.inf], 0, inplace=True)
debt_to_equity_trim.fillna(0, inplace=True)

# Resetar o índice para tornar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
debt_to_equity_trim.reset_index(inplace=True)

# Exibir o DataFrame resultante
print(debt_to_equity_trim.head())


Colunas 'DENOM_CIA' e 'CNPJ_CIA' ausentes em divida_liquida_trim
Colunas 'DENOM_CIA' e 'CNPJ_CIA' ausentes em patrimonio_liquido_trim
                                           DENOM_CIA            CNPJ_CIA  \
0                                    2W ECOBANK S.A.  08.773.135/0001-00   
1                       3R PETROLEUM ÓLEO E GÁS S.A.  12.091.809/0001-55   
2              AEGEA SANEAMENTO E PARTICIPAÇÕES S.A.  08.827.501/0001-58   
3  AERIS IND. E COM. DE EQUIP. PARA GER. DE ENG. ...  12.528.708/0001-07   
4                            AES BRASIL ENERGIA S.A.  37.663.076/0001-07   

   index   Q1_2023   Q2_2023    Q3_2023  Q4_2023   Q1_2024   Q2_2024  Q3_2024  
0    0.0  8.860465  9.610295  15.208722      0.0  0.000000  0.000000      0.0  
1    1.0  0.089159  1.393316   1.499288      0.0  1.619802  1.937523      0.0  
2    1.0  1.809541  1.961071   2.371937      0.0  3.066947  3.613803      0.0  
3    1.0  1.141671  1.113142   1.255745      0.0  0.706263  0.517413      0.0  
4    1.0 

debt_to_total_assets =  dívida líquida / ativo total

Histórico

In [79]:
# Verificar se 'DENOM_CIA' e 'CNPJ_CIA' estão presentes nos DataFrames históricos
if 'DENOM_CIA' not in divida_liquida.columns or 'CNPJ_CIA' not in divida_liquida.columns:
    print("Colunas 'DENOM_CIA' e 'CNPJ_CIA' ausentes em divida_liquida")

if 'DENOM_CIA' not in ativo_total.columns or 'CNPJ_CIA' not in ativo_total.columns:
    print("Colunas 'DENOM_CIA' e 'CNPJ_CIA' ausentes em ativo_total")

# Configurar os índices corretos antes de realizar operações
if 'DENOM_CIA' in divida_liquida.columns and 'CNPJ_CIA' in divida_liquida.columns:
    divida_liquida.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)

if 'DENOM_CIA' in ativo_total.columns and 'CNPJ_CIA' in ativo_total.columns:
    ativo_total.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)

# Garantir que os valores nas colunas sejam numéricos
divida_liquida = divida_liquida.apply(pd.to_numeric, errors='coerce')
ativo_total = ativo_total.apply(pd.to_numeric, errors='coerce')

# Substituir NaN por 0
divida_liquida.fillna(0, inplace=True)
ativo_total.fillna(0, inplace=True)

# Realizar a divisão para calcular Debt-to-Total Assets (Dívida Líquida/Ativo Total)
debt_to_total_assets = divida_liquida.div(ativo_total)

# Substituir valores inf e NaN resultantes da divisão por 0
debt_to_total_assets.replace([np.inf, -np.inf], 0, inplace=True)
debt_to_total_assets.fillna(0, inplace=True)

# Resetar o índice para tornar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
debt_to_total_assets.reset_index(inplace=True)

# Exibir o DataFrame resultante
print(debt_to_total_assets.head())



Colunas 'DENOM_CIA' e 'CNPJ_CIA' ausentes em divida_liquida
Colunas 'DENOM_CIA' e 'CNPJ_CIA' ausentes em ativo_total
                                           DENOM_CIA            CNPJ_CIA  \
0                                    2W ECOBANK S.A.  08.773.135/0001-00   
1                       3R PETROLEUM ÓLEO E GÁS S.A.  12.091.809/0001-55   
2   ADVANCED DIGITAL HEALTH MEDICINA PREVENTIVA S.A.  10.345.009/0001-98   
3              AEGEA SANEAMENTO E PARTICIPAÇÕES S.A.  08.827.501/0001-58   
4  AERIS IND. E COM. DE EQUIP. PARA GER. DE ENG. ...  12.528.708/0001-07   

   index      2012      2013      2014      2015      2016      2017  \
0    0.0  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
1    1.0  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2    1.0 -0.732338 -0.551722 -0.037398 -0.007933 -0.007636 -0.065172   
3    1.0  0.000000 -0.016614  0.497177  0.541135  0.524241  0.645973   
4    1.0  0.000000  0.000000  0.000000  0.000000  0.000000  0.0000

Trimestral

In [80]:
# Verificar se 'DENOM_CIA' e 'CNPJ_CIA' estão presentes nos DataFrames trimestrais
if 'DENOM_CIA' not in divida_liquida_trim.columns or 'CNPJ_CIA' not in divida_liquida_trim.columns:
    print("Colunas 'DENOM_CIA' e 'CNPJ_CIA' ausentes em divida_liquida_trim")

if 'DENOM_CIA' not in ativo_total_trim.columns or 'CNPJ_CIA' not in ativo_total_trim.columns:
    print("Colunas 'DENOM_CIA' e 'CNPJ_CIA' ausentes em ativo_total_trim")

# Configurar os índices corretos antes de realizar operações
if 'DENOM_CIA' in divida_liquida_trim.columns and 'CNPJ_CIA' in divida_liquida_trim.columns:
    divida_liquida_trim.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)

if 'DENOM_CIA' in ativo_total_trim.columns and 'CNPJ_CIA' in ativo_total_trim.columns:
    ativo_total_trim.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)

# Garantir que os valores nas colunas sejam numéricos
divida_liquida_trim = divida_liquida_trim.apply(pd.to_numeric, errors='coerce')
ativo_total_trim = ativo_total_trim.apply(pd.to_numeric, errors='coerce')

# Substituir NaN por 0
divida_liquida_trim.fillna(0, inplace=True)
ativo_total_trim.fillna(0, inplace=True)

# Realizar a divisão para calcular Debt-to-Total Assets (Dívida Líquida/Ativo Total)
debt_to_total_assets_trim = divida_liquida_trim.div(ativo_total_trim)

# Substituir valores inf e NaN resultantes da divisão por 0
debt_to_total_assets_trim.replace([np.inf, -np.inf], 0, inplace=True)
debt_to_total_assets_trim.fillna(0, inplace=True)

# Resetar o índice para tornar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
debt_to_total_assets_trim.reset_index(inplace=True)

# Exibir o DataFrame resultante
print(debt_to_total_assets_trim.head())


Colunas 'DENOM_CIA' e 'CNPJ_CIA' ausentes em divida_liquida_trim
Colunas 'DENOM_CIA' e 'CNPJ_CIA' ausentes em ativo_total_trim
                                           DENOM_CIA            CNPJ_CIA  \
0                                    2W ECOBANK S.A.  08.773.135/0001-00   
1                       3R PETROLEUM ÓLEO E GÁS S.A.  12.091.809/0001-55   
2              AEGEA SANEAMENTO E PARTICIPAÇÕES S.A.  08.827.501/0001-58   
3  AERIS IND. E COM. DE EQUIP. PARA GER. DE ENG. ...  12.528.708/0001-07   
4                            AES BRASIL ENERGIA S.A.  37.663.076/0001-07   

   index   Q1_2023   Q2_2023   Q3_2023  Q4_2023   Q1_2024   Q2_2024  Q3_2024  
0    0.0  0.545812  0.595015  0.637236      0.0  0.000000  0.000000      0.0  
1    1.0  0.047948  0.391814  0.403320      0.0  0.422630  0.440613      0.0  
2    1.0  0.568815  0.570349  0.447560      0.0  0.497304  0.535374      0.0  
3    1.0  0.299436  0.295018  0.267825      0.0  0.223099  0.154928      0.0  
4    1.0  0.589809  0

ROIC = EBIT*(1-T)/ patrimonio líquido + dívida líquida

Histórico

In [81]:
# Verificar se 'DENOM_CIA' e 'CNPJ_CIA' estão presentes nos DataFrames
if 'DENOM_CIA' not in LAIR.columns or 'CNPJ_CIA' not in LAIR.columns:
    print("Colunas 'DENOM_CIA' e 'CNPJ_CIA' ausentes em LAIR")

if 'DENOM_CIA' not in patrimonio_liquido.columns or 'CNPJ_CIA' not in patrimonio_liquido.columns:
    print("Colunas 'DENOM_CIA' e 'CNPJ_CIA' ausentes em patrimonio_liquido")

if 'DENOM_CIA' not in divida_liquida.columns or 'CNPJ_CIA' not in divida_liquida.columns:
    print("Colunas 'DENOM_CIA' e 'CNPJ_CIA' ausentes em divida_liquida")

# Configurar os índices corretos antes de realizar operações
if 'DENOM_CIA' in LAIR.columns and 'CNPJ_CIA' in LAIR.columns:
    LAIR.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)

if 'DENOM_CIA' in patrimonio_liquido.columns and 'CNPJ_CIA' in patrimonio_liquido.columns:
    patrimonio_liquido.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)

if 'DENOM_CIA' in divida_liquida.columns and 'CNPJ_CIA' in divida_liquida.columns:
    divida_liquida.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)

# Garantir que os valores nas colunas sejam numéricos
LAIR = LAIR.apply(pd.to_numeric, errors='coerce')
patrimonio_liquido = patrimonio_liquido.apply(pd.to_numeric, errors='coerce')
divida_liquida = divida_liquida.apply(pd.to_numeric, errors='coerce')

# Substituir NaN por 0
LAIR.fillna(0, inplace=True)
patrimonio_liquido.fillna(0, inplace=True)
divida_liquida.fillna(0, inplace=True)

# Calculando o Lucro Operacional Após Impostos (NOPAT = LAIR * (1 - 0.34))
nopat = LAIR * (1 - 0.34)

# Calculando o Capital Investido (Patrimônio Líquido + Dívida Líquida)
capital_investido = patrimonio_liquido.add(divida_liquida, fill_value=0)

# Evitar divisão por 0 substituindo valores 0 por um valor muito pequeno
capital_investido.replace(0, 1e-10, inplace=True)

# Calculando o ROIC (NOPAT/Capital Investido)
roic = nopat.div(capital_investido)

# Substituir valores inf e NaN resultantes da divisão por 0
roic.replace([np.inf, -np.inf], 0, inplace=True)
roic.fillna(0, inplace=True)

# Resetando o índice para tornar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
roic.reset_index(inplace=True)

# Exibir o DataFrame resultante
print(roic.head())


Colunas 'DENOM_CIA' e 'CNPJ_CIA' ausentes em patrimonio_liquido
Colunas 'DENOM_CIA' e 'CNPJ_CIA' ausentes em divida_liquida
                                           DENOM_CIA            CNPJ_CIA  \
0                                    2W ECOBANK S.A.  08.773.135/0001-00   
1                       3R PETROLEUM ÓLEO E GÁS S.A.  12.091.809/0001-55   
2   ADVANCED DIGITAL HEALTH MEDICINA PREVENTIVA S.A.  10.345.009/0001-98   
3              AEGEA SANEAMENTO E PARTICIPAÇÕES S.A.  08.827.501/0001-58   
4  AERIS IND. E COM. DE EQUIP. PARA GER. DE ENG. ...  12.528.708/0001-07   

       2012      2013      2014      2015      2016      2017      2018  \
0  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
1  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2  0.286632  0.212649  0.210370  0.060852  1.168540  0.333096  0.123040   
3  0.000000  0.140854  0.069613  0.082881  0.083321  0.087580  0.070238   
4  0.000000  0.000000  0.000000  0.000000  0

Trimestral

In [82]:
# Verificar se 'DENOM_CIA' e 'CNPJ_CIA' estão presentes nos DataFrames trimestrais
if 'DENOM_CIA' not in LAIR_trim.columns or 'CNPJ_CIA' not in LAIR_trim.columns:
    print("Colunas 'DENOM_CIA' e 'CNPJ_CIA' ausentes em LAIR_trim")

if 'DENOM_CIA' not in patrimonio_liquido_trim.columns or 'CNPJ_CIA' not in patrimonio_liquido_trim.columns:
    print("Colunas 'DENOM_CIA' e 'CNPJ_CIA' ausentes em patrimonio_liquido_trim")

if 'DENOM_CIA' not in divida_liquida_trim.columns or 'CNPJ_CIA' not in divida_liquida_trim.columns:
    print("Colunas 'DENOM_CIA' e 'CNPJ_CIA' ausentes em divida_liquida_trim")

# Configurar os índices corretos antes de realizar operações
if 'DENOM_CIA' in LAIR_trim.columns and 'CNPJ_CIA' in LAIR_trim.columns:
    LAIR_trim.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)

if 'DENOM_CIA' in patrimonio_liquido_trim.columns and 'CNPJ_CIA' in patrimonio_liquido_trim.columns:
    patrimonio_liquido_trim.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)

if 'DENOM_CIA' in divida_liquida_trim.columns and 'CNPJ_CIA' in divida_liquida_trim.columns:
    divida_liquida_trim.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)

# Garantir que os valores nas colunas sejam numéricos
LAIR_trim = LAIR_trim.apply(pd.to_numeric, errors='coerce')
patrimonio_liquido_trim = patrimonio_liquido_trim.apply(pd.to_numeric, errors='coerce')
divida_liquida_trim = divida_liquida_trim.apply(pd.to_numeric, errors='coerce')

# Substituir NaN por 0
LAIR_trim.fillna(0, inplace=True)
patrimonio_liquido_trim.fillna(0, inplace=True)
divida_liquida_trim.fillna(0, inplace=True)

# Calculando o Lucro Operacional Após Impostos (NOPAT = LAIR * (1 - 0.34))
nopat_trim = LAIR_trim * (1 - 0.34)

# Calculando o Capital Investido (Patrimônio Líquido + Dívida Líquida)
capital_investido_trim = patrimonio_liquido_trim.add(divida_liquida_trim, fill_value=0)

# Evitar divisão por 0 substituindo valores 0 por um valor muito pequeno
capital_investido_trim.replace(0, 1e-10, inplace=True)

# Calculando o ROIC trimestral
roic_trim = nopat_trim.div(capital_investido_trim)

# Substituir valores inf e NaN resultantes da divisão por 0
roic_trim.replace([np.inf, -np.inf], 0, inplace=True)
roic_trim.fillna(0, inplace=True)

# Resetando o índice para tornar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
roic_trim.reset_index(inplace=True)

# Garantir que as colunas de trimestre estejam no formato correto e identificar as colunas de trimestre
colunas_trimestres = [col for col in roic_trim.columns if col.startswith('Q')]

# Ordenar as colunas de trimestre em ordem cronológica
colunas_trimestres_ordenadas = sorted(colunas_trimestres, key=lambda x: (int(x.split('_')[1]), int(x[1])))

# Reordenar o DataFrame, mantendo as colunas 'DENOM_CIA' e 'CNPJ_CIA' no início
roic_trim = roic_trim[['DENOM_CIA', 'CNPJ_CIA'] + colunas_trimestres_ordenadas]

# Exibir o DataFrame resultante
print(roic_trim.head())


Colunas 'DENOM_CIA' e 'CNPJ_CIA' ausentes em patrimonio_liquido_trim
Colunas 'DENOM_CIA' e 'CNPJ_CIA' ausentes em divida_liquida_trim
                                           DENOM_CIA            CNPJ_CIA  \
0                                    2W ECOBANK S.A.  08.773.135/0001-00   
1                       3R PETROLEUM ÓLEO E GÁS S.A.  12.091.809/0001-55   
2              AEGEA SANEAMENTO E PARTICIPAÇÕES S.A.  08.827.501/0001-58   
3  AERIS IND. E COM. DE EQUIP. PARA GER. DE ENG. ...  12.528.708/0001-07   
4                            AES BRASIL ENERGIA S.A.  37.663.076/0001-07   

    Q1_2023   Q2_2023   Q3_2023  Q4_2023   Q1_2024   Q2_2024  Q3_2024  
0  0.042920  0.020092 -0.000867      0.0  0.000000  0.000000      0.0  
1  0.010259  0.003933  0.030922      0.0  0.024170  0.049369      0.0  
2  0.027762  0.023483  0.039173      0.0  0.036582  0.062973      0.0  
3  0.028652  0.027624  0.022313      0.0  0.005150  0.023097      0.0  
4  0.009904  0.008212  0.010759      0.0  0.00655

Alavancagem financeira = atvo total / patrimonio líquido

Histórico

In [83]:
# Verificar se 'DENOM_CIA' e 'CNPJ_CIA' estão presentes nos DataFrames históricos
if 'DENOM_CIA' not in ativo_total.columns or 'CNPJ_CIA' not in ativo_total.columns:
    print("Colunas 'DENOM_CIA' e 'CNPJ_CIA' ausentes em ativo_total")

if 'DENOM_CIA' not in patrimonio_liquido.columns or 'CNPJ_CIA' not in patrimonio_liquido.columns:
    print("Colunas 'DENOM_CIA' e 'CNPJ_CIA' ausentes em patrimonio_liquido")

# Configurar os índices corretos antes de realizar operações
if 'DENOM_CIA' in ativo_total.columns and 'CNPJ_CIA' in ativo_total.columns:
    ativo_total.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)

if 'DENOM_CIA' in patrimonio_liquido.columns and 'CNPJ_CIA' in patrimonio_liquido.columns:
    patrimonio_liquido.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)

# Garantir que os valores nas colunas sejam numéricos
ativo_total = ativo_total.apply(pd.to_numeric, errors='coerce')
patrimonio_liquido = patrimonio_liquido.apply(pd.to_numeric, errors='coerce')

# Substituir NaN por 0
ativo_total.fillna(0, inplace=True)
patrimonio_liquido.fillna(0, inplace=True)

# Evitar divisão por 0 substituindo valores 0 por um valor muito pequeno
patrimonio_liquido.replace(0, 1e-10, inplace=True)

# Calculando a Alavancagem Financeira (Ativo Total / Patrimônio Líquido)
alavancagem_financeira = ativo_total.div(patrimonio_liquido)

# Substituir valores inf e NaN resultantes da divisão por 0
alavancagem_financeira.replace([np.inf, -np.inf], 0, inplace=True)
alavancagem_financeira.fillna(0, inplace=True)

# Resetar o índice para exibir as colunas corretamente
alavancagem_financeira.reset_index(inplace=True)

# Exibir o DataFrame resultante
print(alavancagem_financeira.head())


Colunas 'DENOM_CIA' e 'CNPJ_CIA' ausentes em ativo_total
Colunas 'DENOM_CIA' e 'CNPJ_CIA' ausentes em patrimonio_liquido
                                           DENOM_CIA            CNPJ_CIA  \
0                                    2W ECOBANK S.A.  08.773.135/0001-00   
1                       3R PETROLEUM ÓLEO E GÁS S.A.  12.091.809/0001-55   
2   ADVANCED DIGITAL HEALTH MEDICINA PREVENTIVA S.A.  10.345.009/0001-98   
3              AEGEA SANEAMENTO E PARTICIPAÇÕES S.A.  08.827.501/0001-58   
4  AERIS IND. E COM. DE EQUIP. PARA GER. DE ENG. ...  12.528.708/0001-07   

   index      2012      2013      2014      2015      2016      2017  \
0    0.0  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
1    1.0  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2    1.0 -2.254106 -0.341262 -0.088753 -0.187737 -0.649811 -0.258255   
3    1.0  0.000000  2.467090  2.890441  3.164940  3.227103  4.611920   
4    1.0  0.000000  0.000000  0.000000  0.000000  0.000000  0.

Trimestral

In [84]:
# Verificar se 'DENOM_CIA' e 'CNPJ_CIA' estão presentes nos DataFrames trimestrais
if 'DENOM_CIA' not in ativo_total_trim.columns or 'CNPJ_CIA' not in ativo_total_trim.columns:
    print("Colunas 'DENOM_CIA' e 'CNPJ_CIA' ausentes em ativo_total_trim")

if 'DENOM_CIA' not in patrimonio_liquido_trim.columns or 'CNPJ_CIA' not in patrimonio_liquido_trim.columns:
    print("Colunas 'DENOM_CIA' e 'CNPJ_CIA' ausentes em patrimonio_liquido_trim")

# Configurar os índices corretos antes de realizar operações
if 'DENOM_CIA' in ativo_total_trim.columns and 'CNPJ_CIA' in ativo_total_trim.columns:
    ativo_total_trim.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)

if 'DENOM_CIA' in patrimonio_liquido_trim.columns and 'CNPJ_CIA' in patrimonio_liquido_trim.columns:
    patrimonio_liquido_trim.set_index(['DENOM_CIA', 'CNPJ_CIA'], inplace=True)

# Garantir que os valores nas colunas sejam numéricos
ativo_total_trim = ativo_total_trim.apply(pd.to_numeric, errors='coerce')
patrimonio_liquido_trim = patrimonio_liquido_trim.apply(pd.to_numeric, errors='coerce')

# Substituir NaN por 0
ativo_total_trim.fillna(0, inplace=True)
patrimonio_liquido_trim.fillna(0, inplace=True)

# Evitar divisão por 0 substituindo valores 0 por um valor muito pequeno
patrimonio_liquido_trim.replace(0, 1e-10, inplace=True)

# Calculando a Alavancagem Financeira (Ativo Total / Patrimônio Líquido) para os dados trimestrais
alavancagem_financeira_trim = ativo_total_trim.div(patrimonio_liquido_trim)

# Substituir valores inf e NaN resultantes da divisão por 0
alavancagem_financeira_trim.replace([np.inf, -np.inf], 0, inplace=True)
alavancagem_financeira_trim.fillna(0, inplace=True)

# Resetando o índice para tornar 'DENOM_CIA' e 'CNPJ_CIA' em colunas novamente
alavancagem_financeira_trim.reset_index(inplace=True)

# Exibir o DataFrame resultante
print(alavancagem_financeira_trim.head())


Colunas 'DENOM_CIA' e 'CNPJ_CIA' ausentes em ativo_total_trim
Colunas 'DENOM_CIA' e 'CNPJ_CIA' ausentes em patrimonio_liquido_trim
                                           DENOM_CIA            CNPJ_CIA  \
0                                    2W ECOBANK S.A.  08.773.135/0001-00   
1                       3R PETROLEUM ÓLEO E GÁS S.A.  12.091.809/0001-55   
2              AEGEA SANEAMENTO E PARTICIPAÇÕES S.A.  08.827.501/0001-58   
3  AERIS IND. E COM. DE EQUIP. PARA GER. DE ENG. ...  12.528.708/0001-07   
4                            AES BRASIL ENERGIA S.A.  37.663.076/0001-07   

   index    Q1_2023    Q2_2023    Q3_2023  Q4_2023   Q1_2024   Q2_2024  \
0    0.0  16.233536  16.151360  23.866689      0.0  0.000000  0.000000   
1    1.0   1.859495   3.556064   3.717365      0.0  3.832668  4.397338   
2    1.0   3.181249   3.438372   5.299705      0.0  6.167143  6.750050   
3    1.0   3.812735   3.773135   4.688683      0.0  3.165700  3.339713   
4    1.0   4.548658   4.528327   4.435781 

Número de ações

Obtenção dos relatórios dos formulários de referencia (FRE). Há uma url para os relatórios dos ultimos 5 anos e uma url para os relatórios mais antigos, por isso a necessidade de separa em 2 urls os requerimentos.

In [89]:
# URL base dos relatórios até 2018
url_base_mais_5_anos = "https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/FRE/DADOS/"

# URL base dos relatórios a partir de 2019
url_base_ultimos_5_anos = "https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/FRE/DADOS/"

# Período
ano_atual = datetime.now().year
rel_anteriores = range(ano_atual - 11, ano_atual - 4)  # Anos anteriores a 2019
rel_atuais = range(ano_atual - 4, ano_atual + 1)       # Anos a partir de 2019

# Diretório de destino para download
diretorio_download = r"C:\Users\User\Desktop\valuation_cvm\valuation_cvm_clone\exports\Formulários de referencia (FRE)Download"
diretorio_descompactacao = r"C:\Users\User\Desktop\valuation_cvm\valuation_cvm_clone\exports\Formulários de referencia (FRE)"

# Criando os diretórios se não existirem
os.makedirs(diretorio_download, exist_ok=True)
os.makedirs(diretorio_descompactacao, exist_ok=True)

# Função para baixar e descompactar
def baixar_descompactar(ano, url_base):
    caminho_arquivo_zip = os.path.join(diretorio_download, f"FRE_CIA_ABERTA_{ano}.zip")

    # Verificar se já existe o arquivo
    if os.path.exists(caminho_arquivo_zip):
        print(f"O arquivo para {ano} já foi baixado. Pulando para o próximo ano.")
    else:
        # Baixar o arquivo
        url_ano = f"{url_base}FRE_CIA_ABERTA_{ano}.zip"
        response = requests.get(url_ano, stream=True)
        
        if response.status_code == 200:
            with open(caminho_arquivo_zip, 'wb') as file, tqdm(
                desc=f"Fazendo download de {ano}",
                total=int(response.headers.get('content-length', 0)),
                unit='B', unit_scale=True, unit_divisor=1024,
            ) as bar:
                for data in response.iter_content(chunk_size=1024):
                    bar.update(len(data))
                    file.write(data)
            print(f"Download concluído para {ano}.")
        else:
            print(f"Falha ao obter o arquivo para {ano}. Status: {response.status_code}")
            return

    # Diretório para descompactação
    diretorio_ano_descompactado = os.path.join(diretorio_descompactacao, str(ano))
    if os.path.exists(diretorio_ano_descompactado):
        print(f"Os arquivos para o ano {ano} já foram descompactados. Pulando para o próximo ano.")
    else:
        os.makedirs(diretorio_ano_descompactado, exist_ok=True)
        try:
            with zipfile.ZipFile(caminho_arquivo_zip, 'r') as zip_ref:
                zip_ref.extractall(diretorio_ano_descompactado)
            print(f"Descompactação concluída para {ano}.")
        except FileNotFoundError:
            print(f"Erro: O arquivo {caminho_arquivo_zip} não foi encontrado para descompactação.")
        except zipfile.BadZipFile:
            print(f"Erro: O arquivo {caminho_arquivo_zip} está corrompido.")

# Executa o processo para os anos anteriores a 2019
for ano in rel_anteriores:
    baixar_descompactar(ano, url_base_mais_5_anos)

# Executa o processo para os anos a partir de 2019
for ano in rel_atuais:
    baixar_descompactar(ano, url_base_ultimos_5_anos)

print("Downloads e descompactações concluídos.")


O arquivo para 2013 já foi baixado. Pulando para o próximo ano.
Os arquivos para o ano 2013 já foram descompactados. Pulando para o próximo ano.
O arquivo para 2014 já foi baixado. Pulando para o próximo ano.
Os arquivos para o ano 2014 já foram descompactados. Pulando para o próximo ano.
O arquivo para 2015 já foi baixado. Pulando para o próximo ano.
Os arquivos para o ano 2015 já foram descompactados. Pulando para o próximo ano.
O arquivo para 2016 já foi baixado. Pulando para o próximo ano.
Os arquivos para o ano 2016 já foram descompactados. Pulando para o próximo ano.
O arquivo para 2017 já foi baixado. Pulando para o próximo ano.
Os arquivos para o ano 2017 já foram descompactados. Pulando para o próximo ano.
O arquivo para 2018 já foi baixado. Pulando para o próximo ano.
Os arquivos para o ano 2018 já foram descompactados. Pulando para o próximo ano.
O arquivo para 2019 já foi baixado. Pulando para o próximo ano.
Os arquivos para o ano 2019 já foram descompactados. Pulando para 

Fazendo download de 2020: 100%|██████████| 8.81M/8.81M [00:00<00:00, 18.7MB/s]


Download concluído para 2020.
Os arquivos para o ano 2020 já foram descompactados. Pulando para o próximo ano.


Fazendo download de 2021: 100%|██████████| 10.0M/10.0M [00:00<00:00, 17.9MB/s]


Download concluído para 2021.
Os arquivos para o ano 2021 já foram descompactados. Pulando para o próximo ano.


...

Download concluído para 2022.
Descompactação concluída para 2022.


Fazendo download de 2023: 100%|██████████| 7.76M/7.76M [00:00<00:00, 19.4MB/s]


Download concluído para 2023.
Descompactação concluída para 2023.


Fazendo download de 2024: 100%|██████████| 7.71M/7.71M [00:00<00:00, 17.7MB/s]


Download concluído para 2024.
Descompactação concluída para 2024.
Downloads e descompactações concluídos.


In [92]:
# Lista com todos os arquivos no diretório disponíveis para visualização
diretorio = r"C:\Users\User\Desktop\valuation_cvm\valuation_cvm_clone\exports\Formulários de referencia (FRE)\2018"
arquivos_disponiveis = os.listdir(diretorio)

# Exibe a lista de arquivos disponíveis
for arquivo in arquivos_disponiveis:
    print(arquivo)

fre_cia_aberta_2018.csv
fre_cia_aberta_administrador_membro_conselho_fiscal_2018.csv
fre_cia_aberta_ativo_imobilizado_2018.csv
fre_cia_aberta_ativo_intangivel_2018.csv
fre_cia_aberta_auditor_2018.csv
fre_cia_aberta_auditor_responsavel_2018.csv
fre_cia_aberta_capital_social_2018.csv
fre_cia_aberta_capital_social_aumento_2018.csv
fre_cia_aberta_capital_social_aumento_classe_acao_2018.csv
fre_cia_aberta_capital_social_classe_acao_2018.csv
fre_cia_aberta_capital_social_desdobramento_2018.csv
fre_cia_aberta_capital_social_desdobramento_classe_acao_2018.csv
fre_cia_aberta_capital_social_reducao_2018.csv
fre_cia_aberta_capital_social_reducao_classe_acao_2018.csv
fre_cia_aberta_capital_social_titulo_conversivel_2018.csv
fre_cia_aberta_direito_acao_2018.csv
fre_cia_aberta_distribuicao_capital_2018.csv
fre_cia_aberta_distribuicao_capital_classe_acao_2018.csv
fre_cia_aberta_distribuicao_dividendos_2018.csv
fre_cia_aberta_distribuicao_dividendos_classe_acao_2018.csv
fre_cia_aberta_endividamento_20

In [94]:
# Visualizando relatório a ser aberto
relatorio = "fre_cia_aberta_auditor_responsavel"  # Inserir apenas o nome do arquivo, sem o ano
ano = 2018

# Definindo o caminho do arquivo usando formatação de string
caminho_arquivo = f"C:/Users/User/Desktop/valuation_cvm/valuation_cvm_clone/exports/Formulários de referencia (FRE)/{ano}/{relatorio}_{ano}.csv"

# Carregando o arquivo no DataFrame
df = pd.read_csv(caminho_arquivo, encoding='cp1252', sep=';')
df.head(30)

,CNPJ_Companhia,Data_Referencia,Versao,ID_Documento,Nome_Companhia,ID_Auditor,Data_Inicio_Responsavel_Tecnico,Data_Fim_Responsavel_Tecnico,Responsavel_Tecnico,CPF_Responsavel_Tecnico,...,Cidade,Sigla_UF,UF,Pais,CEP,DDI_Telefone,DDD_Telefone,Telefone,Fax,Email
0,00.000.000/0001-91,2018-01-01,28,82919,BCO BRASIL S.A.,74937,2016-03-21,2019-03-21,MARCELO FARIA PEREIRA,1.351498e+09,...,Brasília,DF,Distrito Federal,Brasil,70070120.0,NaN,61.0,21042406.0,21042406.0,mfpereira@kpmg.com.br
1,00.000.000/0001-91,2018-01-01,28,82919,BCO BRASIL S.A.,74938,2019-03-22,NaN,Luiz Carlos Oseliero Filho,2.734357e+10,...,Brasília,DF,Distrito Federal,Brasil,70308200.0,NaN,61.0,51861224.0,NaN,luizoseliero@deloitte.com
2,00.000.208/0001-00,2018-01-01,24,86871,BRB BANCO DE BRASILIA S.A.,78624,2015-04-06,NaN,Dario Ramos da Cunha,1.345012e+10,...,SAO PAULO,SP,São Paulo,Brasil,4543900.0,NaN,11.0,25736421.0,25735421.0,judidico.sp@br.ey.com
3,00.001.180/0001-26,2018-01-01,23,83958,CENTRAIS ELET BRAS S.A. - ELETROBRAS,75780,2014-01-01,2019-04-01,Danilo Siman Simões,5.240531e+10,...,Rio de Janeiro,RJ,Rio de Janeiro,Brasil,20031000.0,NaN,21.0,35159400.0,35159000.0,dsimoes@kpmg.com.br
4,00.001.180/0001-26,2018-01-01,23,83958,CENTRAIS ELET BRAS S.A. - ELETROBRAS,75781,2019-04-02,NaN,GUILHERME NAVES VALLE,5.419916e+10,...,Rio de Janeiro,RJ,Rio de Janeiro,Brasil,22210907.0,NaN,21.0,32326112.0,NaN,Guilherme.valle@pwc.com
5,00.070.698/0001-11,2018-01-01,13,83120,COMPANHIA ENERGÉTICA DE BRASÍLIA - CEB,75158,2014-06-20,NaN,Alfredo Ferreira Marques Filho,2.823650e+09,...,São Paulo,SP,São Paulo,Brasil,1050030.0,NaN,11.0,38485880.0,38485880.0,Alfredo.marques@bdobrazil.com.br
6,00.115.890/0001-87,2018-01-01,1,75659,SHOPPING CENTER TACARUNA SA,67647,2014-04-01,NaN,Francisco da Silva Pimentel,3.008277e+10,...,Recife,PE,Pernambuco,Brasil,51020280.0,NaN,81.0,32014800.0,32014819.0,francisco.s.pimentel@br.ey.com
7,00.192.769/0001-59,2018-01-01,1,75436,NEUMARKT TRADE AND FINANCIAL CENTER S/A,67229,2017-04-01,NaN,OSVALDO MENDES,7.101105e+09,...,São Paulo,SP,São Paulo,Brasil,1311931.0,NaN,11.0,31202436.0,NaN,om-mendes@iaud.com.br
8,00.272.185/0001-93,2018-01-01,3,80707,CIMS S.A.,72950,2007-01-01,2011-12-31,JOSÉ RENATO MENDONÇA,8.390194e+09,...,Rio de Janeiro,RJ,Rio de Janeiro,Brasil,22440032.0,NaN,21.0,32622500.0,32625133.0,llaydner@vincipartners.com
9,00.272.185/0001-93,2018-01-01,3,80707,CIMS S.A.,72951,2012-01-01,2016-12-31,Flávio Serpejante Peppe,1.250902e+10,...,Rio de Janeiro,RJ,Rio de Janeiro,Brasil,22440032.0,NaN,21.0,32622500.0,32625133.0,llaydner@vincipartners.com


Número de ações emitidas

In [95]:
# Definindo relatório a ser aberto
anos = range(ano_inicial, ano_final+1)

# Criando um DataFrame vazio para armazenar os dados
df_numero_acoes = pd.DataFrame()

# Iterando pelos anos
for ano in anos:
    # Construindo o caminho do arquivo
    caminho_arquivo = f'C:/Users/User/Desktop/valuation_cvm/valuation_cvm_clone/exports/Formulários de referencia (FRE)/{ano}/fre_cia_aberta_capital_social_{ano}.csv'

    # Verificando se o arquivo existe antes de tentar lê-lo
    if os.path.exists(caminho_arquivo):
        # Lendo o arquivo CSV
        df = pd.read_csv(caminho_arquivo, encoding='cp1252', sep=';')

        # Filtrando o DataFrame para as colunas desejadas e o Tipo_Capital correto
        df_filtrado = df[df['Tipo_Capital'] == 'Capital Emitido'][['CNPJ_Companhia', 'Quantidade_Total_Acoes']]

        # Agrupando por CNPJ e somando as quantidades de ações para cada CNPJ
        df_filtrado = df_filtrado.groupby('CNPJ_Companhia')['Quantidade_Total_Acoes'].sum().reset_index()

        # Renomeando a coluna Quantidade_Total_Acoes para o ano correspondente
        df_filtrado = df_filtrado.rename(columns={'Quantidade_Total_Acoes': str(ano)})

        # Adicionando os dados ao DataFrame final
        if df_numero_acoes.empty:
            df_numero_acoes = df_filtrado
        else:
            df_numero_acoes = pd.merge(df_numero_acoes, df_filtrado, on='CNPJ_Companhia', how='outer')

df_numero_acoes

# Substituindo os valores NaN pelo valor do ano anterior
df_numero_acoes.fillna(method='ffill', inplace=True)



C:\Users\User\AppData\Local\Temp\ipykernel_25252\925860266.py:35: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_numero_acoes.fillna(method='ffill', inplace=True)


Salvando relatórios no diretório

In [96]:
# Diretório base onde os arquivos CSV serão salvos
diretorio_base = diretorio_rel_ind_financeiros

# Salvando os DataFrames
fcf_rl.to_csv(os.path.join(diretorio_base, 'fcf_rl.csv'), sep=';', index=False)
fcf_rl_trim.to_csv(os.path.join(diretorio_base, 'fcf_rl_trim.csv'), sep=';', index=False)
ll_sobre_rl.to_csv(os.path.join(diretorio_base, 'll_sobre_rl.csv'), sep=';', index=False)
ll_sobre_rl_trim.to_csv(os.path.join(diretorio_base, 'll_sobre_rl_trim.csv'), sep=';', index=False)
divida_sobre_ebitda.to_csv(os.path.join(diretorio_base, 'divida_sobre_ebitda.csv'), sep=';', index=False)
divida_sobre_ebitda_trim.to_csv(os.path.join(diretorio_base, 'divida_sobre_ebitda_trim.csv'), sep=';', index=False)
roe.to_csv(os.path.join(diretorio_base, 'roe.csv'), sep=';', index=False)
roe_trim.to_csv(os.path.join(diretorio_base, 'roe_trim.csv'), sep=';', index=False)
roa.to_csv(os.path.join(diretorio_base, 'roa.csv'), sep=';', index=False)
roa_trim.to_csv(os.path.join(diretorio_base, 'roa_trim.csv'), sep=';', index=False)
debt_to_equity.to_csv(os.path.join(diretorio_base, 'debt_to_equity.csv'), sep=';', index=False)
debt_to_equity_trim.to_csv(os.path.join(diretorio_base, 'debt_to_equity_trim.csv'), sep=';', index=False)
debt_to_total_assets.to_csv(os.path.join(diretorio_base, 'debt_to_total_assets.csv'), sep=';', index=False)
debt_to_total_assets_trim.to_csv(os.path.join(diretorio_base, 'debt_to_total_assets_trim.csv'), sep=';', index=False)
roic.to_csv(os.path.join(diretorio_base, 'roic.csv'), sep=';', index=False)
roic_trim.to_csv(os.path.join(diretorio_base, 'roic_trim.csv'), sep=';', index=False)
alavancagem_financeira.to_csv(os.path.join(diretorio_base, 'alavancagem_financeira.csv'), sep=';', index=False)
alavancagem_financeira_trim.to_csv(os.path.join(diretorio_base, 'alavancagem_financeira_trim.csv'), sep=';', index=False)
df_numero_acoes.to_csv(os.path.join(diretorio_base, 'df_numero_acoes.csv'), sep=';', index=False)


Lucro por ação (DRE)

In [97]:
# Definindo relatório a ser aberto
relatorio = "dfp_cia_aberta_DRE_con"  # inserir apenas o nome do arquivo, sem o ano

# Lista de anos desejados
anos = list(range(ano_inicial, ano_final+1))

# DataFrame vazio para armazenar os resultados
resultados_dre = pd.DataFrame()

# Loop pelos anos
for ano in anos:
    # Definindo o caminho do arquivo usando formatação de string
    caminho_arquivo = f'{diretorio_destino_arq_csv}/{ano}/{relatorio}_{ano}.csv'

    # Lendo o arquivo CSV
    df = pd.read_csv(caminho_arquivo, encoding='utf-8', sep=',')

    # Filtrando os dados para encontrar o Lucro Líquido do Período
    var_DRE = df[(df['ORDEM_EXERC'] == 'ÚLTIMO') & (df['CD_CONTA'] == '3.99')]

    # Selecionando colunas desejadas
    var_DRE = var_DRE[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA']]

    # Adicionando a coluna 'Ano' com o valor correspondente
    var_DRE['Ano'] = ano

    # Adicionando os resultados ao DataFrame final
    resultados_dre = pd.concat([resultados_dre, var_DRE], ignore_index=True)

# Agregando os valores duplicados
resultados_agregados_dre = resultados_dre.groupby(['DENOM_CIA', 'CNPJ_CIA', 'Ano'])['VL_CONTA'].sum().reset_index()

# Criando um DataFrame pivotado
lucro_acao = resultados_agregados_dre.pivot_table(index=['DENOM_CIA', 'CNPJ_CIA'], columns='Ano', values='VL_CONTA', aggfunc='sum').reset_index()

# Renomeando as colunas para remover o nome da coluna do índice
lucro_acao.name = None


lucro_acao

Ano,DENOM_CIA,CNPJ_CIA,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,2W ECOBANK S.A.,08.773.135/0001-00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN
1,3R PETROLEUM ÓLEO E GÁS S.A.,12.091.809/0001-55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
2,ADVANCED DIGITAL HEALTH MEDICINA PREVENTIVA S.A.,10.345.009/0001-98,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
3,AEGEA SANEAMENTO E PARTICIPAÇÕES S.A.,08.827.501/0001-58,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,AERIS IND. E COM. DE EQUIP. PARA GER. DE ENG. ...,12.528.708/0001-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
651,YOU INC INCORPORADORA E PARTICIPAÇÃO S.A,11.284.204/0001-18,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
652,YUNY INCORPORADORA HOLDING S.A.,09.267.996/0001-70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
653,ZAMP SA,13.574.594/0001-96,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
654,ÉVORA SA,91.820.068/0001-72,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Vendas Líquidas

In [98]:
# Definindo relatório a ser aberto
relatorio = "dfp_cia_aberta_DRE_ind"  # inserir apenas o nome do arquivo, sem o ano

# Lista de anos desejados
anos = list(range(ano_inicial, ano_final+1))

# DataFrame vazio para armazenar os resultados
resultados_dre = pd.DataFrame()

# Loop pelos anos
for ano in anos:
    # Definindo o caminho do arquivo usando formatação de string
    caminho_arquivo = f'{diretorio_destino_arq_csv}/{ano}/{relatorio}_{ano}.csv'

    # Lendo o arquivo CSV
    df = pd.read_csv(caminho_arquivo, encoding='utf-8', sep=',')

    # Filtrando os dados para encontrar o Lucro Líquido do Período
    var_DRE = df[(df['ORDEM_EXERC'] == 'ÚLTIMO') & (df['CD_CONTA'] == '3.01.01')|(df['CD_CONTA']=='3.01.02')]

    # Selecionando colunas desejadas
    var_DRE = var_DRE[['DENOM_CIA', 'CNPJ_CIA', 'VL_CONTA']]

    # Adicionando a coluna 'Ano' com o valor correspondente
    var_DRE['Ano'] = ano

    # Adicionando os resultados ao DataFrame final
    resultados_dre = pd.concat([resultados_dre, var_DRE], ignore_index=True)

# Agregando os valores duplicados
resultados_agregados_dre = resultados_dre.groupby(['DENOM_CIA', 'CNPJ_CIA', 'Ano'])['VL_CONTA'].sum().reset_index()

# Criando um DataFrame pivotado
vendas_liquidas = resultados_agregados_dre.pivot_table(index=['DENOM_CIA', 'CNPJ_CIA'], columns='Ano', values='VL_CONTA', aggfunc='sum').reset_index()

# Renomeando as colunas para remover o nome da coluna do índice
vendas_liquidas.name = None


vendas_liquidas

Ano,DENOM_CIA,CNPJ_CIA,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,ACRUX SECURITIZADORA S/A,07.825.881/0001-29,60060.0,84993.0,97947.0,33219.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ALUPAR INVESTIMENTO S/A,08.364.948/0001-38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90318.0,NaN,NaN,NaN
2,AMPLA ENERGIA E SERVICOS S.A.,33.050.071/0001-58,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ARGO TRANSMISSÃO DE ENERGIA S/A,24.624.490/0001-65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1274404.0,882365.0,752248.0,740456.0
4,ARGO V TRANSMISSÃO DE ENERGIA S.A.,20.514.590/0001-88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95992.0,122988.0,167066.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,XX DE NOVEMBRO INVESTIMENTOS E PARTICIPAÇÕES S.A.,03.538.833/0001-07,23743.0,17169.0,20593.0,14093.0,11665.0,-451.0,0.0,0.0,10284.0,10931.0,9063.0,8229.0
200,YUNY INCORPORADORA HOLDING S.A.,09.267.996/0001-70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7100.0
201,ÁGUAS DO SERTÃO S.A.,45.456.117/0001-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55782.0,216535.0
202,ÁQUILLA SECURITIZADORA S.A.,09.439.783/0001-89,19123914.0,18453016.0,17265829.0,16799523.0,17394020.0,15604537.0,9915382.0,NaN,NaN,NaN,NaN,NaN
